# Research data supporting "Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory"

This notebook accompanies our paper: **Reliable Insights into Surface Chemistry Enabled by Combining Experiments with Fast and Accurate Theory**. It can be found on GitHub at https://github.com/benshi97/Data_Atomistic_Insights and explored interactively on [Colab](https://colab.research.google.com/github/benshi97/Data_Atomistic_Insights/blob/main/analyse.ipynb).

### Abstract

<!-- The adsorption energy of a molecule onto the surface of a material underpins a wide array of applications, spanning heterogeneous catalysis, gas storage and many more. It is the key quantity where experimental measurements and theoretical calculations meet, with agreement being necessary for reliable predictions of reaction rates and mechanisms. The prototypical molecule-surface system is CO adsorbed on MgO, but despite intense scrutiny from theory and experiment, there is still no consensus on its adsorption energy. In particular, the large cost of accurate many-body methods makes reaching converged theoretical estimates difficult, generating a wide range of values. In this work, we address this challenge, leveraging the latest advances in diffusion Monte Carlo (DMC) and coupled cluster theory [CCSD(T)], to obtain accurate predictions for CO on MgO. These reliable theoretical estimates allow us to evaluate the inconsistencies in published temperature programmed desorption experiments, revealing that they arise from variations in employed pre-exponential factors. Using this insight, we derive new experimental estimates of the (electronic) adsorption energy from a more precise pre-exponential factor. With this effort, we are able to reach consensus between multiple theoretical calculations and multiple experiments for the first time. In addition, we show that our recently developed cluster-based CCSD(T) approach provides a low cost route towards achieving accurate adsorption energies. This sets the stage for affordable and reliable theoretical predictions of reaction mechanisms and rates to guide the realization of new catalysts and gas storage materials. -->


## Contents
* [Table S1 and Figure 1 - Past CO on MgO *E*<sub>ads</sub> and Current Work](#tables1fig1)
* [Table S2 and S3 - Validating the revPBE-D4 geometry and Computing its $\Delta_\textrm{geom}$](#tables2s3)
* [Table S4 and S5 - Periodic CCSD(T) Convergence](#tables4)
* [Table S6 - Periodic DMC Convergence](#tables6)
* [Table S7 - DFT Convergence](#tables7)
* [Table S9 to S11 and Figure 2 - SKZCAM Cluster CCSD(T) Convergence](#tables9-s11)
* [Table S8 - Final periodic CCSD(T), periodic DMC and cluster CCSD(T) $E_\textrm{ads}$ and their individual contributions](#tables8)
* [Table S12 - Analysis of previous computational work on CO on MgO](#tables12)
* [Table S13 - Computation of zero-point energy and thermal contribution terms to convert $H_\textrm{ads}$ to $E_\textrm{ads}$](#tables13)
* [Figure 3 and S1 - Converting previous experiment $H_\textrm{ads}$ or $E_\textrm{act}$ (for TPD) to $E_\textrm{ads}$](#fig3)
* [Table S14 - Final best estimate of TPD experimental adsorption energies](#tables14)
* [Table S15 - Effect of CO coverage on $E_\textrm{ads}$](#tables15)


In [1]:
# Import necessary functions
from Scripts.analysis_scripts import *
from Scripts.plot_scripts import *

# Check if we are in Google Colab environment
try:
    import google.colab
    IN_COLAB = True
    usetex = False
    texfalse_import()

except:
    import os
    IN_COLAB = False
    if os.path.expanduser('~') == '/home/shixubenjamin':
        usetex = True
        textrue_import()
    else:
        usetex = False    
        texfalse_import()

# If in Google Colab, install the necessary data and set up the necessary environment
if IN_COLAB == True:
    !rm -rf /content/Data_CO_on_MgO-main /content/main.zip
    !wget https://github.com/benshi97/Data_CO_on_MgO/archive/refs/heads/main.zip
    !unzip /content/main.zip
    ! apt install ase
    !pip install pyblock datetime
    %pwd
    %cd /content/Data_CO_on_MgO-main

In [2]:
# Define the systems to be analysed

# The ensemble of DFT exchange-correlation functionals used according to crystal system
crystal_xc_func_ensemble = {
    'MgO': ["01_PBE-D2-Ne", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_PBE0-D4", "06_B3LYP-D2-Ne"],
    'r-TiO2': ["01_PBE-U-TSHI", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_R2SCAN-rVV10", "06_HSE06-D4"],
    'a-TiO2': ["01_PBE-U-TSHI", "02_revPBE-D4", "03_vdW-DF", "04_rev-vdW-DF2", "05_R2SCAN-rVV10", "06_HSE06-D4"]
}

# All the molecule-surface systems to be analysed
molecule_surface_systems = {
    'MgO': ['CH4', 'CH4 Monolayer', 'C2H6', 'C2H6 Monolayer', 'CO', 'C6H6', 'N2O Parallel', 'N2O Tilted', 'NO Vertical-Hollow', 'NO Vertical-Mg', 'NO Bent-Bridge', 'NO Bent-Mg', 'NO Bent-O', 'NO Dimer', 'H2O Monomer', 'H2O Tetramer', 'CH3OH Tilted', 'CH3OH Parallel', 'CH3OH Tetramer', 'NH3', 'CO2 Physisorbed', 'CO2 Chemisorbed'],
    'MgO_8x8': ['C6H6', 'H2O Tetramer', 'CH3OH Tetramer'], # Systems that use a 8x8 MgO supercell rather than the default 4x4
    'r-TiO2': ['CH4','CO2 Parallel','CO2 Tilted','H2O','CH3OH'],
    'a-TiO2': ['H2O','NH3']
}

# A dictionary which converts the DFT model, surface and adsorbate dictionary labels to LaTeX formatted labels
convert_to_nice_labels = {
    'xc_functionals': {
        '01_PBE-D2-Ne': 'PBE-D2[Ne]',
        '02_revPBE-D4': 'revPBE-D4',
        '03_vdW-DF': 'vdW-DF',
        '04_rev-vdW-DF2': 'rev-vdW-DF2',
        '05_PBE0-D4': 'PBE0-D4',
        '06_B3LYP-D2-Ne': 'B3LYP-D2[Ne]',
        '01_PBE-U-TSHI': 'PBE+U-TS/HI',
        '05_R2SCAN-rVV10': r'r$^2$SCAN-rVV10',
        '06_HSE06-D4': 'HSE06-D4'
    },
    'surface': {
        'MgO': 'MgO (001)',
        'r-TiO2': r'TiO$_2$ rutile (110)',
        'a-TiO2': r'TiO$_2$ anatase (101)'
    },
    'adsorbates': {
        surface: {
            molecule: re.sub(r'(\d+)', r'$_\1$', molecule.split()[0])
            if len(molecule.split()) == 1 else
            molecule.split()[1] + ' ' + re.sub(r'(\d+)', r'$_\1$', molecule.split()[0])
            for molecule in molecules
        }
        for surface, molecules in molecule_surface_systems.items()
    }
}

# *H*<sub>ads</sub> from the experimental literature

In [3]:
expt_hads_df = pd.read_csv('Data/Miscellaneous/Experimental_Data.csv',dtype=object)
expt_hads_df = expt_hads_df.reset_index(drop=True)

In [4]:
# Convert 'Experimental_Data.csv' csv file to pandas DataFrame

expt_hads_df = pd.read_csv('Data/Miscellaneous/Experimental_Data.csv',dtype=object)
expt_hads_df = expt_hads_df.reset_index(drop=True)

# Convert to latex table
convert_df_to_latex_input(
df = expt_hads_df,
start_input = '\\begin{turnpage}\n\\begin{table}\n',
end_input = '\n\\end{table}\n\\end{turnpage}',
label = "tab:expt_hads",
caption = "Experimental adsorption enthalpies for the systems studied within this work.",
replace_input = {
    "+-": r"$\pm$"
},
adjustbox = 1.6,
column_format = "lrrrrrp{12cm}",
index = False,
filename = 'Tables/SI_Table-Experimental_Hads.tex')

expt_and_autoskzcam_hads_df = expt_hads_df.copy()

expt_hads_df.columns = ['Surface','Molecule','Temperature','log(ν)', 'Hads', 'Error', 'Details']
expt_hads_df = expt_hads_df.replace({
    r'\\ce\{|\}|\(001\)|\(110\)|\(101\)|\(\)': '',
    r'\$\\to\$': '->',
    r'\+-': '±',
}, regex=True)

expt_hads_df = expt_hads_df.map(lambda x: x.strip() if isinstance(x, str) else x)
expt_hads_df['Surface'] = expt_hads_df['Surface'].replace({'TiO2 rutile': 'r-TiO2', 'TiO2 anatase': 'a-TiO2'})
expt_hads_df

,Surface,Molecule,Temperature,log(ν),Hads,Error,Details
0,MgO,CH4,47,13.1 ± 2.0,-115,19,Dilute limit $E_\textrm{d$ estimate by Tait \e...
1,MgO,C2H6,75,14.9 ± 2.0,-221,30,Dilute limit $E_\textrm{d$ estimate by Tait \e...
2,MgO,CO,61,13.8 ± 1.6,-176,20,Average of the $H_\textrm{ads$ re-analysis by ...
3,MgO,N2O,77,13.0 -> 14.0 ± 2.0,-239,31,$E_\textrm{d$ measured by Lian \etal{~\cite{li...
4,MgO,C6H6,162,15.1 ± 1.6,-481,69,Average taken of $H_\textrm{ads$ re-analyzed b...
5,MgO,H2O,203,-,-520,121,$H_\textrm{ads$ from Ferry \etal{~\cite{alessi...
6,MgO,NH3,160,13.0 -> 14.0 ± 2.0,-613,65,$E_\textrm{d$ measurement from Arthur \etal{~\...
7,MgO,Physisorbed CO2,120,13.0 -> 14.0 ± 2.0,-431,49,$E_\textrm{d$ measurement from Meixner \etal{~...
8,MgO,Chemisorbed CO2,230,13.0 -> 14.0 ± 2.0,-664,125,$E_\textrm{d$ measurement from Chakradhar and ...
9,MgO,Monolayer CH4,47,13.1 ± 2.0,-131,19,Monolayer $E_\textrm{d$ estimate by Tait \etal...


# Computing *E*<sub>rlx</sub>, *E*<sub>therm</sub> and *E*<sub>ZPV</sub> from DFT

### SI - DFT Parameters

In [5]:
dft_parameters_dict = {
    'MgO': {
        '(meta)GGA': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        },
        'hybrid': {
            'Supercell Size': '4x4' ,
            'Number of Layers': '4(2)' ,
            'K-point Mesh': '2x2x1(2x2x1)',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Mg_pv (4e core)',
        }
    },
    'r-TiO2': {
        '(meta)GGA': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(3) O-Ti-O' ,
            'K-point Mesh': '2x2x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': 'p(4x2)' ,
            'Number of Layers': '5(2) O-Ti-O' ,
            'K-point Mesh': '2x2x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    },
    'a-TiO2': {
        '(meta)GGA': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O layers',
            'K-point Mesh': '3x3x1',
            'Energy cutoff': '600',
            'Vacuum': '15',
            'Metal PAW potential': 'Ti_pv (12e core)',
        },
        'hybrid': {
            'Supercell Size': '3x1' ,
            'Number of Layers': '8(2) O-Ti-O' ,
            'K-point Mesh': '3x3x1(1x1x1)',
            'Energy cutoff': '520',
            'Vacuum': '13',
            'Metal PAW potential': 'Ti (18e core)',
        }
    }
}

dft_parameters_df = pd.DataFrame.from_dict(
    {(convert_to_nice_labels['surface'][surface], xc_func): parameters
     for surface, xc_func_dict in dft_parameters_dict.items()
     for xc_func, parameters in xc_func_dict.items()},
    orient='index'
).transpose()

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_parameters_df,
    start_input = r'\begin{table}',
    label = 'tab:dft_parameters',
    caption = 'DFT parameters used for the three different surfaces. The parameters for hybrids are also different from those for the metaGGA, GGA and vdW-inclusive functionals, grouped as (meta)GGA in the table. The number of layers in parentheses indicates the number of layers fixed at the bottom of the slab. The k-points in parenthesis indicates the k-point mesh used for the exact exchange potential',
    end_input = r'\end{table}',
    replace_input = {
        '_': r'\_',
        'x': r'${\times}$',
    },
    adjustbox = 1,
    df_latex_skip = 0,
    multiindex_sep = r'\cmidrule(lr){2-3}  \cmidrule(lr){4-5} \cmidrule(lr){6-7}',
    filename = 'Tables/SI_Table-DFT_Parameters.tex',
    column_format = 'l' + 'r'*len(dft_parameters_df.columns)
)

dft_parameters_df

MgO (001)                  TiO$_2$ rutile (110)  \
                           (meta)GGA           hybrid            (meta)GGA   
Supercell Size                   4x4              4x4               p(4x2)   
Number of Layers                4(2)             4(2)          5(3) O-Ti-O   
K-point Mesh                   2x2x1     2x2x1(2x2x1)                2x2x1   
Energy cutoff                    600              600                  600   
Vacuum                            15               15                   15   
Metal PAW potential  Mg_pv (4e core)  Mg_pv (4e core)     Ti_pv (12e core)   

                                   TiO$_2$ anatase (101)                 
                            hybrid             (meta)GGA         hybrid  
Supercell Size              p(4x2)                   3x1            3x1  
Number of Layers       5(2) O-Ti-O    8(2) O-Ti-O layers    8(2) O-Ti-O  
K-point Mesh          2x2x1(1x1x1)                 3x3x1   3x3x1(1x1x1)  
Energy cutoff                  520                   600            520  
Vacuum                          13                    15             13  
Metal PAW potential  Ti (18e core)      Ti_pv (12e core)  Ti (18e core)

### SI - *E*<sub>int</sub> for as function of (model ensemble) geometry and single-point (model ensemble)

In [6]:
dft_eint_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        for xc_func_sp in crystal_xc_func_ensemble[surface]: # the model used for the geometry optimization
            for xc_func_geom in crystal_xc_func_ensemble[surface]: # the model used for the single-point calculation
                dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] = calculate_eint(f'Data/05a-Eint_and_Erlx_DFT/{surface}/{molecule_label}/{xc_func_sp}/{xc_func_geom}', code='vasp')*1000/num_monomer

### SI - DFT Lattice Parameters

In [7]:

# Initialize an empty list to store lattice parameters for each DFT model
lattice_parameter_xc_dict = {surface: {convert_to_nice_labels['xc_functionals'][xc_func]: {} for xc_func in crystal_xc_func_ensemble[surface]} for surface in ['MgO', 'r-TiO2', 'a-TiO2']} 
# {xc_func: {'MgO a': 0, 'MgO Error': 0,  'r-TiO2 a': 0, 'r-TiO2 c': 0, 'r-TiO2 RMSE': 0, 'a-TiO2 a': 0, 'a-TiO2 c': 0, 'a-TiO2 RMSE': 0} for xc_func in ['Experiment'] + list(convert_to_nice_labels['xc_functionals'].values())}

experiment_lattice_parameter = {'MgO': {'Lattice Parameter a': '4.214'}, 'r-TiO2': {'Lattice Parameter a': '4.587', 'Lattice Parameter c': '2.954'}, 'a-TiO2': {'Lattice Parameter a': '3.782', 'Lattice Parameter c': '9.502'} }
experiment_lattice_parameter_reference = {
    'MgO': r'~\cite{singhSynthesisCharacterizationAlkalineEarthOxide2018}',
    'r-TiO2': r'~\cite{burdettStructuralelectronicRelationshipsInorganic1987}',
    'a-TiO2': r'~\cite{burdettStructuralelectronicRelationshipsInorganic1987}'
}
skzcam_eint = {'MgO': '-703 ± 4', 'r-TiO2': '-1310 ± 33', 'a-TiO2': '-1208 ± 16'}

# Loop through each DFT model
for surface in ['MgO', 'r-TiO2', 'a-TiO2']:
    for xc_func in crystal_xc_func_ensemble[surface]:
        # Read the CONTCAR file to obtain lattice parameter 'a'
        if surface == 'r-TiO2' and xc_func == '06_HSE06-D4':
            continue
        unit_cell = io.read(f'Data/01-Unit_Cell/{surface}/{xc_func}/OUTCAR')
        # Add the lattice parameter 'a' to the latpar_functional list
        lattice_parameter_xc_dict[surface][convert_to_nice_labels['xc_functionals'][xc_func]][f'Lattice Parameter a'] = f'{unit_cell.get_cell()[0][0]:.3f}'

        if 'TiO2' in surface:
            lattice_parameter_xc_dict[surface][convert_to_nice_labels['xc_functionals'][xc_func]][f'Lattice Parameter c'] = f'{round(unit_cell.get_cell()[2][2],3):.3f}'

        
        if surface == 'MgO':
            lattice_parameter_xc_dict[surface][convert_to_nice_labels['xc_functionals'][xc_func]][f'H2O Eint'] = int(round(dft_eint_dict[surface]['H2O Monomer'][xc_func][xc_func]))

        else:
            lattice_parameter_xc_dict[surface][convert_to_nice_labels['xc_functionals'][xc_func]][f'H2O Eint'] = int(round(dft_eint_dict[surface]['H2O'][xc_func][xc_func]))


    lattice_parameter_xc_dict[surface]['Reference'] = experiment_lattice_parameter[surface]
    lattice_parameter_xc_dict[surface]['Reference']['H2O Eint'] = skzcam_eint[surface]

    # Create a DataFrame from the lattice_parameter_xc_dict
    lattice_parameter_xc_df = pd.DataFrame(lattice_parameter_xc_dict[surface]).T

    # Write the DataFrame to a latex input
    convert_df_to_latex_input(
        lattice_parameter_xc_df,
        start_input = r'\begin{table}',
        label = f'tab:lattice_parameters{surface.lower()}',
        caption = r'Lattice parameter and \ce{H2O} $E_\textrm{int}$ for '  + convert_to_nice_labels['surface'][surface] + r' obtained from the DFT model ensemble. These are compared to experiment' + experiment_lattice_parameter_reference[surface] + r' for lattice parameter and the SKZCAM protocol $E_\textrm{int}$',
        end_input = r'\end{table}',
        replace_input = {
            '&  0.000': '&      -',
            '±': r'$\pm$',
            'Eint': r'$E_\textrm{int}$'
        },
        df_latex_skip = 0,
        filename = f'Tables/SI_Table-Lattice_Parameters_{surface}.tex',
        column_format = 'l' + 'r'*len(lattice_parameter_xc_df.columns)
    )
    lattice_parameter_xc_df = lattice_parameter_xc_df.style.set_caption(surface)

    display(lattice_parameter_xc_df)

,Lattice Parameter a,H2O Eint
PBE-D2[Ne],4.234,-693
revPBE-D4,4.220,-637
vdW-DF,4.273,-567
rev-vdW-DF2,4.220,-711
PBE0-D4,4.175,-695
B3LYP-D2[Ne],4.202,-618
Reference,4.214,-703 ± 4


,Lattice Parameter a,Lattice Parameter c,H2O Eint
PBE+U-TS/HI,4.647,3.026,-1601
revPBE-D4,4.598,2.958,-1212
vdW-DF,4.685,2.995,-1023
rev-vdW-DF2,4.618,2.961,-1352
r$^2$SCAN-rVV10,4.590,2.957,-1556
HSE06-D4,nan,nan,nan
Reference,4.587,2.954,-1310 ± 33


,Lattice Parameter a,Lattice Parameter c,H2O Eint
PBE+U-TS/HI,3.859,9.676,-1303
revPBE-D4,3.790,9.549,-1100
vdW-DF,3.839,9.767,-915
rev-vdW-DF2,3.798,9.593,-1177
r$^2$SCAN-rVV10,3.786,9.531,-1392
HSE06-D4,3.769,9.518,-1209
Reference,3.782,9.502,-1208 ± 16


### SI - *E*<sub>rlx</sub> for DFT model ensemble for monomers and monolayers

In [8]:
# Erlx is calculated with the model used to originally generate the geometry

dft_eads_true_dict = {surface : { molecule : {xc_func : 0 for xc_func in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}
dft_erlx_dict = {surface : { molecule : {xc_func_sp : 0 for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in dft_eads_true_dict[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
        for xc_func in crystal_xc_func_ensemble[surface]:
            molecule_surface_energy = get_energy(f'Data/04-Molecule_Surface/{surface}/{molecule_label}/{xc_func}/OUTCAR', code='vasp')
            if molecule in molecule_surface_systems['MgO_8x8']:
                molecule_surface_energy = get_energy(f'Data/05a-Eint_and_Erlx_DFT/{surface}/{molecule_label}/{xc_func}/{xc_func}/Molecule-Surface/OUTCAR', code='vasp')
                surface_energy = get_energy(f'Data/02-Surface/MgO_8x8/{xc_func}/OUTCAR', code='vasp')
                molecule_energy = get_energy(f'Data/03-Molecule/{surface}/{molecule_label}/{xc_func}/OUTCAR', code='vasp')
            else:
                surface_energy = get_energy(f'Data/02-Surface/{surface}/{xc_func}/OUTCAR', code='vasp')
                molecule_energy = get_energy(f'Data/03-Molecule/{surface}/{molecule.split()[0]}/{xc_func}/OUTCAR', code='vasp')

            dft_eads_true_dict[surface][molecule][xc_func] = (molecule_surface_energy - surface_energy - num_monomer*molecule_energy)*1000/num_monomer

            if molecule not in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer','NO Dimer','CO2 Chemisorbed']:
                dft_erlx_dict[surface][molecule][xc_func] = dft_eads_true_dict[surface][molecule][xc_func] - dft_eint_dict[surface][molecule][xc_func][xc_func]
            

### SI - *E*<sub>coh</sub>, *E*<sub>conf</sub> and *E*<sub>rlx</sub> for the monolayer, clusters and chemisorbed CO2 with the model ensemble

In [9]:
# Calculating the conformational energy for the CO2 geometry chemisorbed to the MgO surface back to its pristine geometry

dft_econf = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in crystal_xc_func_ensemble['MgO']} for molecule in ['CO2 Chemisorbed']}

for molecule in ['CO2 Chemisorbed']:
    molecule_label = molecule.replace(' ', '_')
    for xc_func_sp in crystal_xc_func_ensemble['MgO']:
        for xc_func_geom in crystal_xc_func_ensemble['MgO']:
            gas_phase_energy = get_energy(f'Data/07-Econf/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Molecule/OUTCAR', code='vasp')
            adsorbed_energy = get_energy(f'Data/07-Econf/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Molecule-Surface/OUTCAR', code='vasp')
            dft_econf[molecule][xc_func_sp][xc_func_geom] = (adsorbed_energy - gas_phase_energy)*1000

        if molecule == 'CO2 Chemisorbed':
            dft_erlx_dict['MgO']['CO2 Chemisorbed'][xc_func_sp] = dft_eads_true_dict['MgO']['CO2 Chemisorbed'][xc_func_sp] - dft_eint_dict['MgO']['CO2 Chemisorbed'][xc_func_sp][xc_func_sp] - dft_econf['CO2 Chemisorbed'][xc_func_sp][xc_func_sp]

In [10]:
# Calculating the cohesive energy that bind the molecules within the cluster or monolayer systems

dft_cluster_monolayer_ecoh = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in crystal_xc_func_ensemble['MgO']} for molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer','NO Dimer']}

dft_monolayer_ecoh_gas = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in crystal_xc_func_ensemble['MgO']} for molecule in ['CH4 Monolayer', 'C2H6 Monolayer']}

for molecule in dft_cluster_monolayer_ecoh:
    molecule_label = molecule.replace(' ', '_')
    num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
    for xc_func_sp in crystal_xc_func_ensemble['MgO']:
        for xc_func_geom in crystal_xc_func_ensemble['MgO']:
            if 'Monolayer' in molecule:
                if xc_func_sp in ['05_PBE0-D4','06_B3LYP-D2-Ne']:
                    continue
                monolayer_gas_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer/OUTCAR', code='vasp')
                monomer_energy = []
                for monomer_idx in range(1, num_monomer+1):
                    monomer_energy.append(get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer-Monomer-{monomer_idx}/OUTCAR', code='vasp'))
                dft_monolayer_ecoh_gas[molecule][xc_func_sp][xc_func_geom] = (monolayer_gas_energy - np.sum(monomer_energy))/num_monomer*1000

                monolayer_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer-Surface/OUTCAR', code='vasp')
                surface_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Surface/OUTCAR', code='vasp')
                monomer_energy = []
                for monomer_idx in range(1, num_monomer+1):
                    monomer_energy.append(get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monolayer-Monomer-{monomer_idx}/OUTCAR', code='vasp'))
                dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom] = (monolayer_energy - np.sum(monomer_energy) - surface_energy)/num_monomer*1000 - dft_eint_dict['MgO'][molecule][xc_func_sp][xc_func_geom] 

            elif 'Dimer' in molecule:
                dimer_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Dimer/OUTCAR', code='vasp')
                monomer_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Monomer/OUTCAR', code='vasp')
                dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom] = (dimer_energy/num_monomer - monomer_energy)*1000
            else:
                cluster_energy = get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Cluster/OUTCAR', code='vasp')
                monomer_energy = []
                for monomer_idx in range(1, num_monomer+1):
                    monomer_energy.append(get_energy(f'Data/08-Ecoh/MgO/{molecule_label}/{xc_func_sp}/{xc_func_geom}/Cluster-Monomer-{monomer_idx}/OUTCAR', code='vasp'))
                dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom] = (cluster_energy/num_monomer - np.mean(monomer_energy))*1000
                
            dft_erlx_dict['MgO'][molecule][xc_func_sp] = dft_eads_true_dict['MgO'][molecule][xc_func_sp] - dft_eint_dict['MgO'][molecule][xc_func_sp][xc_func_sp] - dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_sp]


#### Table of relaxation energy

In [11]:
dft_erlx_df = pd.DataFrame({convert_to_nice_labels['adsorbates'][surface][molecule] + ' on ' + convert_to_nice_labels['surface'][surface]: {convert_to_nice_labels['xc_functionals'][xc_func]: dft_erlx_dict[surface][molecule][xc_func] for xc_func in crystal_xc_func_ensemble[surface]} for surface in ['MgO'] for molecule in molecule_surface_systems[surface]}).T

dft_erlx_df = dft_erlx_df.round().astype(int)

display(dft_erlx_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_erlx_df,
    start_input = r'\begin{table}',
    label = f'tab:lattice_parameters_mgo',
    caption = r'Relaxation energy $E_\textrm{rlx}$ for the molecule-surface systems with the MgO surface obtained from the DFT model ensemble.',
    end_input = r'\end{table}',
    replace_input = {
        '&  0.000': '&      -',
        '±': r'$\pm$',
        'Eint': r'$E_\textrm{int}$'
    },
    df_latex_skip = 0,
    filename = f'Tables/SI_Table-Erlx_MgO.tex',
    column_format = 'l' + 'r'*len(dft_erlx_df.columns)
)

dft_erlx_df = pd.DataFrame({convert_to_nice_labels['adsorbates'][surface][molecule] + ' on ' + convert_to_nice_labels['surface'][surface]: {convert_to_nice_labels['xc_functionals'][xc_func]: dft_erlx_dict[surface][molecule][xc_func] for xc_func in crystal_xc_func_ensemble[surface]} for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}).T

dft_erlx_df = dft_erlx_df.round().astype(int)

display(dft_erlx_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_erlx_df,
    start_input = r'\begin{table}',
    label = f'tab:lattice_parameters_tio2',
    caption = r'Relaxation energy $E_\textrm{rlx}$ for the molecule-surface systems with the \ce{TiO2} surfaces obtained from the DFT model ensemble.',
    end_input = r'\end{table}',
    replace_input = {
        '&  0.000': '&      -',
        '±': r'$\pm$',
        'Eint': r'$E_\textrm{int}$'
    },
    df_latex_skip = 0,
    filename = f'Tables/SI_Table-Erlx_TiO2.tex',
    column_format = 'l' + 'r'*len(dft_erlx_df.columns)
)


,PBE-D2[Ne],revPBE-D4,vdW-DF,rev-vdW-DF2,PBE0-D4,B3LYP-D2[Ne]
CH$_4$ on MgO (001),4,2,0,2,0,0
Monolayer CH$_4$ on MgO (001),3,1,0,2,0,0
C$_2$H$_6$ on MgO (001),3,1,-2,2,-4,-3
Monolayer C$_2$H$_6$ on MgO (001),4,4,1,2,0,0
CO on MgO (001),13,8,13,13,9,11
C$_6$H$_6$ on MgO (001),24,26,11,18,6,7
Parallel N$_2$O on MgO (001),5,3,4,6,2,4
Tilted N$_2$O on MgO (001),9,10,3,11,7,6
Vertical-Hollow NO on MgO (001),75,29,13,62,2,3
Vertical-Mg NO on MgO (001),10,9,9,48,5,5


,PBE+U-TS/HI,revPBE-D4,vdW-DF,rev-vdW-DF2,r$^2$SCAN-rVV10,HSE06-D4
CH$_4$ on TiO$_2$ rutile (110),33,20,5,27,44,27
Parallel CO$_2$ on TiO$_2$ rutile (110),49,14,15,28,32,30
Tilted CO$_2$ on TiO$_2$ rutile (110),80,50,37,71,92,78
H$_2$O on TiO$_2$ rutile (110),397,237,174,295,324,312
CH$_3$OH on TiO$_2$ rutile (110),417,304,267,370,377,390
H$_2$O on TiO$_2$ anatase (101),260,230,155,244,267,240
NH$_3$ on TiO$_2$ anatase (101),214,213,193,222,237,247


 ### SI - Comparing the true and approximate *E*<sub>ads</sub> value to estimate errors in the final *E*<sub>ads</sub>

In [12]:
dft_eads_approx_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

dft_geom_error_dict = {surface : { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for xc_func_sp in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

dft_geom_rmse_dict = {surface : { molecule : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in dft_geom_error_dict[surface]:
        for xc_func_idx, xc_func_sp in enumerate(crystal_xc_func_ensemble[surface]):

            for xc_func_geom in crystal_xc_func_ensemble[surface]:
                if molecule in ['NO Dimer','CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer']:
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom] + dft_cluster_monolayer_ecoh[molecule][xc_func_sp][xc_func_geom]                    
                elif molecule == 'CO2 Chemisorbed':
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom] + dft_econf[molecule][xc_func_sp][xc_func_geom]
                else:
                    dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eint_dict[surface][molecule][xc_func_sp][xc_func_geom] + dft_erlx_dict[surface][molecule][xc_func_geom]


                dft_geom_error_dict[surface][molecule][xc_func_sp][xc_func_geom] = dft_eads_true_dict[surface][molecule][xc_func_sp] - dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom]
        for xc_func_geom in crystal_xc_func_ensemble[surface]:
            cumulative_squared_error = 0
            for xc_func_sp in crystal_xc_func_ensemble[surface]:
                if xc_func_geom != xc_func_sp:
                    cumulative_squared_error += dft_geom_error_dict[surface][molecule][xc_func_sp][xc_func_geom]**2
            
            dft_geom_rmse_dict[surface][molecule][xc_func_geom] = np.sqrt(cumulative_squared_error/5)

#### Heatmap showing model geometry errors in select systems

In [13]:
fig, axs = plt.subplots(2,2,figsize=(8,9.5),dpi=450,constrained_layout=True)
plt.set_cmap('bwr')

system_list = []
system_idx = 0

system_title_list = ['CO on MgO (001)', r'H$_2$O on MgO (001)', r'CH$_4$ on TiO$_2$ rutile (110)', r'NH$_3$ on TiO$_2$ anatase (101)']

for molecule, surface in [('CO','MgO'), ('H2O Monomer', 'MgO'), ('CH4', 'r-TiO2'), ('NH3', 'a-TiO2')]:
    system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'
    system_list += [system_label]
    system_geom_error = np.zeros(np.array([6,6])+1)
    system_geom_mad = np.zeros(np.array([6,6])+1)

    for sp_idx, xc_func_sp in enumerate(crystal_xc_func_ensemble[surface]):
        for geom_idx, xc_func_geom in enumerate(crystal_xc_func_ensemble[surface]):
            system_geom_error[sp_idx,geom_idx] = dft_eads_approx_dict[surface][molecule][xc_func_sp][xc_func_geom] - dft_eads_true_dict[surface][molecule][xc_func_sp]
                    
    for xc_idx, xc_func in enumerate(crystal_xc_func_ensemble[surface]):
        system_geom_mad[xc_idx,-1] = np.mean(np.abs(system_geom_error[xc_idx,:-1]))
        system_geom_mad[-1,xc_idx] = np.mean(np.abs(system_geom_error[:-1,xc_idx]))
    
    # Round the values to nearest integer
    system_geom_error = np.round(system_geom_error,0).astype(int)
    system_geom_mad = np.round(system_geom_mad,0).astype(int)

    divmodi = divmod(system_idx,2)

    fig_ylabels = []
    xc_nice_list_vals = []
    counter = 0
    for xc_idx, xc_func in  enumerate(crystal_xc_func_ensemble[surface]):
        if divmodi[1] == 0:
            fig_ylabels += ['{0} ({1:.0f} meV)'.format(convert_to_nice_labels['xc_functionals'][xc_func],dft_eads_true_dict[surface][molecule][xc_func])]
            counter += 1 
        else:
            fig_ylabels += ['({0:.0f} meV)'.format(dft_eads_true_dict[surface][molecule][xc_func])]
            counter += 1 

    fig_ylabels += ['MAD']
    fig_xlabels = [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble[surface]] + ['MAD']

    im = axs[divmodi[0],divmodi[1]].imshow(system_geom_error, vmin=-200,vmax=200)

    # Define custom colormap that transitions from invisible to black
    colors = [(1, 1, 1, 0), (0, 0, 0, 0.7)]  # RGBA tuples: transparent to black
    n_bins = 100  # Discretize into 100 steps
    cmap_custom = LinearSegmentedColormap.from_list('invisible_to_black', colors, N=n_bins)
    im_custom = axs[divmodi[0],divmodi[1]].imshow(system_geom_mad, cmap=cmap_custom, alpha=1,vmin=0,vmax=150)  # Use 

    # Show all ticks and label them with the respective list entries
    axs[divmodi[0],divmodi[1]].set_xticks(np.arange(7), labels=fig_xlabels)
    axs[divmodi[0],divmodi[1]].set_yticks(np.arange(7), labels=fig_ylabels)

    for i12 in range(7):
        for j in range(7):
            if j == 7-1 and i12 == 7-1:
                continue
            elif j == 7-1 or i12 == 7-1:
                text = axs[divmodi[0],divmodi[1]].text(j, i12, system_geom_mad[i12,j],
                        ha="center", va="center", color="k")
            else:
                text = axs[divmodi[0],divmodi[1]].text(j, i12, system_geom_error[i12, j],
                        ha="center", va="center", color="k")
    axs[divmodi[0],divmodi[1]].set_title(system_title_list[system_idx])

    axs[divmodi[0],divmodi[1]].set_xticklabels(fig_xlabels, rotation=45, ha="right",rotation_mode="anchor")
    if divmodi[0] == 1:
        axs[divmodi[0],divmodi[1]].set_xlabel(r'XC Functional (XC2) for generating \textbf{geometries}')
    system_idx += 1


axs[0,0].set_ylabel(r'XC Functional (XC1) for \textbf{calculating} $E_\textrm{ads}^\textrm{approx}$')
axs[1,0].set_ylabel(r'XC Functional (XC1) for \textbf{calculating} $E_\textrm{ads}^\textrm{approx}$')

cbar = fig.colorbar(im, ax=axs, orientation='horizontal', fraction=0.02)
cbar.ax.set_xlabel(r'$E_\textrm{ads}^\textrm{approx}[\textrm{XC1//XC2}]$ - $E_\textrm{ads}^\textrm{True}[\textrm{XC1//XC1}]$ [meV]')
cbar_custom = fig.colorbar(im_custom, ax=axs, orientation='horizontal', fraction=0.02)
cbar_custom.ax.set_xlabel('Mean Absolute Deviation [meV]')

plt.savefig('Figures/SI_Figure-Geometry_Error.png')

### SI - Estimating Errors in *E*<sub>ads</sub> from the DFT XC ensemble

#### Monomers on MgO(001)

In [14]:
# Starting with the single monomers on MgO surface

dft_eads_geom_error_dict = {surface : { molecule : {xc_func: {'Eint': 0, 'Erlx': 0, 'Ecoh': 0, 'Econf': 0, 'Approx': 0, 'True': 0, 'Error': 0} for xc_func in crystal_xc_func_ensemble[surface]} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

mgo_monomer_molecules = [molecule for molecule in molecule_surface_systems['MgO'] if ('Tetramer' not in molecule) and ('Dimer' not in molecule) and ('Monolayer' not in molecule) and ('Chemisorbed' not in molecule)]

for surface in ['MgO']:
    for molecule in mgo_monomer_molecules:
        cumulative_squared_error = 0
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['True']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        


# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {( f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict['MgO'][molecule][xc_func][quantity] for molecule in mgo_monomer_molecules] for xc_func in crystal_xc_func_ensemble['MgO'] for quantity in ['Eint', 'Erlx', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict['MgO'][molecule]['RMSE'] for molecule in mgo_monomer_molecules]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int)

dft_eads_geom_error_df.index = mgo_monomer_molecules

display(dft_eads_geom_error_df.T)


dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'}' if len(molecule.split()) == 2 else r'\ce{' + molecule + r'}'   for molecule in mgo_monomer_molecules]

dft_eads_geom_error_df = dft_eads_geom_error_df.T

dft_eads_geom_error_df.index = pd.MultiIndex.from_tuples([(r'\rotatebox{90}{' + a[:-11] + r'}' ,b) for a,b in dft_eads_geom_error_df.index.tolist()])

# Write the DataFrame to a latex input
latex_input_str = '\n'.join(convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors_mgo',
    caption = r'For the monomers on the MgO(001) surface, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.',
    end_input = '\\end{table}',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$',
        r'\multirow[t]' : r'\multirow[]',
        r'\rotatebox{90}{} & RMSE': r'$\epsilon_\textrm{geom}$ & RMSE'
    },
    adjustbox = 1,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True,
    output_str = True,
    column_format = 'll' + 'r'*len(dft_eads_geom_error_df.columns)
).splitlines()[7:-4]) + '\n'

with open('Tables/SI_Table-Eads_Errors_Monomers_on_MgO.tex', 'w') as f:
    f.write(r"""\LTcapwidth=\textwidth
    
\begin{longtable}{llrrrrrrrrrrrrrrrr}
\caption{\label{tab:eads_dft_ensemble_errors_mgo}For the monomers on the MgO(001) surface, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.} \\

\toprule
 &  & \rotatebox{90}{\ce{CH4}} & \rotatebox{90}{\ce{C2H6}} & \rotatebox{90}{\ce{CO}} & \rotatebox{90}{\ce{C6H6}} & \rotatebox{90}{Parallel \ce{N2O}} & \rotatebox{90}{Tilted \ce{N2O}} & \rotatebox{90}{Vertical-Hollow \ce{NO}} & \rotatebox{90}{Vertical-Mg \ce{NO}} & \rotatebox{90}{Bent-Bridge \ce{NO}} & \rotatebox{90}{Bent-Mg \ce{NO}} & \rotatebox{90}{Bent-O \ce{NO}} & \rotatebox{90}{Monomer \ce{H2O}} & \rotatebox{90}{Tilted \ce{CH3OH}} & \rotatebox{90}{Parallel \ce{CH3OH}} & \rotatebox{90}{\ce{NH3}} & \rotatebox{90}{Physisorbed \ce{CO2}} \\ 
\midrule
\endfirsthead



\caption[]{(continued)} \\
\endhead

\multicolumn{18}{r}{{Continued on next page}} \\
\endfoot

\bottomrule
\endlastfoot

""")
    f.write(latex_input_str)
    f.write(r"\end{longtable}")



CH4  C2H6   CO  C6H6  N2O Parallel  \
PBE-D2[Ne]//revPBE-D4   Eint   -115  -157 -233  -278          -180   
                        Erlx      2     1    8    26             3   
                        Approx -114  -156 -224  -252          -177   
                        True   -115  -158 -228  -261          -179   
                        Error     2     2    3     9             2   
revPBE-D4//revPBE-D4    Eint   -143  -219 -215  -565          -192   
                        Erlx      2     1    8    26             3   
                        Approx -142  -218 -206  -538          -189   
                        True   -142  -218 -206  -538          -189   
                        Error     0     0    0     0             0   
vdW-DF//revPBE-D4       Eint   -137  -197 -221  -391          -246   
                        Erlx      2     1    8    26             3   
                        Approx -135  -196 -212  -365          -242   
                        True   -156  -233 -233  -466          -261   
                        Error    21    37   21   101            19   
rev-vdW-DF2//revPBE-D4  Eint   -141  -207 -273  -460          -247   
                        Erlx      2     1    8    26             3   
                        Approx -140  -206 -265  -433          -244   
                        True   -140  -207 -266  -449          -246   
                        Error     0     1    1    16             2   
PBE0-D4//revPBE-D4      Eint   -159  -227 -250  -523          -246   
                        Erlx      2     1    8    26             3   
                        Approx -158  -226 -241  -497          -243   
                        True   -158  -229 -234  -521          -251   
                        Error     0     4   -7    24             8   
B3LYP-D2[Ne]//revPBE-D4 Eint    -89  -127 -156  -210          -168   
                        Erlx      2     1    8    26             3   
                        Approx  -87  -126 -148  -184          -164   
                        True    -88  -131 -149  -219          -173   
                        Error     1     5    1    35             8   
DeltaGeom               RMSE     10    17   10    50            10   

                                N2O Tilted  NO Vertical-Hollow  \
PBE-D2[Ne]//revPBE-D4   Eint          -137                -263   
                        Erlx            10                  29   
                        Approx        -127                -234   
                        True          -132                -184   
                        Error            5                 -50   
revPBE-D4//revPBE-D4    Eint          -133                -184   
                        Erlx            10                  29   
                        Approx        -123                -154   
                        True          -123                -154   
                        Error            0                   0   
vdW-DF//revPBE-D4       Eint          -181                -174   
                        Erlx            10                  29   
                        Approx        -171                -145   
                        True          -220                -187   
                        Error           50                  42   
rev-vdW-DF2//revPBE-D4  Eint          -192                -293   
                        Erlx            10                  29   
                        Approx        -182                -264   
                        True          -183                -227   
                        Error            1                 -37   
PBE0-D4//revPBE-D4      Eint          -166                -103   
                        Erlx            10                  29   
                        Approx        -156                 -74   
                        True          -160                -102   
                        Error            4                  28   
B3LYP-D2[Ne]//revPBE-D4 Eint           -98                 -37   
         

#### Clusters and Monolayer on MgO(001)

In [15]:
mgo_monolayer_cluster_molecules = [molecule for molecule in molecule_surface_systems['MgO'] if ('Tetramer' in molecule) or ('Monolayer' in molecule) or ('Dimer' in molecule) or ('Chemisorbed' in molecule)]

for surface in ['MgO']:
    for molecule in mgo_monolayer_cluster_molecules:
        cumulative_squared_error = 0
        if molecule == 'NO Dimer':
            xc_func_geom = '06_B3LYP-D2-Ne'
        else:
            xc_func_geom = '02_revPBE-D4'
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func][xc_func_geom]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule][xc_func_geom]

            if molecule == 'CO2 Chemisorbed':
                dft_eads_geom_error_dict[surface][molecule][xc_func]['Ecoh/Econf'] = dft_econf[molecule][xc_func][xc_func_geom]
            else:
                dft_eads_geom_error_dict[surface][molecule][xc_func]['Ecoh/Econf'] = dft_cluster_monolayer_ecoh[molecule][xc_func][xc_func_geom]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func][xc_func_geom]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        


# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {(f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict['MgO'][molecule][xc_func][quantity] for molecule in mgo_monolayer_cluster_molecules] for xc_func in crystal_xc_func_ensemble['MgO'] for quantity in ['Eint', 'Erlx', 'Ecoh/Econf', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict['MgO'][molecule]['RMSE'] for molecule in mgo_monolayer_cluster_molecules]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int)

dft_eads_geom_error_df.index = mgo_monolayer_cluster_molecules

display(dft_eads_geom_error_df.round().astype(int).T)


dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'}' if len(molecule.split()) == 2 else r'\ce{' + molecule + r'}'   for molecule in mgo_monolayer_cluster_molecules]

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int).T

dft_eads_geom_error_df.index = pd.MultiIndex.from_tuples([(r'\rotatebox{90}{' + a[:-11] + r'}' ,b) for a,b in dft_eads_geom_error_df.index.tolist()])

# Write the DataFrame to a latex input
latex_input_str = '\n'.join(convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors_mgo',
    caption = r'For the clusters and monolayer systems as well as chemisorbed \ce{CO2} on MgO(001) surface, we estimate the errors for using the DFT geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$. For the NO dimer, this is done with respect to the B3LYP-D2[N2] geometry and $E_\textrm{rlx}$. There is an additional cohesive energy $E_\textrm{coh}$ term for the monolayer and cluster systems. For the chemisorbed \ce{CO2}, there is an additional conformational energy $E_\textrm{conf}$ term.',
    end_input = '\\end{table}',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$',
        'Ecoh/Econf': r'$E_\textrm{coh}$ or $E_\textrm{conf}$',
        r'\multirow[t]' : r'\multirow[]',
        r'\rotatebox{90}{} & RMSE': r'$\epsilon_\textrm{geom}$ & RMSE'
    },
    adjustbox = 1,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True,
    output_str = True,
    column_format = 'll' + 'r'*len(dft_eads_geom_error_df.columns)
).splitlines()[7:-4]) + '\n'

with open('Tables/SI_Table-Eads_Errors_Clusters_Monolayers_on_MgO.tex', 'w') as f:
    f.write(r"""\LTcapwidth=\textwidth
    
\begin{longtable}{llrrrrrr}
\caption{\label{tab:eads_dft_ensemble_errors_ml_clus_mgo} For the clusters and monolayer systems as well as chemisorbed \ce{CO2} on MgO(001) surface, we estimate the errors for using the DFT geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$. For the NO dimer, this is done with respect to the B3LYP-D2[N2] geometry and $E_\textrm{rlx}$. There is an additional cohesive energy $E_\textrm{coh}$ term for the monolayer and cluster systems. For the chemisorbed \ce{CO2}, there is an additional conformational energy $E_\textrm{conf}$ term.} \\

\toprule
 &  & \rotatebox{90}{Monolayer \ce{CH4}} & \rotatebox{90}{Monolayer \ce{C2H6}} & \rotatebox{90}{Dimer \ce{NO}} & \rotatebox{90}{Tetramer \ce{H2O}} & \rotatebox{90}{Tetramer \ce{CH3OH}} & \rotatebox{90}{Chemisorbed \ce{CO2}} \\ 
\midrule
\endfirsthead



\caption[]{(continued)} \\
\endhead

\multicolumn{8}{r}{{Continued on next page}} \\
\endfoot

\bottomrule
\endlastfoot

""")
    f.write(latex_input_str)
    f.write(r"\end{longtable}")

CH4 Monolayer  C2H6 Monolayer  NO Dimer  \
PBE-D2[Ne]//revPBE-D4   Eint                 -114            -142      -340   
                        Erlx                    1               4        26   
                        Ecoh/Econf            -40            -106      -234   
                        Approx               -153            -245      -548   
                        True                 -153            -248      -551   
                        Error                   0              -3        -3   
revPBE-D4//revPBE-D4    Eint                 -142            -207      -351   
                        Erlx                    1               4        26   
                        Ecoh/Econf            -31             -48      -186   
                        Approx               -171            -251      -511   
                        True                 -171            -251      -514   
                        Error                   0               0        -3   
vdW-DF//revPBE-D4       Eint                 -136            -200      -337   
                        Erlx                    1               4        26   
                        Ecoh/Econf            -58             -67      -145   
                        Approx               -192            -263      -457   
                        True                 -219            -319      -478   
                        Error                 -27             -55       -21   
rev-vdW-DF2//revPBE-D4  Eint                 -140            -197      -398   
                        Erlx                    1               4        26   
                        Ecoh/Econf            -43             -83      -276   
                        Approx               -181            -277      -648   
                        True                 -182            -282      -654   
                        Error                  -1              -5        -5   
PBE0-D4//revPBE-D4      Eint                 -158            -211      -290   
                        Erlx                    1               4        26   
                        Ecoh/Econf              0               0        62   
                        Approx               -157            -207      -202   
                        True                 -193            -274      -219   
                        Error                 -37             -66       -17   
B3LYP-D2[Ne]//revPBE-D4 Eint                  -87            -111      -231   
                        Erlx                    1               4        26   
                        Ecoh/Econf              0               0        63   
                        Approx                -86            -108      -143   
                        True                 -127            -238      -143   
                        Error                 -41            -130         0   
DeltaGeom               RMSE                   27              70        13   

                                    H2O Tetramer  CH3OH Tetramer  \
PBE-D2[Ne]//revPBE-D4   Eint                -397            -416   
                        Erlx                  52              68   
                        Ecoh/Econf          -309            -380   
                        Approx              -654            -729   
                        True                -660            -733   
                        Error                 -6              -5   
revPBE-D4//revPBE-D4    Eint                -404            -483   
                        Erlx                  52              68   
                        Ecoh/Econf          -261            -328   
                        Approx              -613            -743   
                        True                -613            -743   
                        Error                  0               0   
vdW-DF//revPBE-D4       Eint                -376            -447   
                        Erlx                  52              68   
             

#### Monomers on TiO<sub>2</sub>

In [16]:
for surface in ['r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        cumulative_squared_error = 0
        for xc_func in crystal_xc_func_ensemble[surface]:
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Eint'] = dft_eint_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Erlx'] = dft_erlx_dict[surface][molecule]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] = dft_eads_true_dict[surface][molecule][xc_func]
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx'] = dft_eads_approx_dict[surface][molecule][xc_func]['02_revPBE-D4']
            dft_eads_geom_error_dict[surface][molecule][xc_func]['Error'] = dft_eads_geom_error_dict[surface][molecule][xc_func]['True'] - dft_eads_geom_error_dict[surface][molecule][xc_func]['Approx']
            cumulative_squared_error += dft_eads_geom_error_dict[surface][molecule][xc_func]['Error']**2
        dft_eads_geom_error_dict[surface][molecule]['RMSE'] = np.sqrt(cumulative_squared_error/5)
        
# Create a DataFrame from the dft_eads_geom_error_dict

dft_eads_geom_error_df_dict = {(f"{convert_to_nice_labels['xc_functionals'][xc_func]}//revPBE-D4" , quantity): [dft_eads_geom_error_dict[surface][molecule][xc_func][quantity] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]] for xc_func in crystal_xc_func_ensemble['r-TiO2'] for quantity in ['Eint', 'Erlx', 'Approx', 'True', 'Error']}

dft_eads_geom_error_df_dict[(r'DeltaGeom', 'RMSE')] = [dft_eads_geom_error_dict[surface][molecule]['RMSE'] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]


dft_eads_geom_error_df = pd.DataFrame.from_dict(dft_eads_geom_error_df_dict)
dft_eads_geom_error_df.index = [ molecule + ' on ' + convert_to_nice_labels['surface'][surface] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]
display(dft_eads_geom_error_df.round().astype(int).T)

dft_eads_geom_error_df.index = [ molecule.split()[1] + r' \ce{'  + molecule.split()[0] + r'} on ' + convert_to_nice_labels['surface'][surface] if len(molecule.split()) == 2 else r'\ce{' + molecule + r'} on ' + convert_to_nice_labels['surface'][surface] for surface in ['r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]

dft_eads_geom_error_df = dft_eads_geom_error_df.round().astype(int).T

dft_eads_geom_error_df.index = pd.MultiIndex.from_tuples([(r'\rotatebox{90}{' + a[:-11] + r'}' ,b) for a,b in dft_eads_geom_error_df.index.tolist()])

# Write the DataFrame to a latex input
latex_input_str = '\n'.join(convert_df_to_latex_input(
    dft_eads_geom_error_df,
    start_input = '\\begin{table}',
    label = 'tab:eads_dft_ensemble_errors_tio2',
    caption = r'For the monomers on the \ce{TiO2} rutile (110) and anatase (101) surfaces, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.',
    end_input = '\\end{table}',
    replace_input = {
        'Erlx': r'$E_\textrm{rlx}$',
        'Eint': r'$E_\textrm{int}$',
        'True': r'$E_\textrm{ads}^\textrm{true}$',
        'Approx': r'$E_\textrm{ads}^\textrm{approx}$',
        'DeltaGeom': r'$\epsilon_\textrm{geom}$',
        r'\multirow[t]' : r'\multirow[]',
        r'\rotatebox{90}{} & RMSE': r'$\epsilon_\textrm{geom}$ & RMSE'
    },
    adjustbox = 1,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True,
    output_str = True,
    column_format = 'll' + 'r'*len(dft_eads_geom_error_df.columns)
).splitlines()[7:-4]) + '\n'

with open('Tables/SI_Table-Eads_Errors_Monomers_on_TiO2.tex', 'w') as f:
    f.write(r"""\LTcapwidth=\textwidth
    
\begin{longtable}{llrrrrrrr}
\caption{\label{tab:eads_dft_ensemble_errors_tio2} For the monomers on the \ce{TiO2} rutile (110) and anatase (101) surfaces, we estimate the errors for using the revPBE-D4 geometry and $E_\textrm{rlx}$ in the final $E_\textrm{ads}$ of the autoSKZCAM protocol using an ensemble of 6 different DFT models. The errors are calculated as the difference between the true $E_\textrm{ads}^\textrm{true}$ (using the appropriate DFT model) and the approximated $E_\textrm{ads}^\textrm{approx}$ using the revPBE-D4 geometry and $E_\textrm{rlx}$.} \\

\toprule
 &  & \rotatebox{90}{\ce{CH4} on TiO$_2$ rutile (110)} & \rotatebox{90}{Parallel \ce{CO2} on TiO$_2$ rutile (110)} & \rotatebox{90}{Tilted \ce{CO2} on TiO$_2$ rutile (110)} & \rotatebox{90}{\ce{H2O} on TiO$_2$ rutile (110)} & \rotatebox{90}{\ce{CH3OH} on TiO$_2$ rutile (110)} & \rotatebox{90}{\ce{H2O} on TiO$_2$ anatase (101)} & \rotatebox{90}{\ce{NH3} on TiO$_2$ anatase (101)} \\ 
\midrule
\endfirsthead



\caption[]{(continued)} \\
\endhead

\multicolumn{9}{r}{{Continued on next page}} \\
\endfoot

\bottomrule
\endlastfoot

""")
    f.write(latex_input_str)
    f.write(r"\end{longtable}")



CH4 on TiO$_2$ rutile (110)  \
PBE+U-TS/HI//revPBE-D4     Eint                           -294   
                           Erlx                             20   
                           Approx                         -274   
                           True                           -302   
                           Error                           -28   
revPBE-D4//revPBE-D4       Eint                           -288   
                           Erlx                             20   
                           Approx                         -268   
                           True                           -268   
                           Error                             0   
vdW-DF//revPBE-D4          Eint                           -238   
                           Erlx                             20   
                           Approx                         -218   
                           True                           -244   
                           Error                           -26   
rev-vdW-DF2//revPBE-D4     Eint                           -274   
                           Erlx                             20   
                           Approx                         -254   
                           True                           -257   
                           Error                            -3   
r$^2$SCAN-rVV10//revPBE-D4 Eint                           -334   
                           Erlx                             20   
                           Approx                         -314   
                           True                           -325   
                           Error                           -11   
HSE06-D4//revPBE-D4        Eint                           -285   
                           Erlx                             20   
                           Approx                         -265   
                           True                           -271   
                           Error                            -6   
DeltaGeom                  RMSE                             18   

                                   CO2 Parallel on TiO$_2$ rutile (110)  \
PBE+U-TS/HI//revPBE-D4     Eint                                    -345   
                           Erlx                                      14   
                           Approx                                  -331   
                           True                                    -396   
                           Error                                    -65   
revPBE-D4//revPBE-D4       Eint                                    -402   
                           Erlx                                      14   
                           Approx                                  -388   
                           True                                    -388   
                           Error                                      0   
vdW-DF//revPBE-D4          Eint                                    -368   
                           Erlx                                      14   
                           Approx                                  -354   
                           True                                    -380   
                           Error                                    -26   
rev-vdW-DF2//revPBE-D4     Eint                                    -409   
                           Erlx                                      14   
                           Approx                                  -395   
                           True                                    -401   
                           Error                                     -7   
r$^2$SCAN-rVV10//revPBE-D4 Eint                                    -532   
                           Erlx                                      14   
                           Approx                                  -518   
                           True                                    -537   
                           Error               

### SI - Getting *E*<sub>ZPV</sub> and *E*<sub>therm</sub> with the DFT XC ensemble

In [17]:
dft_ezpv_etherm_dict = {surface : { molecule : {xc_func: {'E_ZPV': 0, 'E_T': 0, 'kT': 0, 'DeltaH': 0} for xc_func in crystal_xc_func_ensemble[surface][:4] + ['Mean']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

dft_ezpv_etherm_df_dict = {} # DataFrame dictionary

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        if len(molecule.split()) == 2:
            system_label = convert_to_nice_labels['adsorbates'][surface][molecule] + f" on {convert_to_nice_labels['surface'][surface]}"
        else:
            system_label = convert_to_nice_labels['adsorbates'][surface][molecule] + f" on {convert_to_nice_labels['surface'][surface]}"

        if ('Tetramer' in molecule or 'NO' in molecule or 'CH3OH' in molecule) and surface == 'MgO':
            expt_molecule_label = 'Cluster ' + molecule.split()[0]
        elif 'Monolayer' in molecule:
            expt_molecule_label =  'Monolayer ' + molecule.split()[0]
        elif 'CO2' in molecule and surface == 'MgO':
            expt_molecule_label =  molecule.split()[1] + ' ' + molecule.split()[0]
        else:
            expt_molecule_label = molecule.split()[0]

        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 4 if 'Monolayer' in molecule else 2 if 'Dimer' in molecule else 1
        temperature = float(expt_hads_df.loc[(expt_hads_df['Surface'] == surface) & (expt_hads_df['Molecule'] == expt_molecule_label), 'Temperature'].values[0])

        for xc_func in crystal_xc_func_ensemble[surface][:4]:
            dft_ezpv_etherm_dict[surface][molecule][xc_func]['E_ZPV'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['E_T'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['kT'], dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] = calculate_ezpv_etherm(f'Data/06-Etherm_and_EZPV_DFT/{surface}/{molecule_label}/{xc_func}', ['Molecule-Surface','Molecule'], temperature, num_monomers=num_monomer)

        dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'] = np.mean([dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] for xc_func in crystal_xc_func_ensemble[surface][:4]])
        dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error'] = 2*np.std([dft_ezpv_etherm_dict[surface][molecule][xc_func]['DeltaH'] for xc_func in crystal_xc_func_ensemble[surface][:4]])

        dft_ezpv_etherm_df_dict[system_label] = [temperature, dft_ezpv_etherm_dict[surface][molecule][xc_func]['kT']] + [dft_ezpv_etherm_dict[surface][molecule][xc_func][quantity] for xc_func_idx, xc_func in enumerate(crystal_xc_func_ensemble[surface][:4]) for quantity in ['E_ZPV','E_T','DeltaH']] +  [ dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'], dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error']]

# Create a DataFrame from the dft_ezpv_etherm_df_dict
dft_ezpv_etherm_df = pd.DataFrame.from_dict(dft_ezpv_etherm_df_dict)
dft_ezpv_etherm_df.index = ['Temperature [K]', 'kT [meV]'] + [f"{quantity} [XC {xc_func_idx+1}]" for xc_func_idx, xc_func in enumerate(crystal_xc_func_ensemble[surface][:4]) for quantity in ['E_ZPV','E_T','DeltaH']] + ['Final DeltaH','Error']

# Convert to int
dft_ezpv_etherm_df = dft_ezpv_etherm_df.round().astype(int)

display(dft_ezpv_etherm_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_ezpv_etherm_df,
    start_input = '\\begin{turnpage}\n\\begin{table}',
    label = 'tab:ethermal',
    caption = r'The zero-point vibrational energy ($E_\textrm{ZPV}$), thermal energy ($E_\textrm{therm}$), and overall enthalpy ($\Delta H$) contributions to the adsorption enthalpy for all studied systems using an ensemble of 4 different DFT models. The errors are calculated as the 2$\sigma$ standard deviation of the $\Delta H$ values for the 4 DFT models.',
    end_input = '\\end{table}\n\\end{turnpage}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Ethermal.tex',
    replace_input = {
        'E_ZPV': r'$E_\textrm{ZPV}$',
        'E_T': r'$E_\textrm{T}$',
        'DeltaH': r'$\Delta H$'
    },
    rotate_column_header = True
)

,CH$_4$ on MgO (001),Monolayer CH$_4$ on MgO (001),C$_2$H$_6$ on MgO (001),Monolayer C$_2$H$_6$ on MgO (001),CO on MgO (001),C$_6$H$_6$ on MgO (001),Parallel N$_2$O on MgO (001),Tilted N$_2$O on MgO (001),Vertical-Hollow NO on MgO (001),Vertical-Mg NO on MgO (001),...,NH$_3$ on MgO (001),Physisorbed CO$_2$ on MgO (001),Chemisorbed CO$_2$ on MgO (001),CH$_4$ on TiO$_2$ rutile (110),Parallel CO$_2$ on TiO$_2$ rutile (110),Tilted CO$_2$ on TiO$_2$ rutile (110),H$_2$O on TiO$_2$ rutile (110),CH$_3$OH on TiO$_2$ rutile (110),H$_2$O on TiO$_2$ anatase (101),NH$_3$ on TiO$_2$ anatase (101)
Temperature [K],47,47,75,75,61,162,77,77,80,80,...,160,120,230,85,177,177,303,370,257,410
kT [meV],4,4,6,6,5,14,7,7,7,7,...,14,10,20,7,15,15,26,32,22,35
E_ZPV [XC 1],29,29,12,17,31,1,1,3,18,13,...,72,2,47,22,13,13,102,58,113,111
E_T [XC 1],-5,-5,-2,-4,-6,2,-1,0,-5,-3,...,-13,0,-8,-5,2,3,-23,7,-18,-2
DeltaH [XC 1],20,20,3,7,20,-12,-6,-4,7,3,...,46,-8,19,10,0,1,53,34,73,74
E_ZPV [XC 2],23,28,13,37,28,9,2,6,22,14,...,72,1,47,25,9,12,111,74,112,115
E_T [XC 2],-4,-5,-2,-5,-5,3,-1,-1,-5,-4,...,-12,0,-8,-4,3,3,-15,9,-15,1
DeltaH [XC 2],15,20,4,26,18,-3,-5,-2,10,4,...,46,-9,19,13,-3,0,69,51,75,80
E_ZPV [XC 3],10,10,0,28,28,-1,0,0,12,10,...,69,-2,44,6,5,7,96,71,90,105
E_T [XC 3],-2,-2,0,-5,-5,2,0,0,-3,-3,...,-12,1,-8,-1,3,3,-7,13,-9,3


#### Verifying accuracy of freezing surface in vibrational frequency calculations

In [18]:
dft_deltaH_surface_contribution_dict = {molecule: {'Molecule': '', 'Molecule+Surface': ''} for molecule in ['CO', 'H2O Monomer','CO2 Physisorbed','CO2 Chemisorbed']}

surface = 'MgO'

for molecule in ['CO', 'H2O Monomer','CO2 Physisorbed','CO2 Chemisorbed']:

    if 'CO2' in molecule and surface == 'MgO':
        expt_molecule_label =  molecule.split()[1] + ' ' + molecule.split()[0]
    else:
        expt_molecule_label = molecule.split()[0]

    molecule_label = molecule.replace(' ', '_')
    num_monomer = 1
    temperature = float(expt_hads_df.loc[(expt_hads_df['Surface'] == surface) & (expt_hads_df['Molecule'] == expt_molecule_label), 'Temperature'].values[0])

    deltaH_list = []
    for xc_func in crystal_xc_func_ensemble[surface][:4]:
        e_zpv, etherm, kT, deltaH = calculate_ezpv_etherm(f'Data/Miscellaneous/Etherm_and_EZPV_Surface_Test/{surface}/{molecule_label}/{xc_func}', ['Molecule-Surface','Molecule','Surface'], temperature, num_monomers=num_monomer)
        deltaH_list.append(deltaH)


    dft_deltaH_surface_contribution_dict[molecule]['Molecule'] = f'{round(dft_ezpv_etherm_dict[surface][molecule]["Mean"]["DeltaH"]):.0f} $\pm$ {round(2*dft_ezpv_etherm_dict[surface][molecule]["Mean"]["Error"]):.0f}'

    dft_deltaH_surface_contribution_dict[molecule]['Molecule+Surface'] = f'{round(np.mean(deltaH_list)):.0f} $\pm$ {round(2*np.std(deltaH_list)):.0f}'

# Convert to DataFrame then LaTeX input

dft_deltaH_surface_contribution_df = pd.DataFrame.from_dict(dft_deltaH_surface_contribution_dict)

display(dft_deltaH_surface_contribution_df)

dft_deltaH_surface_contribution_df.columns = ['CO', r'H$_2$O Monomer',r'CO$_2$ Physisorbed',r'CO$_2$ Chemisorbed']

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_deltaH_surface_contribution_df,
    start_input = '\\begin{table}',
    label = 'tab:ethermal_slab_effect',
    caption = r'Comparing the effect of only the molecule vibrational degrees of freedom and the inclusion of surface degrees of freedom on the enthalpy ($\Delta H$) contribution to the adsorption enthalpy for a select few molecules adsorbed on MgO (001). $\Delta H$ is calculated as the mean from an ensemble of 4 DFT models with the error being the 2$\sigma$.',
    end_input = '\\end{table}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Ethermal_Surface_Effect.tex',
    replace_input = {
        'E_ZPV': r'$E_\textrm{ZPV}$',
        'E_therm': r'$E_\textrm{therm}$',
        'DeltaH': r'$\Delta H$'
    },
    rotate_column_header = True
)

,CO,H2O Monomer,CO2 Physisorbed,CO2 Chemisorbed
Molecule,19 $\pm$ 6,47 $\pm$ 5,-10 $\pm$ 5,18 $\pm$ 5
Molecule+Surface,19 $\pm$ 1,48 $\pm$ 5,-8 $\pm$ 2,31 $\pm$ 3


# Computing *E*<sub>int</sub> with the SKZCAM protocol

### SI - *E*<sub>int</sub> as a function of SKZCAM cluster size/number

In [19]:
skzcam_cluster_size_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}
skzcam_cluster_eint_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        for cluster_number in ['-2','-1','1','2','3','4','5','6','7','8']:
            # Looping over possibility of small metal cation ECP core and large metal cation ECP core
            for basis_type in ['aV','awCV']:
                filedir = f'Data/05b-Eint_SKZCAM/{surface}/{molecule_label}/Cluster_{cluster_number}'
                # First obtain the MP2 Eint values 
                outputname = f'MP2_{basis_type}'
                try:
                    # First get DZ Eint value
                    mp2_total_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_total', basis='DZ')/num_monomer
                    # If dictionary does not exist, create it
                    if cluster_number not in skzcam_cluster_eint_dict[surface][molecule]:
                        skzcam_cluster_eint_dict[surface][molecule][cluster_number] = {}
                        skzcam_cluster_size_dict[surface][molecule][cluster_number] = {}


                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 {basis_type}DZ'] = mp2_total_eint
                    skzcam_cluster_size_dict[surface][molecule][cluster_number] = get_skzcam_cluster_size(f'{filedir}/Surface/{outputname}DZ.orca.out')
                except:
                    pass
                    
                try:
                    # Then get CBS(DZ/TZ) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                try:
                    # Then get CBS(TZ/QZ) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_hf', basis=['TZ','QZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='orca', method='mp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}TZ/{basis_type}QZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                # Then obtain the Eint values with CCSD(T)
                outputname = f'LNOCCSDT_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) LNO-CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    ccsd_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lccsd_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='lmp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsd_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint

                except:
                    pass

                output_ccsdt_name = f'DLPNOCCSDT_{basis_type}'
                output_mp2_name = f'DLPNOMP2_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) LNO-CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsdt_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    ccsd_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_ccsdt_name, code='orca', method='dlpnoccsd_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsd_corr_eint


                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_mp2_name, code='orca', method='dlpnomp2_hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    mp2_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = output_mp2_name, code='orca', method='dlpnomp2_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + mp2_corr_eint
                except:
                    pass

                outputname = f'CCSDT_{basis_type}'

                try:
                    # Get the CBS(DZ/TZ) Canonical CCSD(T) Eint value
                    hf_scf_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='hf', basis=['DZ','TZ'], cbs_type='scf_energy')/num_monomer
                    ccsdt_corr_eint = calculate_skzcam_eint(filedir = filedir, outputname = outputname, code='mrcc', method='ccsdt_corr', basis=['DZ','TZ'], cbs_type='correlation_energy')/num_monomer
                    skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] = hf_scf_eint + ccsdt_corr_eint

                except:
                    pass

### SI - Extrapolate *E*<sub>int</sub> to the bulk limit

In [20]:
# Now extrapolating the Eint values to the bulk limit

skzcam_extrap_eint_dict = {surface: {molecule: {} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

latex_input_str = ''
counter = 0
for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        num_monomer = 4 if 'Tetramer' in molecule else 2 if 'Dimer' in molecule else 1
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'
        system_skzcam_all_eint_dict = {} # A dictionary for writing into DataFrame
        system_skzcam_all_eint_dict[r'# of atoms'] = [skzcam_cluster_size_dict[surface][molecule][cluster_number] for cluster_number in ['1','2','3','4','5','6','7']]
        for basis in [f'{basis_type}DZ', f'CBS({basis_type}DZ/{basis_type}TZ)']:
            basis_eint_list = []
            basis_extrap_eint_list = []
            cluster_size_list = []

            for cluster_number in ['1','2','3','4','5','6','7']:
                try:
                    basis_eint_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}']]
                    cluster_size_list += [skzcam_cluster_size_dict[surface][molecule][cluster_number]]

                    if int(cluster_number) > 3:
                        if cluster_number not in skzcam_extrap_eint_dict[surface][molecule]:
                            skzcam_extrap_eint_dict[surface][molecule][cluster_number] = {}

                        # If rutile surface or CO2 chemisorbed on MgO, do not extrapolate as convergence is not smooth and instead take the largest cluster as the bulk limit
                        if 'TiO2' in surface or (surface == 'MgO' and 'CO2 Chemisorbed' == molecule):
                            eint_bulklim = skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}']
                            skzcam_extrap_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}'] = eint_bulklim
                            basis_extrap_eint_list += [eint_bulklim]

                        else: # Otherwise, extrapolate to the bulk limit
                            A_opt, gamma, eint_bulklim = fit_eint(cluster_size_list, basis_eint_list)
                            skzcam_extrap_eint_dict[surface][molecule][cluster_number][f'MP2 {basis}'] = eint_bulklim
                            basis_extrap_eint_list += [eint_bulklim]
                    else:
                        basis_extrap_eint_list += ['-']

                except:
                    basis_eint_list += ['-']
                    basis_extrap_eint_list += ['-']
                    pass
            system_skzcam_all_eint_dict[f'Eint MP2 {basis}'] = basis_eint_list
            system_skzcam_all_eint_dict[f'Eintbulk MP2 {basis}'] = basis_extrap_eint_list

        if basis_type == 'aV':
            extra_method_list = [f'MP2 CBS(awCVDZ/awCVTZ)', f'MP2 CBS({basis_type}TZ/{basis_type}QZ)', f'MP2 CBS(awCVTZ/awCVQZ)', f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)',f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)', f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)',f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' ]
        else:
            extra_method_list = [f'MP2 CBS({basis_type}TZ/{basis_type}QZ)', f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)',f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)']

        for method in extra_method_list:
            if method in skzcam_cluster_eint_dict[surface][molecule]['1']:
                system_skzcam_all_eint_dict[f'Eint {method}'] = [skzcam_cluster_eint_dict[surface][molecule][cluster_number][method] if method in skzcam_cluster_eint_dict[surface][molecule][cluster_number] else '-' for cluster_number in ['1', '2', '3', '4', '5', '6', '7']]
        system_skzcam_all_eint_df = pd.DataFrame.from_dict(system_skzcam_all_eint_dict).T
        system_skzcam_all_eint_df.columns = list(range(1,len(system_skzcam_all_eint_df.columns)+1))
        # Function to round only floats
        system_skzcam_all_eint_df = system_skzcam_all_eint_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)

        # Identify the type of SKZCAM clusters in Figure X of the SI
        SKZCAM_cluster_type = 'none'

        if system_skzcam_all_eint_dict[r'# of atoms'][0] == 6:
            SKZCAM_cluster_type = 'Mg-centered'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 17:
            SKZCAM_cluster_type = 'O-centered'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 10:
            SKZCAM_cluster_type = 'Hollow-centered'
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 34:
            SKZCAM_cluster_type = ''
        elif system_skzcam_all_eint_dict[r'# of atoms'][0] == 29:
            SKZCAM_cluster_type = ''

        # if molecule in adsorbate_config[system]:
        caption = """$E_\\textrm{int}$ (in meV) of the SKZCAM clusters for """ + f"{convert_to_nice_labels['adsorbates'][surface][molecule]} on {convert_to_nice_labels['surface'][surface]}." + f" The clusters correspond to the `{SKZCAM_cluster_type}' type visualized in Figure" + r"~\ref{fig:skzcam_clusters}."

        # Remove the r'# of atoms' column
        system_skzcam_all_eint_df = system_skzcam_all_eint_df.drop(r'# of atoms')

        if 'TiO2' in surface:
            system_skzcam_all_eint_df.columns.name = f"{convert_to_nice_labels['adsorbates'][surface][molecule]} on {convert_to_nice_labels['surface'][surface]}"
        else:
            system_skzcam_all_eint_df.columns.name = f"{convert_to_nice_labels['adsorbates'][surface][molecule]} on {convert_to_nice_labels['surface'][surface]} ({SKZCAM_cluster_type})"            
        
        counter += 1
        if counter == 1:
            latex_input_str += '\n'.join(convert_df_to_latex_input(
    system_skzcam_all_eint_df,
    start_input = '\\begin{table}',
    label = f'tab:system_eint_{surface.lower()}_{molecule_label.lower()}',
    caption = caption,
    end_input = '\\end{table}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-SKZCAM_System_Eint.tex',
    replace_input = {
        'Eintbulk': r'$E_\textrm{int}^\textrm{bulk}$',
        'Eint': r'$E_\textrm{int}$',
        r'#': r'\#'
    },
    output_str = True
    ).splitlines()[7:-4]) + '\n'

        else:
            latex_input_str += '\n'.join(convert_df_to_latex_input(
    system_skzcam_all_eint_df,
    start_input = '\\begin{table}',
    label = f'tab:system_eint_{surface.lower()}_{molecule_label.lower()}',
    caption = caption,
    end_input = '\\end{table}',
    adjustbox = 1.4,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-SKZCAM_System_Eint.tex',
    replace_input = {
        'Eintbulk': r'$E_\textrm{int}^\textrm{bulk}$',
        'Eint': r'$E_\textrm{int}$',
        r'#': r'\#'
    },
    output_str = True
    ).splitlines()[4:-4]) + '\n'
        
# Write out the latex input string
with open('Tables/SI_Table-SKZCAM_System_Eint.tex', 'w') as f:
    f.write(r"""\LTcapwidth=\textwidth
\begin{longtable}{lrrrrrrr}
\caption{\label{tab:skzcam_system_eint} $E_\textrm{int}$ (in meV) of the clusters generated by the SKZCAM protocol for the 19 molecule-surface systems and their studied adsorption configurations. The type of clusters used is given within the brackets and the corresponding size for each cluster is provided in Figure~\ref{fig:skzcam_clusters}.} \\

\toprule
CH$_4$ on MgO (001) (Mg-centered) & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\ 
\midrule
\endfirsthead

\caption[]{(continued)} \\
\endhead

\multicolumn{8}{r}{{Continued on next page}} \\
\endfoot

\bottomrule
\endlastfoot

""")
    f.write(latex_input_str)
    f.write(r"\end{longtable}")


### SI - Validating local approximations in ΔCC correction

In [21]:
# Table comparing the LNO-CCSD(T) estimate against the CCSD(T) estimate for all of the systems

deltacc_methods = ['C-MP2','L-MP2','L-MP2 Error','C-CCSD(T)','LNO-CCSD(T)','LNO-CCSD(T) Error','C-DeltaCC','(L-)DeltaCC','(L-)DeltaCC Error']

deltacc_local_error_dict = {} # {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {method: 0 for method in deltacc_methods} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

deltacc_method_mad_dict = {method: [] for method in deltacc_methods}


for surface in ['MgO','r-TiO2','a-TiO2']:
    cluster_number = '1' if surface == 'MgO' else '-2'
    basis_type = 'aV' if 'TiO2' in surface else 'awCV'
    if surface == 'MgO':
        molecule_list = ['CH4', 'C2H6', 'CO', 'N2O Parallel', 'N2O Tilted', 'NO Dimer', 'H2O Monomer', 'CH3OH Tilted', 'CH3OH Parallel', 'NH3', 'CO2 Physisorbed']
    else:
        molecule_list = molecule_surface_systems[surface]
    for molecule in molecule_list:
        lmp2_type = 'DLPNO-MP2' if (molecule == 'NO' and molecule != 'NO Dimer')  else 'LMP2'
        lccsdt_type = 'DLPNO-CCSD(T)' if (molecule == 'NO' and molecule != 'NO Dimer') else 'LNO-CCSD(T)'
        deltacc_local_error_dict[f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = {
                'C-MP2': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'],
                'L-MP2': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                'L-MP2 Error': abs(skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)']),
                'C-CCSD(T)': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'],
                'LNO-CCSD(T)': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                'LNO-CCSD(T) Error': abs(skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)']),
                'C-DeltaCC': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)'],
                '(L-)DeltaCC': skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)'],
                '(L-)DeltaCC Error': abs((skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lccsdt_type} CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'{lmp2_type} CBS({basis_type}DZ/{basis_type}TZ)']) - (skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']))
            }
        for method in deltacc_methods:
            if 'Error' in method:
                deltacc_method_mad_dict[method] += [deltacc_local_error_dict[f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'][method]]
            else:
                deltacc_method_mad_dict[method] += [0]


deltacc_local_error_df = pd.DataFrame.from_dict(deltacc_local_error_dict).T

# Add a row for the mean absolute deviation
deltacc_local_error_df.loc['MAD'] = [np.mean(deltacc_method_mad_dict[method]) for method in deltacc_methods]

# Convert to int
deltacc_local_error_df = deltacc_local_error_df.round().astype(int)
display(deltacc_local_error_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    deltacc_local_error_df,
    start_input = '\\begin{table}',
    label = f'tab:deltacc_lno_errors',
    caption = r"Comparing canonical (C-)MP2, canonical CCSD(T) and canonical $\Delta$CC against their local variants (i.e., LMP2, LNO-CCSD(T), L-$\Delta$CC). This was compared for the first cluster generated by the SKZCAM protocol for the specific MgO adsorption site in Fig.~\ref{fig:skzcam_clusters}, while it corresponds to cluster `${-}2$' for the \ce{TiO2} surfaces.",
    end_input = '\\end{table}',
    adjustbox = 1,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True,
    filename = 'Tables/SI_Table-DeltaCC_LNO_Errors.tex')


# A plot for the SI to showcase how DeltaCC works
fig, axs = plt.subplots(figsize=(2.4, 2),dpi=1200,constrained_layout=True)


# Bar graph of different functionals in funct_line: Loop over the functionals and plot the MAD
axs.barh([1-0.2],skzcam_cluster_eint_dict['MgO']['CO']['1']['MP2 CBS(awCVDZ/awCVTZ)'],0.4,color=color_dict['blue'],label='MP2')
axs.barh([1+0.3],skzcam_cluster_eint_dict['MgO']['CO']['1']['CCSD(T) CBS(awCVDZ/awCVTZ)'] ,0.4,color=color_dict['yellow'],label='CCSD(T)') #edgecolor='black',alpha=1)

axs.barh([2.5-0.2],skzcam_extrap_eint_dict['MgO']['CO']['5']['MP2 CBS(awCVDZ/awCVTZ)'],0.4,color=color_dict['blue'],alpha=1)
axs.barh([2.5+0.3],skzcam_extrap_eint_dict['MgO']['CO']['5']['MP2 CBS(awCVDZ/awCVTZ)'] + skzcam_cluster_eint_dict['MgO']['CO']['1']['CCSD(T) CBS(awCVDZ/awCVTZ)']  - skzcam_cluster_eint_dict['MgO']['CO']['1']['MP2 CBS(awCVDZ/awCVTZ)'],0.4,color=color_dict['yellow'],edgecolor='black',alpha=1, label='SKZCAM')

axs.set_yticks([1.05,2.55])
axs.set_yticklabels([]) #['Smallest Cluster','Bulk Limit'])

axs.set_xlabel(r'$E_\textrm{int}$ [meV]')
axs.legend(fontsize=7)
axs.set_ylim([3.3,0.3])
axs.set_xlim([-140,-220])

plt.savefig('Figures/SI_Figure-DeltaCC_Example.png')

,C-MP2,L-MP2,L-MP2 Error,C-CCSD(T),LNO-CCSD(T),LNO-CCSD(T) Error,C-DeltaCC,(L-)DeltaCC,(L-)DeltaCC Error
CH$_4$ on MgO (001),-71,-70,0,-91,-86,5,-20,-15,4
C$_2$H$_6$ on MgO (001),-90,-92,2,-116,-112,4,-26,-20,6
CO on MgO (001),-155,-153,2,-165,-160,5,-9,-7,3
Parallel N$_2$O on MgO (001),-206,-203,2,-215,-209,6,-9,-6,3
Tilted N$_2$O on MgO (001),-122,-120,2,-107,-104,4,15,17,2
Dimer NO on MgO (001),-222,-217,5,-205,-199,6,17,18,2
Monomer H$_2$O on MgO (001),-641,-639,2,-670,-665,5,-29,-26,3
Tilted CH$_3$OH on MgO (001),-703,-701,2,-729,-723,7,-27,-22,5
Parallel CH$_3$OH on MgO (001),-443,-442,1,-466,-458,8,-23,-16,6
NH$_3$ on MgO (001),-551,-547,4,-583,-578,5,-32,-31,1


### SI - ΔCC, Δ<sub>basis</sub> and Δ<sub>core</sub> corrections

In [22]:
skzcam_delta_dict = {surface: {molecule: {delta: np.array([0.0,0.0]) for delta in ['DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

skzcam_delta_df_dict = {delta: {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': [] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]} for delta in ['DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'

        deltacc_ccsdt_list = []
        deltacc_ccsd_list = []
        deltabasis_list = []
        deltacore_list = []
        for cluster_number in ['1','2','3','4','5','6','7']:
            # First obtain the deltaCC values                    
            try:
                if f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                    deltacc_ccsdt_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                    deltacc_ccsd_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'LMP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                elif f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number].keys():
                    deltacc_ccsdt_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD(T) CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
                    deltacc_ccsd_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-CCSD CBS({basis_type}DZ/{basis_type}TZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'DLPNO-MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
            except:
                pass      

            # Next obtain the deltaBasis values

            try:
                if f'MP2 CBS({basis_type}TZ/{basis_type}QZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                    deltabasis_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}TZ/{basis_type}QZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']]
            except:
                pass

            # Finally obtain the deltaCore values

            if basis_type == 'aV':
                try:
                    if f'MP2 CBS(awCVTZ/awCVQZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                        deltacore_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(awCVTZ/awCVQZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(aVTZ/aVQZ)']]
                    elif f'MP2 CBS(awCVDZ/awCVTZ)' in skzcam_cluster_eint_dict[surface][molecule][cluster_number]:
                        deltacore_list += [skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(awCVDZ/awCVTZ)'] - skzcam_cluster_eint_dict[surface][molecule][cluster_number][f'MP2 CBS(aVDZ/aVTZ)']]
                except:
                    pass
            
        # Take DeltaCC to be the average of the list and its error to be the maximum deviation from the average
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][0] = np.mean(deltacc_ccsdt_list)
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][1] = np.max(np.abs(np.array(deltacc_ccsdt_list) - skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD(T)]'][0]))   
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][0] = np.mean(deltacc_ccsd_list)
        skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][1] = np.max(np.abs(np.array(deltacc_ccsd_list) - skzcam_delta_dict[surface][molecule]['DeltaCC [CCSD]'][0]))   

        # Take DeltaBasis to be the average of the list and its error to be the maximum deviation from the average
        skzcam_delta_dict[surface][molecule]['DeltaBasis'][0] = np.mean(deltabasis_list)
        skzcam_delta_dict[surface][molecule]['DeltaBasis'][1] = np.max(np.abs(np.array(deltabasis_list) - skzcam_delta_dict[surface][molecule]['DeltaBasis'][0]))

        # Take DeltaCore to be the average of the list and its error to be the maximum deviation from the average
        if len(deltacore_list) > 1:
            skzcam_delta_dict[surface][molecule]['DeltaCore'][0] = np.mean(deltacore_list)
            skzcam_delta_dict[surface][molecule]['DeltaCore'][1] = np.max(np.abs(np.array(deltacore_list) - skzcam_delta_dict[surface][molecule]['DeltaCore'][0]))



        skzcam_delta_df_dict['DeltaCC [CCSD(T)]'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltacc_ccsdt_list + ['-']*(7-len(deltacc_ccsdt_list))
        skzcam_delta_df_dict['DeltaBasis'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltabasis_list + ['-']*(7-len(deltabasis_list))
        skzcam_delta_df_dict['DeltaCore'][f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'] = deltacore_list + ['-']*(7-len(deltacore_list))

delta_label_dict ={
    'DeltaCC [CCSD(T)]': r'$\Delta$CC',
    'DeltaBasis': r'$\Delta_\textrm{Basis}$',
    'DeltaCore': r'$\Delta_\textrm{Core}$'
}

for delta in ['DeltaCC [CCSD(T)]','DeltaBasis','DeltaCore']:
    
    delta_df = pd.DataFrame.from_dict(skzcam_delta_df_dict[delta]).T
    # Only keep first 3 columns
    delta_df = delta_df.iloc[:, :3]
    delta_df.columns = list(range(1,4))

 # for molecule in molecule_surface_systems['r-TiO2'] if molecule in delta_df.index]]

    # Add a column for the mean and another column for the maximum deviation
    delta_df['Mean'] = [skzcam_delta_dict[surface][molecule][delta][0] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]
    delta_df['Error'] = [skzcam_delta_dict[surface][molecule][delta][1] for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]]
    # Convert to int
    delta_df = delta_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)
    # If DeltaCore, only keep rows with TiO2 or NO
    if delta == 'DeltaCore':
        delta_df = delta_df[(delta_df.index.str.contains('TiO') | delta_df.index.str.contains('NO')) & ~delta_df.index.str.contains('Dimer')]
    delta_df.index.name = 'System'
    delta_df.columns.name = 'Cluster'

    # Convert to LaTeX input
    convert_df_to_latex_input(
        delta_df,
        start_input = '\\begin{table}',
        label = f'tab:{delta.split()[0].lower()}',
        caption = f'Comparison of the {delta_label_dict[delta]} values for the clusters generated from SKZCAM protocol for all of the studied systems. The mean is calculated from the set of clusters used, with the error being the maximum deviation from the mean.',
        end_input = '\\end{table}',
        adjustbox = 1,
        center = True,
        df_latex_skip = 0,
        filename = f'Tables/SI_Table-{delta.split()[0]}.tex',
)
    delta_df = delta_df.style.set_caption(delta)
    display(delta_df)


Cluster,1,2,3,Mean,Error
System,,,,,
CH$_4$ on MgO (001),-15,-16,-16,-16,0
Monolayer CH$_4$ on MgO (001),-15,-16,-16,-16,0
C$_2$H$_6$ on MgO (001),-20,-20,-20,-20,0
Monolayer C$_2$H$_6$ on MgO (001),-19,-19,-20,-20,1
CO on MgO (001),-7,-7,-9,-8,1
C$_6$H$_6$ on MgO (001),24,24,25,24,1
Parallel N$_2$O on MgO (001),-6,-7,-7,-6,1
Tilted N$_2$O on MgO (001),17,16,16,16,1
Vertical-Hollow NO on MgO (001),289,-,-,289,0


Cluster,1,2,3,Mean,Error
System,,,,,
CH$_4$ on MgO (001),3,0,2,2,1
Monolayer CH$_4$ on MgO (001),3,1,2,2,1
C$_2$H$_6$ on MgO (001),3,3,4,3,1
Monolayer C$_2$H$_6$ on MgO (001),2,2,4,3,1
CO on MgO (001),0,-1,3,1,2
C$_6$H$_6$ on MgO (001),-15,-15,-2,-11,9
Parallel N$_2$O on MgO (001),-2,-2,2,-1,2
Tilted N$_2$O on MgO (001),-6,-6,-3,-5,2
Vertical-Hollow NO on MgO (001),10,8,8,9,1


Cluster,1,2,3,Mean,Error
System,,,,,
Vertical-Hollow NO on MgO (001),0,0,0,0,0
Vertical-Mg NO on MgO (001),0,0,0,0,0
Bent-Bridge NO on MgO (001),0,0,0,0,0
Bent-Mg NO on MgO (001),0,0,0,0,0
Bent-O NO on MgO (001),0,0,0,0,0
CH$_4$ on TiO$_2$ rutile (110),-2,-3,-,-2,0
Parallel CO$_2$ on TiO$_2$ rutile (110),-8,-7,-,-7,0
Tilted CO$_2$ on TiO$_2$ rutile (110),8,10,-,9,1
H$_2$O on TiO$_2$ rutile (110),37,44,-,41,4


### SI - Final *E*<sub>int</sub> estimate

In [23]:
skzcam_final_eint_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['Final MP2', 'Final CCSD','Final CCSD(T)', 'Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        basis_type = 'aV' if 'TiO2' in surface or ('NO' in molecule and 'NO Dimer' not in molecule) else 'awCV'
        cbs_cluster_number = [cluster_number for cluster_number in skzcam_extrap_eint_dict[surface][molecule].keys() if f'MP2 CBS({basis_type}DZ/{basis_type}TZ)' in skzcam_extrap_eint_dict[surface][molecule][cluster_number].keys()][-1]
        bulk_mp2_eint = skzcam_extrap_eint_dict[surface][molecule][cbs_cluster_number][f'MP2 CBS({basis_type}DZ/{basis_type}TZ)']
        if 'TiO2' in surface or (surface == 'MgO' and molecule == 'CO2 Chemisorbed'):
            bulk_mp2_eint_list = np.array([skzcam_extrap_eint_dict[surface][molecule][str(cluster_number)][f'MP2 {basis_type}DZ'] for cluster_number in range(int(cbs_cluster_number), int(list(skzcam_cluster_eint_dict[surface][molecule].keys())[-1])+1)])
            bulk_mp2_eint_error = np.max(np.abs(skzcam_extrap_eint_dict[surface][molecule][cbs_cluster_number][f'MP2 {basis_type}DZ'] - bulk_mp2_eint_list))
        else:
            bulk_mp2_eint_error = abs(skzcam_extrap_eint_dict[surface][molecule][cbs_cluster_number][f'MP2 {basis_type}DZ'] - skzcam_extrap_eint_dict[surface][molecule][list(skzcam_cluster_eint_dict[surface][molecule].keys())[-1]][f'MP2 {basis_type}DZ'])
        skzcam_final_eint_dict[surface][molecule]['Bulk MP2 Eint'] = [bulk_mp2_eint, bulk_mp2_eint_error]

        for quantity in ['DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']:
            skzcam_final_eint_dict[surface][molecule][quantity] = skzcam_delta_dict[surface][molecule][quantity]

        # Now obtain the final Eint value for MP2
        skzcam_final_eint_dict[surface][molecule]['Final MP2'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final MP2'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaBasis','DeltaCore']]))

        # Now obtain the final Eint value for CCSD
        skzcam_final_eint_dict[surface][molecule]['Final CCSD'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final CCSD'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore']]))

        # Now obtain the final Eint value for CCSD(T)
        skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0] = np.sum([skzcam_final_eint_dict[surface][molecule][quantity][0] for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']])
        skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][1] = np.sqrt(np.sum([skzcam_final_eint_dict[surface][molecule][quantity][1]**2 for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaBasis','DeltaCore']]))

final_eint_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(skzcam_final_eint_dict[surface][molecule][quantity][0]))} ± {int(round(skzcam_final_eint_dict[surface][molecule][quantity][1]))}' for quantity in ['Bulk MP2 Eint', 'DeltaCC [CCSD(T)]', 'DeltaCC [CCSD]', 'DeltaBasis','DeltaCore','Final MP2', 'Final CCSD','Final CCSD(T)']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

# Convert to DataFrame
final_eint_df = pd.DataFrame.from_dict(final_eint_df_dict).T

display(final_eint_df)
final_eint_df.index.name = 'System'
final_eint_df.columns.name = 'Quantity'
final_eint_df.columns = [r'$E_\textrm{int}^\textrm{bulk MP2}$', r'$\Delta$CC [CCSD(T)]', r'$\Delta$CC [CCSD]', r'$\Delta_\textrm{Basis}$', r'$\Delta_\textrm{Core}$', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [MP2]', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [CCSD]', r'$E_\textrm{int}^\textrm{autoSKZCAM}$ [CCSD(T)]']

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    final_eint_df,
    start_input = '\\begin{table}',
    label = f'tab:final_eint',
    caption = r"Final E$_\textrm{int}$ values for the systems studied in this work. We show the individual terms which make up these final $E_\textrm{int}$ values and also give Final MP2, CCSD and CCSD(T) estimates, where the latter two are obtained by adding the $\Delta$CC values to the final MP2 E$_\textrm{int}$ value.",
    end_input = '\\end{table}',
    adjustbox = 0.9,
    center = True,
    df_latex_skip = 0,
    replace_input = {
        '±': r'$\pm$'
    },
    rotate_column_header = True,
    filename = 'Tables/SI_Table-Final_Eint.tex')


,Bulk MP2 Eint,DeltaCC [CCSD(T)],DeltaCC [CCSD],DeltaBasis,DeltaCore,Final MP2,Final CCSD,Final CCSD(T)
CH$_4$ on MgO (001),-108 ± 2,-16 ± 0,18 ± 7,2 ± 1,0 ± 0,-106 ± 2,-88 ± 7,-122 ± 2
Monolayer CH$_4$ on MgO (001),-107 ± 2,-16 ± 0,18 ± 7,2 ± 1,0 ± 0,-105 ± 3,-87 ± 7,-121 ± 3
C$_2$H$_6$ on MgO (001),-158 ± 3,-20 ± 0,32 ± 13,3 ± 1,0 ± 0,-155 ± 4,-122 ± 14,-175 ± 4
Monolayer C$_2$H$_6$ on MgO (001),-144 ± 3,-20 ± 1,29 ± 13,3 ± 1,0 ± 0,-141 ± 3,-112 ± 13,-161 ± 3
CO on MgO (001),-200 ± 3,-8 ± 1,33 ± 8,1 ± 2,0 ± 0,-199 ± 3,-166 ± 9,-207 ± 4
C$_6$H$_6$ on MgO (001),-460 ± 2,24 ± 1,164 ± 24,-11 ± 9,0 ± 0,-470 ± 9,-307 ± 26,-446 ± 9
Parallel N$_2$O on MgO (001),-249 ± 2,-6 ± 1,40 ± 6,-1 ± 2,0 ± 0,-250 ± 3,-210 ± 7,-256 ± 3
Tilted N$_2$O on MgO (001),-179 ± 3,16 ± 1,64 ± 11,-5 ± 2,0 ± 0,-184 ± 4,-120 ± 11,-168 ± 4
Vertical-Hollow NO on MgO (001),-244 ± 1,289 ± 0,334 ± 0,9 ± 1,0 ± 0,-235 ± 2,98 ± 2,54 ± 2
Vertical-Mg NO on MgO (001),26 ± 2,-68 ± 0,-49 ± 0,5 ± 4,0 ± 0,32 ± 4,-17 ± 4,-36 ± 4


# Contributions for the cohesive and conformational energy from WFT in selected systems

### Conformational energy *E*<sub>conf</sub> for chemisorbed CO<sub>2</sub> on MgO

In [24]:
wft_econf = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in crystal_xc_func_ensemble['MgO']} for xc_func_sp in ['CCSD(T)','CCSD','MP2']} for molecule in ['CO2 Chemisorbed']}

# Now get the Econf values for chemisorbed CO2 and NO dimer
for molecule in ['CO2 Chemisorbed']:
    molecule_label = molecule.replace(' ', '_')
    num_monomer = 2 if 'NO Dimer' in molecule else 1
    for xc_func_geom in ['02_revPBE-D4','06_B3LYP-D2-Ne']:
        filedir = f'Data/07-Econf/MgO/{molecule_label}/07_CCSDT/{xc_func_geom}'
        hf_scf_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        mp2_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='mp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        ccsd_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='ccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        ccsdt_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = f'CCSDT_aV', code='mrcc', method='ccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Molecule-Surface','Molecule'])/num_monomer
        wft_econf[molecule]['CCSD(T)'][xc_func_geom] = hf_scf_energy + ccsdt_corr_energy
        wft_econf[molecule]['CCSD'][xc_func_geom] = hf_scf_energy + ccsd_corr_energy
        wft_econf[molecule]['MP2'][xc_func_geom] = hf_scf_energy + mp2_corr_energy

econf_df_dict = {convert_to_nice_labels["xc_functionals"][xc_func_sp]: dft_econf['CO2 Chemisorbed'][xc_func_sp]['02_revPBE-D4'] for xc_func_sp in crystal_xc_func_ensemble['MgO']}
for wft_method in ['MP2','CCSD','CCSD(T)']:
    econf_df_dict[wft_method] = wft_econf['CO2 Chemisorbed'][wft_method]['02_revPBE-D4']

econf_df = pd.DataFrame.from_dict(econf_df_dict, orient='index')

# Round to nearest integer
econf_df = econf_df.round().astype(int)
econf_df.columns = [r'$E_\textrm{conf}$ [meV]']

display(econf_df)

convert_df_to_latex_input(
    econf_df,
    start_input = '\\begin{table}',
    label = f'tab:econf_co2',
    caption = r"Comparison of the $E_\textrm{conf}$ values for chemisorbed CO$_2$ chemisorbed on MgO for several DFT models as well as MP2, CCSD and CCSD(T).",
    end_input = '\\end{table}',
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Econf.tex')


,$E_\textrm{conf}$ [meV]
PBE-D2[Ne],1841
revPBE-D4,1797
vdW-DF,1763
rev-vdW-DF2,1836
PBE0-D4,2153
B3LYP-D2[Ne],2075
MP2,2045
CCSD,2304
CCSD(T),2094


### Cohesive energy *E*<sub>coh</sub> for the alkane monolayers on MgO

In [25]:
wft_cluster_monolayer_ecoh = { molecule : {xc_func_sp : {xc_func_geom : 0 for xc_func_geom in ['02_revPBE-D4']} for xc_func_sp in ['CCSD(T)','CCSD','MP2','revPBE-D4']} for molecule in ['CH4 Monolayer', 'C2H6 Monolayer', 'CH3OH Tetramer','H2O Tetramer']}

monolayer_ecoh_df_dict = {molecule: {}  for molecule in ['CH4 Monolayer', 'C2H6 Monolayer']}

# First get the DFT Ecoh values for the monolayers
for molecule in ['CH4 Monolayer', 'C2H6 Monolayer']:
    molecule_label = molecule.replace(' ', '_')
    for xc_func in crystal_xc_func_ensemble['MgO']:
        monolayer_ecoh_df_dict[molecule][convert_to_nice_labels["xc_functionals"][xc_func]] = {'Ecoh gas 2B': 0.0, 'Ecoh gas': 0.0, 'Ecoh': 0.0}
        monolayer_ecoh_df_dict[molecule][convert_to_nice_labels["xc_functionals"][xc_func]]['Ecoh'] = dft_cluster_monolayer_ecoh[molecule][xc_func]['02_revPBE-D4']
    for xc_func in crystal_xc_func_ensemble['MgO']:
        monolayer_ecoh_df_dict[molecule][convert_to_nice_labels["xc_functionals"][xc_func]]['Ecoh gas'] = dft_monolayer_ecoh_gas[molecule][xc_func]['02_revPBE-D4']



# Now get the Ecoh values for the monolayers using a two-body many-body expansion
for molecule in ['CH4 Monolayer', 'C2H6 Monolayer']:
    two_body_terms = 68 if molecule == 'CH4 Monolayer' else 72 if molecule == 'C2H6 Monolayer' else 0
    molecule_label = molecule.replace(' ', '_')
    hf_scf_energy = 0
    mp2_corr_energy = 0
    ccsd_corr_energy = 0
    ccsdt_corr_energy = 0
    revpbe_d4_energy = 0
    for dimer_idx in range(1,two_body_terms+1):
        filedir = f'Data/08-Ecoh/MgO/{molecule_label}/07_CCSDT_2B/02_revPBE-D4/{dimer_idx:02d}'
        hf_scf_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        mp2_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnomp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        ccsd_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnoccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        ccsdt_corr_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'FNOCCSDT_aV', code='mrcc', method='fnoccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2
        filedir = f'Data/08-Ecoh/MgO/{molecule_label}/02_revPBE-D4_2B/02_revPBE-D4/{dimer_idx:02d}'

        revpbe_d4_energy += calculate_skzcam_eint(filedir = filedir, outputname = f'revPBE-D4_QZVPD', code='orca', method='dft_total', basis='', cbs_type='scf_energy', structure_labels=['Dimer','Monomer_A','Monomer_B'])/2

    wft_cluster_monolayer_ecoh[molecule]['revPBE-D4']['02_revPBE-D4'] = revpbe_d4_energy

    wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'] = hf_scf_energy + ccsdt_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']
    wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4'] = hf_scf_energy + ccsd_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']
    wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4'] = hf_scf_energy + mp2_corr_energy - revpbe_d4_energy + dft_cluster_monolayer_ecoh[molecule]['02_revPBE-D4']['02_revPBE-D4']

    monolayer_ecoh_df_dict[molecule]['revPBE-D4']['Ecoh gas 2B'] =  revpbe_d4_energy
    monolayer_ecoh_df_dict[molecule]['MP2'] = {'Ecoh gas 2B': hf_scf_energy + mp2_corr_energy, 'Ecoh gas':  hf_scf_energy + mp2_corr_energy - revpbe_d4_energy + dft_monolayer_ecoh_gas[molecule]['02_revPBE-D4']['02_revPBE-D4'], 'Ecoh': wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4']}
    monolayer_ecoh_df_dict[molecule]['CCSD'] = {'Ecoh gas 2B': hf_scf_energy + ccsd_corr_energy, 'Ecoh gas': hf_scf_energy + ccsd_corr_energy - revpbe_d4_energy + dft_monolayer_ecoh_gas[molecule]['02_revPBE-D4']['02_revPBE-D4'], 'Ecoh': wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4']}
    monolayer_ecoh_df_dict[molecule]['CCSD(T)'] = {'Ecoh gas 2B': hf_scf_energy + ccsdt_corr_energy, 'Ecoh gas': hf_scf_energy + ccsdt_corr_energy - revpbe_d4_energy + dft_monolayer_ecoh_gas[molecule]['02_revPBE-D4']['02_revPBE-D4'], 'Ecoh': wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4']}

# Convert to multiindex DataFrame
monolayer_ecoh_df = pd.DataFrame.from_dict({
    (molecule, quantity): {
        method: monolayer_ecoh_df_dict[molecule][method][quantity]
        for method in monolayer_ecoh_df_dict[molecule]
        if quantity in monolayer_ecoh_df_dict[molecule][method]
    }
    for molecule in monolayer_ecoh_df_dict
    for method in monolayer_ecoh_df_dict[molecule]
    for quantity in monolayer_ecoh_df_dict[molecule][method]
})
monolayer_ecoh_df = monolayer_ecoh_df.round().astype(int)

display(monolayer_ecoh_df)
convert_df_to_latex_input(
    monolayer_ecoh_df,
    start_input = '\\begin{table}',
    label = f'tab:monolayer_ecoh',
    caption = r"Table showcasing how the final CCSD(T)-quality $E_\textrm{coh}$ is computed. CCSD(T) (and other WFT methods) is used to compute the two-body (2B) contribution to the many-body expansion of the $E_\textrm{coh}^\textrm{gas}$ - the cohesive energy of the alkane monolayer in the absence of the MgO surface. This contribution is used to correct the 2B contribution to $E_\textrm{coh}^\textrm{gas}$ for revPBE-D4 (i.e., it is used to account for 3B and beyond contributions). Finally, we reach the final $E_\textrm{coh}$ by incorporating the effect of the MgO surface at the revPBE-D4 level (i.e., the difference between $E_\textrm{coh}$ and $E_\textrm{coh}^\textrm{gas}$).",
    end_input = '\\end{table}',
    replace_input = {
        'Ecoh gas 2B': r'$E_\textrm{coh}^\textrm{2B gas}$',
        'Ecoh gas': r'$E_\textrm{coh}^\textrm{gas}$',
        'Ecoh': r'$E_\textrm{coh}$',
        '& 0': '& -'
    },
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Monolayer_Ecoh.tex')

CH4 Monolayer               C2H6 Monolayer              
               Ecoh gas 2B Ecoh gas Ecoh    Ecoh gas 2B Ecoh gas Ecoh
PBE-D2[Ne]               0      -53  -40              0     -125 -106
revPBE-D4              -46      -42  -31            -79      -62  -48
vdW-DF                   0      -72  -58              0      -88  -67
rev-vdW-DF2              0      -40  -43              0      -69  -83
PBE0-D4                  0        0    0              0        0    0
B3LYP-D2[Ne]             0        0    0              0        0    0
MP2                    -15      -11    0            -58      -41  -27
CCSD                   -33      -29  -18            -59      -43  -29
CCSD(T)                -40      -37  -25            -92      -75  -61

### Cohesive energy *E*<sub>coh</sub> for the H<sub>2</sub>O and CH<sub>3</sub>OH tetramers on MgO

In [26]:
tetramer_ecoh_df_dict = {molecule.split()[0]: {}  for molecule in ['CH3OH Tetramer', 'H2O Tetramer']}

# Calculate wave-function theory Ecoh values for the clusters
for molecule in ['CH3OH Tetramer','H2O Tetramer']:
    molecule_label = molecule.replace(' ', '_')
    filedir = f'Data/08-Ecoh/MgO/{molecule_label}/07_CCSDT/02_revPBE-D4'
    hf_scf_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='hf', basis=['TZ','QZ'], cbs_type='scf_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    mp2_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnomp2_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    ccsd_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnoccsd_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    ccsdt_corr_energy = calculate_skzcam_eint(filedir = filedir, outputname = 'FNOCCSDT_aV', code='mrcc', method='fnoccsdt_corr', basis=['TZ','QZ'], cbs_type='correlation_energy', structure_labels=['Cluster','Cluster_Monomer_1','Cluster_Monomer_2','Cluster_Monomer_3','Cluster_Monomer_4'])
    wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'] = (hf_scf_energy + ccsdt_corr_energy)/4 
    wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4'] = (hf_scf_energy + ccsd_corr_energy)/4
    wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4'] = (hf_scf_energy + mp2_corr_energy)/4
    
    for xc_func in crystal_xc_func_ensemble['MgO']:
        tetramer_ecoh_df_dict[molecule.split()[0]][convert_to_nice_labels["xc_functionals"][xc_func]] = dft_cluster_monolayer_ecoh[molecule][xc_func]['02_revPBE-D4']

    tetramer_ecoh_df_dict[molecule.split()[0]]['MP2'] = wft_cluster_monolayer_ecoh[molecule]['MP2']['02_revPBE-D4']
    tetramer_ecoh_df_dict[molecule.split()[0]]['CCSD'] = wft_cluster_monolayer_ecoh[molecule]['CCSD']['02_revPBE-D4']
    tetramer_ecoh_df_dict[molecule.split()[0]]['CCSD(T)'] = wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4']

# Convert to multiindex DataFrame
tetramer_ecoh_df = pd.DataFrame.from_dict(tetramer_ecoh_df_dict)
tetramer_ecoh_df = tetramer_ecoh_df.round().astype(int)
display(tetramer_ecoh_df)

tetramer_ecoh_df.columns = [r'\ce{CH3OH}', r'\ce{H2O}']

convert_df_to_latex_input(
    tetramer_ecoh_df,
    start_input = '\\begin{table}',
    label = f'tab:tetramer_ecoh',
    caption = r"Comparison between several DFT models, MP2 and CCSD against CCSD(T) for the cohesive energy $E_\textrm{coh}$ (per monomer) of the tetramer \ce{CH3OH} and \ce{H2O} cluster.",
    end_input = '\\end{table}',
    replace_input = {
        'Ecoh gas 2B': r'$E_\textrm{coh}^\textrm{2B gas}$',
        'Ecoh gas': r'$E_\textrm{coh}^\textrm{gas}$',
        'Ecoh': r'$E_\textrm{coh}$',
        '& 0': '& -'
    },
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Tetramer_Ecoh.tex')

,CH3OH,H2O
PBE-D2[Ne],-380,-309
revPBE-D4,-328,-261
vdW-DF,-291,-240
rev-vdW-DF2,-357,-289
PBE0-D4,-367,-302
B3LYP-D2[Ne],-371,-303
MP2,-341,-276
CCSD,-311,-269
CCSD(T),-336,-281


### Cohesive energy *E*<sub>coh</sub> for the NO dimer on MgO

In [27]:
no_dimer_ecoh_df_dict = {}

# Add the DFT Ecoh values for the dimers
for xc_func in crystal_xc_func_ensemble['MgO']:
    no_dimer_ecoh_df_dict[convert_to_nice_labels["xc_functionals"][xc_func]] = int(round(dft_cluster_monolayer_ecoh['NO Dimer'][xc_func][xc_func]))

no_dimer_ecoh_df_dict['CCSD(T)'] = int(round(-1.03/2*kcalmol_to_meV))
no_dimer_ecoh_df_dict['Experiment'] = f'-{2.8/2*kcalmol_to_meV:.0f} to -{3.8/2*kcalmol_to_meV:.0f}'
no_dimer_ecoh_df_dict['MRMP2(18,14)'] = int(round(-3.46/2*kcalmol_to_meV))

dimer_ecoh_df = pd.DataFrame.from_dict(no_dimer_ecoh_df_dict, orient='index')
dimer_ecoh_df.columns = [r'$E_\textrm{coh}$ [meV]']
display(dimer_ecoh_df)

dimer_ecoh_df.rename(index={
    dimer_ecoh_df.index[6]: r'CCSD(T)~\cite{tobitaCriticalComparisonSinglereference2003}',
    dimer_ecoh_df.index[7]: r'Experiment~\cite{ivanicHighlevelTheoreticalStudy2012a}',
    dimer_ecoh_df.index[8]: r'MRMP2(18,14)~\cite{ivanicHighlevelTheoreticalStudy2012a}'
}, inplace=True)

dimer_ecoh_df.columns.name = 'Method'

convert_df_to_latex_input(
    dimer_ecoh_df,
    start_input = '\\begin{table}',
    label = f'tab:dimer_ecoh',
    caption = r"Comparison of experiment and MRMP2 against several DFT models and CCSD(T) for the cohesive energy $E_\textrm{coh}$ (per monomer) of the NO dimer.",
    end_input = '\\end{table}',
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-NO_Dimer_Ecoh.tex')

,$E_\textrm{coh}$ [meV]
PBE-D2[Ne],-239
revPBE-D4,-192
vdW-DF,-160
rev-vdW-DF2,-281
PBE0-D4,53
B3LYP-D2[Ne],63
CCSD(T),-22
Experiment,-61 to -82
"MRMP2(18,14)",-75


# Final *E*<sub>ads</sub> and *H*<sub>ads</sub> estimates from autoSKZCAM

### Final *E*<sub>ads</sub> estimates

In [28]:
autoskzcam_final_eads_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['SKZCAM Eint','DFT Erlx', 'WFT Ecoh', 'WFT Econf', 'Final']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')

        # First obtain the SKZCAM Eint values
        autoskzcam_final_eads_dict[surface][molecule]['SKZCAM Eint'] = [skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0], skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][1]]

        # Then obtain the DFT Erlx values
        autoskzcam_final_eads_dict[surface][molecule]['DFT Erlx'] = [dft_erlx_dict[surface][molecule]['02_revPBE-D4'], dft_geom_rmse_dict[surface][molecule]['02_revPBE-D4']]

        # Then obtain the CCSD(T) Ecoh values
        if molecule in wft_cluster_monolayer_ecoh:
            autoskzcam_final_eads_dict[surface][molecule]['WFT Ecoh'] = [wft_cluster_monolayer_ecoh[molecule]['CCSD(T)']['02_revPBE-D4'], 0]

        if molecule in ['NO Dimer']:
            autoskzcam_final_eads_dict[surface][molecule]['WFT Ecoh'] = [float(no_dimer_ecoh_df_dict['MRMP2(18,14)']),0]

        # Then obtain the CCSD(T) Econf values
        if molecule in wft_econf:
            autoskzcam_final_eads_dict[surface][molecule]['WFT Econf'] = [wft_econf[molecule]['CCSD(T)']['02_revPBE-D4'], 0]

        autoskzcam_final_eads_dict[surface][molecule]['Final'] = [np.sum([autoskzcam_final_eads_dict[surface][molecule][quantity][0] for quantity in ['SKZCAM Eint','DFT Erlx', 'WFT Ecoh', 'WFT Econf']]), np.sqrt(np.sum([autoskzcam_final_eads_dict[surface][molecule][quantity][1]**2 for quantity in ['SKZCAM Eint','DFT Erlx', 'WFT Ecoh', 'WFT Econf']]))]


### Final *H*<sub>ads</sub> estimates

In [29]:
autoskzcam_final_hads_dict = {surface: {molecule: {quantity: [0.0,0.0] for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']} for molecule in molecule_surface_systems[surface]} for surface in ['MgO','r-TiO2','a-TiO2']}

for surface in ['MgO','r-TiO2','a-TiO2']:
    for molecule in molecule_surface_systems[surface]:
        molecule_label = molecule.replace(' ', '_')

        # First obtain the SKZCAM Eint values
        autoskzcam_final_hads_dict[surface][molecule]['autoSKZCAM Eads'] = [autoskzcam_final_eads_dict[surface][molecule]['Final'][0], autoskzcam_final_eads_dict[surface][molecule]['Final'][1]]

        # Then obtain the DFT DeltaH values
        autoskzcam_final_hads_dict[surface][molecule]['DFT DeltaH'] = [dft_ezpv_etherm_dict[surface][molecule]['Mean']['DeltaH'], dft_ezpv_etherm_dict[surface][molecule]['Mean']['Error']]

        autoskzcam_final_hads_dict[surface][molecule]['Final'] = [np.sum([autoskzcam_final_hads_dict[surface][molecule][quantity][0] for quantity in ['autoSKZCAM Eads', 'DFT DeltaH']]), np.sqrt(np.sum([autoskzcam_final_hads_dict[surface][molecule][quantity][1]**2 for quantity in ['autoSKZCAM Eads', 'DFT DeltaH']]))]


# autoskzcam_final_hads_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][1]))}' for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}

# autoskzcam_final_eads_df_dict = {f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': {quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' for quantity in ['SKZCAM Eint','DFT Erlx', 'CCSD(T) Ecoh', 'CCSD(T) Econf', 'Final']} for surface in ['MgO','r-TiO2','a-TiO2'] for molecule in molecule_surface_systems[surface]}


autoskzcam_hads_terms_df_dict = {
    f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}': 
    {**{
        quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['SKZCAM Eint', 'DFT Erlx', 'WFT Ecoh', 'WFT Econf']
    },
    **{
        quantity: f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][0]))}' 
        for quantity in ['DFT Erlx', 'WFT Ecoh', 'WFT Econf']
    },  
    **{
        'epsilon geom': f'{int(round(autoskzcam_final_eads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['DFT Erlx']
    },      
    **{
        quantity: f'{int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][0]))} ± {int(round(autoskzcam_final_hads_dict[surface][molecule][quantity][1]))}' 
        for quantity in ['autoSKZCAM Eads', 'DFT DeltaH', 'Final']
    }}
    for surface in ['MgO', 'r-TiO2', 'a-TiO2'] 
    for molecule in molecule_surface_systems[surface]
}


autoskzcam_hads_terms_df = pd.DataFrame.from_dict(autoskzcam_hads_terms_df_dict).T
autoskzcam_hads_terms_df.columns.name = 'System'
display(autoskzcam_hads_terms_df)
autoskzcam_hads_terms_df.columns = [r'$E_\textrm{int}^\textrm{SKZCAM}$', r'$E_\textrm{rlx}^\textrm{DFT}$', r'$E_\textrm{coh}^\textrm{CCSD(T)}$', r'$E_\textrm{conf}^\textrm{CCSD(T)}$', r'$\epsilon_\textrm{geom}$', r'$E_\textrm{ads}^\textrm{autoSKZCAM}$', r'$\Delta H^\textrm{DFT}$', r'$H_\textrm{ads}^\textrm{autoSKZCAM}$']
autoskzcam_hads_terms_df.columns.name = 'Quantity'

convert_df_to_latex_input(
    autoskzcam_hads_terms_df,
    start_input = '\\begin{table}',
    label = 'tab:autoskzcam_hads_terms',
    caption = r"A summary of the terms which make up the final autoSKZCAM $E_\textrm{ads}$ and $H_\textrm{ads}$ estimate. Here, the final $E_\textrm{ads}$ value is the sum of $E_\textrm{int}^\textrm{SKZCAM}$, $E_\textrm{rlx}^\textrm{DFT}$, $E_\textrm{coh}^\textrm{CCSD(T)}$ and $E_\textrm{conf}^\textrm{CCSD(T)}$ values, where the last two terms are only included for systems where they are calculated in Sec.~\ref{sec:wft_ecoh_econf}. The errors due to $E_\textrm{rlx}^\textrm{DFT}$ and from using the revPBE-D4 geometry for all the $E_\textrm{ads}$ terms is encapsulated in $\epsilon_\textrm{geom}$ from an ensemble of DFT models. An additional $\Delta$H term is also calculated, further calculated from a DFT model ensemble.",
    end_input = '\\end{table}',
    filename = 'Tables/SI_Table-AutoSKZCAM_Hads_Terms.tex',
    adjustbox = 0.9,
    center = True,
    df_latex_skip = 0,
    rotate_column_header = True
)


System,SKZCAM Eint,DFT Erlx,WFT Ecoh,WFT Econf,epsilon geom,autoSKZCAM Eads,DFT DeltaH,Final
CH$_4$ on MgO (001),-122 ± 2,2,0,0,10,-120 ± 10,11 ± 13,-109 ± 16
Monolayer CH$_4$ on MgO (001),-121 ± 3,1,-25,0,27,-145 ± 28,13 ± 14,-132 ± 31
C$_2$H$_6$ on MgO (001),-175 ± 4,1,0,0,17,-174 ± 17,-1 ± 10,-175 ± 20
Monolayer C$_2$H$_6$ on MgO (001),-161 ± 3,4,-61,0,70,-218 ± 70,15 ± 14,-203 ± 72
CO on MgO (001),-207 ± 4,8,0,0,10,-198 ± 11,19 ± 3,-180 ± 11
C$_6$H$_6$ on MgO (001),-446 ± 9,26,0,0,50,-420 ± 51,-10 ± 9,-430 ± 51
Parallel N$_2$O on MgO (001),-256 ± 3,3,0,0,10,-253 ± 10,-6 ± 1,-259 ± 10
Tilted N$_2$O on MgO (001),-168 ± 4,10,0,0,23,-157 ± 23,-4 ± 4,-161 ± 24
Vertical-Hollow NO on MgO (001),54 ± 2,29,0,0,45,83 ± 45,6 ± 6,90 ± 46
Vertical-Mg NO on MgO (001),-36 ± 4,9,0,0,21,-27 ± 21,2 ± 3,-25 ± 21


# Comparison between theory and/or experiment

### *H*<sub>ads</sub> comparison between autoSKZCAM and experiment

In [30]:
# Keep only Surface, Molecule, Hads and Error columns in expt_hads_df
hads_comparison_df = expt_hads_df[['Surface','Molecule','Hads','Error']]
# Remove the 'Physisorbed CO2' row
hads_comparison_df = hads_comparison_df[hads_comparison_df['Molecule'] != 'Physisorbed CO2']
# Rename the columns to Expt. Hads and Expt. Error
hads_comparison_df.columns = ['Surface','Molecule','Expt. Hads','Expt. Error']

# Save autoSKZCAM and experiment hads into a nested dictionary with surface -> molecule -> experiment or autosKZCAM for plotting in next cell
hads_comparison_dict = {}

for idx, row in hads_comparison_df.iterrows():
    surface = row["Surface"]
    molecule = row["Molecule"]
    
    if surface not in hads_comparison_dict:
        hads_comparison_dict[surface] = {}
        
    hads_comparison_dict[surface][molecule] = {
        'Experiment': [int(row['Expt. Hads']), int(row['Expt. Error'])],
        'autoSKZCAM': [0, 0]
    }

# Add the AutoSKZCAM Hads and Error columns to the DataFrame
# Loop over the DataFrame and add the AutoSKZCAM Hads and Error values
autoskzcam_ground_state_hads = []
autoskzcam_ground_state_hads_error = []
system_label_list = []
hads_delta_min_list = []

for idx, row in hads_comparison_df.iterrows():
    surface = row['Surface']
    molecule = row['Molecule']

    system_expt_hads = int(row['Expt. Hads'])
    system_expt_error = int(row['Expt. Error'])
    system_expt_hads_lower_bound = system_expt_hads - system_expt_error
    system_expt_hads_upper_bound = system_expt_hads + system_expt_error
    
    if len(molecule.split()) == 2:
        if molecule.split()[0] == 'Cluster':
            cluster_size = 'Tetramer' if molecule.split()[1] in ['H2O','CH3OH'] else 'Dimer'
            molecule_label = molecule.split()[1] + ' ' + cluster_size
        else:
            molecule_label = molecule.split()[1] + ' ' + molecule.split()[0]
    elif surface == 'MgO' and molecule == 'H2O':
        molecule_label = 'H2O Monomer'
    elif surface == 'MgO' and molecule == 'N2O':
        molecule_label = 'N2O Parallel'
    elif surface == 'r-TiO2' and molecule == 'CO2':
        molecule_label = 'CO2 Tilted'

    else:
        molecule_label = molecule

    system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule_label]} on {convert_to_nice_labels["surface"][surface]}'

    system_label_list += [system_label]
    autoskzcam_ground_state_hads += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0]))]
    autoskzcam_ground_state_hads_error += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]))]
    hads_comparison_dict[surface][molecule]['autoSKZCAM'] = [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0])), int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]) )]

    system_autoskzcam_hads_lower_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] - autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    system_autoskzcam_hads_upper_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] + autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    # Find the minimum difference between the experimental Hads and the AutoSKZCAM Hads errors
    if max(system_expt_hads_lower_bound, system_autoskzcam_hads_lower_bound) <= min(system_expt_hads_upper_bound, system_autoskzcam_hads_upper_bound):
        delta_min = 0
    else:
        delta_min = min(abs(system_expt_hads_upper_bound - system_autoskzcam_hads_lower_bound), abs(system_expt_hads_lower_bound - system_autoskzcam_hads_upper_bound))


    hads_delta_min_list += [int(round(delta_min))]


# Add the system labels, AutoSKZCAM Hads and Error columns to the DataFrame
hads_comparison_df['autoSKZCAM Hads'] = autoskzcam_ground_state_hads
hads_comparison_df['autoSKZCAM Error'] = autoskzcam_ground_state_hads_error
hads_comparison_df['DeltaMin'] = hads_delta_min_list
hads_comparison_df['System'] = system_label_list

# Rearrange the columns
hads_comparison_df = hads_comparison_df[['System','Expt. Hads','Expt. Error','autoSKZCAM Hads','autoSKZCAM Error','DeltaMin']]

# Round to nearest integer
hads_comparison_df = hads_comparison_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)
hads_comparison_df.columns.name = 'System' 


hads_comparison_dict['MgO']['CO2'] = hads_comparison_dict['MgO']['Chemisorbed CO2']
del hads_comparison_dict['MgO']['Chemisorbed CO2']

display(hads_comparison_df)

convert_df_to_latex_input(
    hads_comparison_df,
    start_input = '\\begin{table}',
    label = 'tab:hads_comparison',
    caption = r"Comparison of the experimental and autoSKZCAM $H_\textrm{ads}$ values for the systems studied in this work. The $\Delta_\textrm{min}$ column shows the minimum difference between the experimental $H_\textrm{ads}$ value and the autoSKZCAM $H_\textrm{ads}$ value accounting for their error bars",
    end_input = '\\end{table}',
    adjustbox = 0.9,
    center = True,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Hads_Comparison.tex',
    replace_input = {
        'Hads': r'$H_\textrm{ads}$',
        'Error': r'$\epsilon$',
        'DeltaMin': r'$\Delta_\textrm{min}$'
    }
)


System,System,Expt. Hads,Expt. Error,autoSKZCAM Hads,autoSKZCAM Error,DeltaMin
0,CH$_4$ on MgO (001),-115,19,-109,16,0
1,C$_2$H$_6$ on MgO (001),-221,30,-175,20,0
2,CO on MgO (001),-176,20,-180,11,0
3,Parallel N$_2$O on MgO (001),-239,31,-259,10,0
4,C$_6$H$_6$ on MgO (001),-481,69,-430,51,0
5,Monomer H$_2$O on MgO (001),-520,121,-542,37,0
6,NH$_3$ on MgO (001),-613,65,-524,28,0
8,Chemisorbed CO$_2$ on MgO (001),-664,125,-721,95,0
9,Monolayer CH$_4$ on MgO (001),-131,19,-132,31,0
10,Monolayer C$_2$H$_6$ on MgO (001),-236,30,-203,72,0


In [31]:
fig, axs = plt.subplots(1,2,figsize=(6.69,1.5),dpi=1200, sharey=True, constrained_layout=True,gridspec_kw={'width_ratios':(1.85,1)})


# Plot monomers on MgO Hads values
xticklens = [1,1.8,2.98,3.87,4.73,5.625,6.335,6.95]
axs[0].errorbar(np.array(xticklens[:8])-0.1, [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][0] for molecule in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][1] for molecule in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],color=color_dict['blue'],markerfacecolor='none',capsize=2,fmt="o",alpha=1,label='SKZCAM')
axs[0].errorbar(np.array(xticklens[:8])+0.1, [hads_comparison_dict['MgO'][molecule]['Experiment'][0] for molecule in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],   [hads_comparison_dict['MgO'][molecule]['Experiment'][1] for molecule in ['CH4', 'C2H6','CO','N2O', 'C6H6',  'H2O', 'NH3', 'CO2']],capsize=2,fmt="o",alpha=1,color=color_dict['red'],markerfacecolor='none',label='Experiment')

# Plot monolayers on MgO Hads values
xticklens1 = [14.25,15.75]
axs[1].errorbar(np.array(xticklens1)-0.1, [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][0] for molecule in ['Monolayer CH4', 'Monolayer C2H6']],   [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][1] for molecule in ['Monolayer CH4', 'Monolayer C2H6']],color=color_dict['blue'],capsize=2,fmt="o",markerfacecolor='none', label='autoSKZCAM')
axs[1].errorbar(np.array(xticklens1)+0.1, [hads_comparison_dict['MgO'][molecule]['Experiment'][0] for molecule in ['Monolayer CH4', 'Monolayer C2H6']],   [hads_comparison_dict['MgO'][molecule]['Experiment'][1] for molecule in ['Monolayer CH4', 'Monolayer C2H6']],capsize=2,fmt="o",color=color_dict['red'],markerfacecolor='none', label='Experiment')

# Plot CO and CO2 on MgO DFT literature
# axs[0].bar([xticklens[2]-0.3, xticklens[5]-0.3], height = [89--379,-481--608] , color=color_dict['yellow'], bottom = [-379 + autoskzcam_final_hads_dict['MgO']['CO']['DFT DeltaH'][0],-608 + autoskzcam_final_hads_dict['MgO']['H2O Monomer']['DFT DeltaH'][0]], width=0.15,alpha=0.7, label = 'cWFT literature')
# axs[0].bar([xticklens[7]+0.3], height = [770--680] , color=color_dict['grey'], bottom = [-680+ autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['DFT DeltaH'][0]], width=0.15,alpha=0.7, label = 'DFT literature')
# axs[0].annotate('770', xy=(xticklens[7]+0.3, 0), xytext=(xticklens[7]+0.3, -200),fontsize=7,horizontalalignment="center",arrowprops=dict(arrowstyle="->"))


axs[0].set_xticks(xticklens)
axs[0].set_xticklabels([r'CH\textsubscript{4}', r'C\textsubscript{2}H\textsubscript{6}','CO',r'N\textsubscript{2}O', r'C\textsubscript{6}H\textsubscript{6}',  r'H\textsubscript{2}O', r'NH\textsubscript{3}', r'CO\textsubscript{2}']) 
axs[0].set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')
axs[0].set_ylim([-1000,0])
axs[1].legend(frameon=True,loc='lower left')

axs[1].set_xticks(xticklens1)
axs[1].set_xticklabels([r'CH\textsubscript{4}',r'C\textsubscript{2}H\textsubscript{6}']) #[r'H\textsubscript{2}O', 

axs[0].grid(axis = "y",ls='--', lw=.5)
axs[0].set_xlim([0.5,7.5])

axs[1].grid(axis = "y",ls='--', lw=.5)
axs[1].set_xlim([13.5,16.5])

axs[0].xaxis.tick_top()
axs[0].xaxis.set_label_position('top') 
axs[1].xaxis.tick_top()
axs[1].xaxis.set_label_position('top') 

plt.savefig('Figures/MAIN_Figure-autoSKZCAM_Expt_Hads_MgO.png')

In [32]:
fig, axs = plt.subplots(1,2,figsize=(6.69,2),dpi=1200, sharey=True, constrained_layout=True,gridspec_kw={'width_ratios':(1.55,1)})

# Plot monomers on TiO2 Hads values starting with rutile then anatase
xticklens = [0.9,2,2.9,3.95,5.6,6.32]
axs[0].errorbar( np.array(xticklens[:4])-0.1, [hads_comparison_dict['r-TiO2'][molecule]['autoSKZCAM'][0] for molecule in ['CH4','CO2','H2O','CH3OH']],   [hads_comparison_dict['r-TiO2'][molecule]['autoSKZCAM'][1] for molecule in ['CH4','CO2','H2O','CH3OH']],color=color_dict['blue'],capsize=2,fmt="o",label='SKZCAM',markerfacecolor='none')
axs[0].errorbar( np.array(xticklens[:4])+0.1, [hads_comparison_dict['r-TiO2'][molecule]['Experiment'][0] for molecule in ['CH4','CO2','H2O','CH3OH']],   [hads_comparison_dict['r-TiO2'][molecule]['Experiment'][1] for molecule in ['CH4','CO2','H2O','CH3OH']],capsize=2,fmt="o",color=color_dict['red'],label='Experiment',markerfacecolor='none')


axs[0].errorbar(np.array(xticklens[-2:])-0.1, [hads_comparison_dict['a-TiO2'][molecule]['autoSKZCAM'][0] for molecule in ['H2O','NH3']],   [hads_comparison_dict['a-TiO2'][molecule]['autoSKZCAM'][1] for molecule in ['H2O','NH3']],color=color_dict['blue'],capsize=2,fmt="o",markerfacecolor='none')
axs[0].errorbar(np.array(xticklens[-2:])+0.1, [hads_comparison_dict['a-TiO2'][molecule]['Experiment'][0] for molecule in ['H2O','NH3']],   [hads_comparison_dict['a-TiO2'][molecule]['Experiment'][1] for molecule in ['H2O','NH3']],capsize=2,fmt="o",color=color_dict['red'],markerfacecolor='none')

# axs[0].bar([xticklens[2]-0.3], height = [-954- -1270] , color=color_dict['yellow'], bottom = [-954 + autoskzcam_final_hads_dict['r-TiO2']['H2O']['DFT DeltaH'][0]], width=0.15, label = 'cWFT literature')
# axs[0].bar([xticklens[2]+0.3], height = [-230 - -930] , color=color_dict['grey'], bottom = [-840 + autoskzcam_final_hads_dict['r-TiO2']['H2O']['DFT DeltaH'][0]], width=0.15, label = 'DFT literature')


axs[0].set_xticks(xticklens)
axs[0].set_xticklabels([r'CH\textsubscript{4}',r'CO\textsubscript{2}', r'H\textsubscript{2}O', r'CH\textsubscript{3}OH', ] + [r'H\textsubscript{2}O', r'NH\textsubscript{3}'])

# Plot clusters on MgO Hads values
xticklens = [7.98,9.05,10.15]
axs[1].errorbar(np.array(xticklens[-3:])-0.1, [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][0] for molecule in ['Cluster NO','Cluster H2O','Cluster CH3OH']],   [hads_comparison_dict['MgO'][molecule]['autoSKZCAM'][1] for molecule in ['Cluster NO','Cluster H2O','Cluster CH3OH']],color=color_dict['blue'],capsize=2,fmt="o",alpha=1.0,markerfacecolor='none')
axs[1].errorbar(np.array(xticklens[-3:])+0.1, [hads_comparison_dict['MgO'][molecule]['Experiment'][0] for molecule in ['Cluster NO','Cluster H2O','Cluster CH3OH']],   [hads_comparison_dict['MgO'][molecule]['Experiment'][1] for molecule in ['Cluster NO','Cluster H2O','Cluster CH3OH']],capsize=2,fmt="o",alpha=1.0,color=color_dict['red'],markerfacecolor='none')

axs[1].set_xticks(xticklens)
axs[1].set_xticklabels(['NO',r'H\textsubscript{2}O',r'CH\textsubscript{3}OH'])
axs[1].grid(axis = "y",ls='--', lw=.5)

axs[0].set_ylim([-1400,0])
axs[0].grid(axis = "y",ls='--', lw=.5)
axs[0].set_xlim([0.5,7])
axs[0].set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')

axs[1].set_xlim([7.5,11])
axs[1].xaxis.tick_top()
axs[0].xaxis.tick_top()
axs[1].xaxis.set_label_position('top') 
axs[0].xaxis.set_label_position('top') 

plt.savefig('Figures/MAIN_Figure-autoSKZCAM_Expt_Hads_TiO2.png')

### Effect of using the conventional nu=10^13 in TPD analysis

In [33]:
# Go over the expt_hads_df and get the Hads and temperature values
expt_hads_nu13_dict = {}

for idx, row in expt_hads_df.iterrows():
    surface = row["Surface"]
    molecule = row["Molecule"]
    current_lognu = row["log(ν)"]
    if surface == 'MgO' and molecule == 'H2O':
        lognu13_correction = 0.0 # Because this is not a TPD experiment
    else:
        lognu13_correction = 2.3*(float(current_lognu.split()[-3])-13)*0.086173303*float(row['Temperature'])
        
    expt_hads_nu13_dict[idx] = {'Surface': surface, 'Molecule': molecule, 'Hads (re-analysis)': int(row['Hads']), 'Hads (log(nu)=13)': int(float(int(row['Hads'])) + lognu13_correction)}

# Keep only Surface, Molecule, Hads and Error columns in expt_hads_df
hads_nu13_comparison_df = pd.DataFrame.from_dict(expt_hads_nu13_dict, orient='index')
# Remove the 'Physisorbed CO2' row
hads_nu13_comparison_df = hads_nu13_comparison_df[hads_nu13_comparison_df['Molecule'] != 'Physisorbed CO2']

# Add the autoSKZCAM estimates to the Dataframe
total_squared_error = 0
total_squared_error_nu13 = 0
autoskzcam_ground_state_hads = []
system_label_list = []

for idx, row in hads_nu13_comparison_df.iterrows():
    surface = row['Surface']
    molecule = row['Molecule']
    
    if len(molecule.split()) == 2:
        if molecule.split()[0] == 'Cluster':
            cluster_size = 'Tetramer' if molecule.split()[1] in ['H2O','CH3OH'] else 'Dimer'
            molecule_label = molecule.split()[1] + ' ' + cluster_size
        else:
            molecule_label = molecule.split()[1] + ' ' + molecule.split()[0]
    elif surface == 'MgO' and molecule == 'H2O':
        molecule_label = 'H2O Monomer'
    elif surface == 'MgO' and molecule == 'N2O':
        molecule_label = 'N2O Parallel'
    elif surface == 'r-TiO2' and molecule == 'CO2':
        molecule_label = 'CO2 Tilted'

    else:
        molecule_label = molecule

    system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule_label]} on {convert_to_nice_labels["surface"][surface]}'

    system_label_list += [system_label]
    system_autoskzcam_hads = int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0]))
    system_autoskzcam_hads_error = int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]))
    autoskzcam_ground_state_hads += [f'{system_autoskzcam_hads} $\pm$ {system_autoskzcam_hads_error}']
    total_squared_error += (system_autoskzcam_hads-row['Hads (re-analysis)'])**2
    total_squared_error_nu13 += (system_autoskzcam_hads-row['Hads (log(nu)=13)'])**2

# Add the system labels, AutoSKZCAM Hads and Error columns to the DataFrame
hads_nu13_comparison_df['autoSKZCAM'] = autoskzcam_ground_state_hads
hads_nu13_comparison_df['System'] = system_label_list

# Rearrange the columns
hads_nu13_comparison_df = hads_nu13_comparison_df[['System','autoSKZCAM','Hads (re-analysis)','Hads (log(nu)=13)']]

# Add additional row for the RMSD
hads_nu13_comparison_df.loc['RMSD'] = ['RMSD','',(total_squared_error/len(hads_nu13_comparison_df))**0.5,(total_squared_error_nu13/len(hads_nu13_comparison_df))**0.5]

# Round to nearest integer
hads_nu13_comparison_df = hads_nu13_comparison_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)

# Rename the columns to Expt. Hads and Expt. Error
hads_nu13_comparison_df.columns = ['System','autoSKZCAM',r'Experiment (Table S1)',r'Experiment ($\nu{=}10^{13}$)']


hads_nu13_comparison_df

convert_df_to_latex_input(
    hads_nu13_comparison_df,
    start_input = '\\begin{table}',
    label = 'tab:hads_nu13_comparison',
    caption = r"Comparison of the experimental $H_\textrm{ads}$ values in meV with the autoSKZCAM $H_\textrm{ads}$ values for the systems studied in this work. The $H_\textrm{ads}$ values are compared with the re-analysed values in Table~\ref{tab:expt_hads} and the values obtained by using the conventional $\log(\nu)=13$ in temperature programmed desorption (TPD) experiments. The root mean squared deviation (RMSD) is also calculated against the autoSKZCAM $H_\textrm{ads}$ values.",
    end_input = '\\end{table}',
    adjustbox = 1,
    center = True,
    index = False,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-Hads_Nu13_Comparison.tex'
)

In [36]:
# Go over the expt_hads_df and get the Hads and temperature values
expt_hads_nu13_dict = {}

for idx, row in expt_hads_df.iterrows():
    surface = row["Surface"]
    molecule = row["Molecule"]
    current_lognu = row["log(ν)"]
    if surface == 'MgO' and molecule == 'H2O':
        lognu13_correction = 0.0 # Because this is not a TPD experiment
    else:
        lognu13_correction = 2.3*(float(current_lognu.split()[-3])-13)*0.086173303*float(row['Temperature'])
        
    expt_hads_nu13_dict[idx] = {'Surface': surface, 'Molecule': molecule, 'Hads (re-analysis)': int(row['Hads']), 'Hads (nu=13)': float(int(row['Hads'])) + lognu13_correction}

# Keep only Surface, Molecule, Hads and Error columns in expt_hads_df
hads_nu13_comparison_df = expt_hads_nu13_df[['Surface','Molecule','Hads (re-analysis)','Hads (nu=13)']]
# Remove the 'Physisorbed CO2' row
hads_nu13_comparison_df = hads_nu13_comparison_df[hads_nu13_comparison_df['Molecule'] != 'Physisorbed CO2']
# Rename the columns to Expt. Hads and Expt. Error
# hads_nu13_comparison_df.columns = ['Surface','Molecule','Experiment',r'Experiment ($nu=10^{13}$)']

# Add the AutoSKZCAM Hads and Error columns to the DataFrame
# Loop over the DataFrame and add the AutoSKZCAM Hads and Error values
autoskzcam_ground_state_hads = []
autoskzcam_ground_state_hads_error = []
system_label_list = []
hads_delta_min_list = []

for idx, row in hads_nu13_comparison_df.iterrows():
    surface = row['Surface']
    molecule = row['Molecule']

    system_expt_hads = int(row['Hads (re-analysis)'])
    system_expt_nu13_hads = int(row['Hads (nu=13)'])
    
    if len(molecule.split()) == 2:
        if molecule.split()[0] == 'Cluster':
            cluster_size = 'Tetramer' if molecule.split()[1] in ['H2O','CH3OH'] else 'Dimer'
            molecule_label = molecule.split()[1] + ' ' + cluster_size
        else:
            molecule_label = molecule.split()[1] + ' ' + molecule.split()[0]
    elif surface == 'MgO' and molecule == 'H2O':
        molecule_label = 'H2O Monomer'
    elif surface == 'MgO' and molecule == 'N2O':
        molecule_label = 'N2O Parallel'
    elif surface == 'r-TiO2' and molecule == 'CO2':
        molecule_label = 'CO2 Tilted'

    else:
        molecule_label = molecule

    system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule_label]} on {convert_to_nice_labels["surface"][surface]}'

    system_label_list += [system_label]
    autoskzcam_ground_state_hads += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0]))]
    autoskzcam_ground_state_hads_error += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]))]
    hads_nu13_comparison_dict[surface][molecule]['autoSKZCAM'] = [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0])), int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]) )]

    system_autoskzcam_hads_lower_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] - autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    system_autoskzcam_hads_upper_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] + autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    # Find the minimum difference between the experimental Hads and the AutoSKZCAM Hads errors
    if max(system_expt_hads_lower_bound, system_autoskzcam_hads_lower_bound) <= min(system_expt_hads_upper_bound, system_autoskzcam_hads_upper_bound):
        delta_min = 0
    else:
        delta_min = min(abs(system_expt_hads_upper_bound - system_autoskzcam_hads_lower_bound), abs(system_expt_hads_lower_bound - system_autoskzcam_hads_upper_bound))


    hads_delta_min_list += [int(round(delta_min))]


# Add the system labels, AutoSKZCAM Hads and Error columns to the DataFrame
hads_nu13_comparison_df['autoSKZCAM Hads'] = autoskzcam_ground_state_hads
hads_nu13_comparison_df['autoSKZCAM Error'] = autoskzcam_ground_state_hads_error
hads_nu13_comparison_df['System'] = system_label_list

# Rearrange the columns
hads_nu13_comparison_df = hads_nu13_comparison_df[['System','Expt. Hads','Expt. Error','autoSKZCAM Hads','autoSKZCAM Error','DeltaMin']]

# Round to nearest integer
hads_nu13_comparison_df = hads_nu13_comparison_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)
hads_nu13_comparison_df.columns.name = 'System' 


hads_nu13_comparison_dict['MgO']['CO2'] = hads_nu13_comparison_dict['MgO']['Chemisorbed CO2']
del hads_nu13_comparison_dict['MgO']['Chemisorbed CO2']

display(hads_nu13_comparison_df)

NameError: name 'expt_hads_nu13_df' is not defined

In [ ]:
total_sum = 0
total_sq_sum = 0
total_percentage = 0
for idx, row in hads_nu13_comparison_df.iterrows():
    autoskzcam_hads = float(row['autoSKZCAM Hads'])
    expt_hads = float(row['Expt. Hads'])
    print(expt_hads, autoskzcam_hads, np.abs(expt_hads - autoskzcam_hads))
    total_sum += np.abs(expt_hads - autoskzcam_hads)
    total_sq_sum += (expt_hads - autoskzcam_hads)**2
    total_percentage += np.abs((expt_hads - autoskzcam_hads)/autoskzcam_hads)*100
    # total_sum += np.abs(float(row['Expt. Hads']) - float(row['autoSKZCAM Hads']))
    # total_sq_sum += (float(row['Expt. Hads']) - float(row['autoSKZCAM Hads']))**2
    # print(np.abs(float(row['Expt. Hads']) - float(row['autoSKZCAM Hads'])/float(row['autoSKZCAM Hads'])*100))
    # total_percentage += np.abs(float(row['Expt. Hads']) - float(row['autoSKZCAM Hads'])/float(row['autoSKZCAM Hads']))*100
print(total_sum/19, np.sqrt(total_sq_sum/19),total_percentage/19)


-114.0 -109.0 5.0
-193.0 -175.0 18.0
-166.0 -180.0 14.0
-224.0 -259.0 35.0
-414.0 -430.0 16.0
-520.0 -542.0 22.0
-581.0 -524.0 57.0
-618.0 -721.0 103.0
-130.0 -132.0 2.0
-208.0 -203.0 5.0
-216.0 -232.0 16.0
-624.0 -632.0 8.0
-760.0 -741.0 19.0
-217.0 -242.0 25.0
-472.0 -443.0 29.0
-815.0 -1011.0 196.0
-1014.0 -1285.0 271.0
-705.0 -909.0 204.0
-1156.0 -1090.0 66.0
58.473684210526315 96.34723386628289 8.929619788572571


In [ ]:
total_sum = 0
total_sq_sum = 0
total_percentage = 0
for idx, row in hads_comparison_df.iterrows():
    autoskzcam_hads = float(row['autoSKZCAM Hads'])
    expt_hads = float(row['Expt. Hads'])
    print(expt_hads, autoskzcam_hads, np.abs(expt_hads - autoskzcam_hads))
    total_sum += np.abs(expt_hads - autoskzcam_hads)
    total_sq_sum += (expt_hads - autoskzcam_hads)**2
    total_percentage += np.abs((expt_hads - autoskzcam_hads)/autoskzcam_hads)*100
print(total_sum/19, np.sqrt(total_sq_sum/19),total_percentage/19)


-115.0 -109.0 6.0
-221.0 -175.0 46.0
-176.0 -180.0 4.0
-239.0 -259.0 20.0
-481.0 -430.0 51.0
-520.0 -542.0 22.0
-613.0 -524.0 89.0
-664.0 -721.0 57.0
-131.0 -132.0 1.0
-236.0 -203.0 33.0
-232.0 -232.0 0.0
-694.0 -632.0 62.0
-890.0 -741.0 149.0
-249.0 -242.0 7.0
-493.0 -443.0 50.0
-917.0 -1011.0 94.0
-1197.0 -1285.0 88.0
-786.0 -909.0 123.0
-1180.0 -1090.0 90.0
52.21052631578947 67.40138995911771 9.557354224857965


In [ ]:
hads_nu13_comparison_dict

{'MgO': {'CH4': {'Experiment': [-114, 19], 'autoSKZCAM': [-109, 16]},
  'C2H6': {'Experiment': [-192, 30], 'autoSKZCAM': [-175, 20]},
  'CO': {'Experiment': [-166, 20], 'autoSKZCAM': [-180, 11]},
  'N2O': {'Experiment': [-223, 31], 'autoSKZCAM': [-259, 10]},
  'C6H6': {'Experiment': [-413, 69], 'autoSKZCAM': [-430, 51]},
  'H2O': {'Experiment': [-520, 121], 'autoSKZCAM': [-542, 37]},
  'NH3': {'Experiment': [-581, 65], 'autoSKZCAM': [-524, 28]},
  'Monolayer CH4': {'Experiment': [-130, 19], 'autoSKZCAM': [-132, 31]},
  'Monolayer C2H6': {'Experiment': [-207, 30], 'autoSKZCAM': [-203, 72]},
  'Cluster NO': {'Experiment': [-216, 30], 'autoSKZCAM': [-232, 30]},
  'Cluster H2O': {'Experiment': [-624, 77], 'autoSKZCAM': [-632, 20]},
  'Cluster CH3OH': {'Experiment': [-759, 100], 'autoSKZCAM': [-741, 28]},
  'CO2': {'Experiment': [-618, 125], 'autoSKZCAM': [-721, 95]}},
 'r-TiO2': {'CH4': {'Experiment': [-216, 34], 'autoSKZCAM': [-242, 22]},
  'CO2': {'Experiment': [-471, 58], 'autoSKZCAM': 

In [ ]:
total_percentage/19

25136.842105263157

In [ ]:
total_sum = 0
total_sq_sum = 0
for system in hads_nu13_comparison_dict['MgO']:
    total_sum += np.abs(hads_nu13_comparison_dict['MgO'][system]['Experiment'][0] - hads_nu13_comparison_dict['MgO'][system]['autoSKZCAM'][0])
    total_sq_sum += (hads_nu13_comparison_dict['MgO'][system]['Experiment'][0] - hads_nu13_comparison_dict['MgO'][system]['autoSKZCAM'][0])**2
print(total_sum/19, np.sqrt(total_sq_sum/19))

In [ ]:
total_sum = 0
total_sq_sum = 0
for system in hads_comparison_dict['MgO']:
    total_sum += np.abs(hads_comparison_dict['MgO'][system]['Experiment'][0] - hads_comparison_dict['MgO'][system]['autoSKZCAM'][0])
    total_sq_sum += (hads_comparison_dict['MgO'][system]['Experiment'][0] - hads_comparison_dict['MgO'][system]['autoSKZCAM'][0])**2
print(total_sum/19, np.sqrt(total_sq_sum/19))

28.42105263157895 48.099677206488835


In [ ]:
total_sum

319

In [ ]:
expt_hads_nu13_dict

{0: {'Surface': 'MgO',
  'Molecule': 'CH4',
  'Hads': -114.06846659457,
  'Error': 19.0},
 1: {'Surface': 'MgO',
  'Molecule': 'C2H6',
  'Hads': -192.75669994174999,
  'Error': 30.0},
 2: {'Surface': 'MgO',
  'Molecule': 'CO',
  'Hads': -166.32790847127998,
  'Error': 20.0},
 3: {'Surface': 'MgO',
  'Molecule': 'N2O',
  'Hads': -223.7387080387,
  'Error': 31.0},
 4: {'Surface': 'MgO',
  'Molecule': 'C6H6',
  'Hads': -413.57283733462,
  'Error': 69.0},
 5: {'Surface': 'MgO', 'Molecule': 'H2O', 'Hads': -520.0, 'Error': 121.0},
 6: {'Surface': 'MgO',
  'Molecule': 'NH3',
  'Hads': -581.288224496,
  'Error': 65.0},
 7: {'Surface': 'MgO',
  'Molecule': 'Physisorbed CO2',
  'Hads': -407.216168372,
  'Error': 49.0},
 8: {'Surface': 'MgO',
  'Molecule': 'Chemisorbed CO2',
  'Hads': -618.414322713,
  'Error': 125.0},
 9: {'Surface': 'MgO',
  'Molecule': 'Monolayer CH4',
  'Hads': -130.06846659457,
  'Error': 19.0},
 10: {'Surface': 'MgO',
  'Molecule': 'Monolayer C2H6',
  'Hads': -207.756699941

In [ ]:
expt_hads_df

,Surface,Molecule,Temperature,log(ν),Hads,Error,Details
0,MgO,CH4,47,13.1 ± 2.0,-115,19,Dilute limit $E_\textrm{d$ estimate by Tait \e...
1,MgO,C2H6,75,14.9 ± 2.0,-221,30,Dilute limit $E_\textrm{d$ estimate by Tait \e...
2,MgO,CO,61,13.8 ± 1.6,-176,20,Average of the $H_\textrm{ads$ re-analysis by ...
3,MgO,N2O,77,13.0 -> 14.0 ± 2.0,-239,31,$E_\textrm{d$ measured by Lian \etal{~\cite{li...
4,MgO,C6H6,162,15.1 ± 1.6,-481,69,Average taken of $H_\textrm{ads$ re-analyzed b...
5,MgO,H2O,203,-,-520,121,$H_\textrm{ads$ from Ferry \etal{~\cite{alessi...
6,MgO,NH3,160,13.0 -> 14.0 ± 2.0,-613,65,$E_\textrm{d$ measurement from Arthur \etal{~\...
7,MgO,Physisorbed CO2,120,13.0 -> 14.0 ± 2.0,-431,49,$E_\textrm{d$ measurement from Meixner \etal{~...
8,MgO,Chemisorbed CO2,230,13.0 -> 14.0 ± 2.0,-664,125,$E_\textrm{d$ measurement from Chakradhar and ...
9,MgO,Monolayer CH4,47,13.1 ± 2.0,-131,19,Monolayer $E_\textrm{d$ estimate by Tait \etal...


# Previous DFT literature

In [37]:
with open('Data/Miscellaneous/Computational_Eads_Literature.csv', 'r') as f:
    dft_literature_hads = f.read().splitlines()

computational_eads_literature = {}
no_configuration_eads = {}
co2_configuration_eads = {}
for line in dft_literature_hads[1:]:
    if r'//' in line:
        system = re.sub(r'(\d+)', r'$_\1$', line.strip(',').split('//')[0]) + ' on ' + convert_to_nice_labels['surface'][line.strip(',').split('//')[1]]
        computational_eads_literature[system] = {'DFT': [], 'DFT References': [], 'WFT': [], 'WFT References': []}
    elif 'DFT' in line:
        computational_eads_literature[system]['DFT'].append(line.split(',')[2])
        computational_eads_literature[system]['DFT References'].append(line.split(',')[2] + r"~\cite{" + line.split(',')[3].replace(' + ',',') + r"}")
        if 'NO' in system:
            configuration = line.split(',')[4]
            if configuration not in no_configuration_eads:
                no_configuration_eads[configuration] = {'DFT': [], 'DFT References': [], 'WFT': [], 'WFT References': []}
            no_configuration_eads[configuration]['DFT'].append(computational_eads_literature[system]['DFT'][-1])
            no_configuration_eads[configuration]['DFT References'].append(computational_eads_literature[system]['DFT References'][-1])
        elif system == r'CO$_2$ on MgO (001)':
            configuration = line.split(',')[4]
            if configuration not in co2_configuration_eads:
                co2_configuration_eads[configuration] = {'DFT': [], 'DFT References': [], 'WFT': [], 'WFT References': []}
            co2_configuration_eads[configuration]['DFT'].append(computational_eads_literature[system]['DFT'][-1])
            co2_configuration_eads[configuration]['DFT References'].append(computational_eads_literature[system]['DFT References'][-1])
    elif 'WFT' in line:
        computational_eads_literature[system]['WFT'].append(f"{line.split(',')[2]} [{line.split(',')[1].split()[-1].replace('delta','Δ')}]")
        computational_eads_literature[system]['WFT References'].append(f"{line.split(',')[2]} [{line.split(',')[1].split()[-1].replace('delta','Δ')}]" + r"~\cite{" + line.split(',')[3].replace(' + ',',') + r"}")
        if 'NO' in system:
            configuration = line.split(',')[4]
            no_configuration_eads[configuration]['WFT'].append(computational_eads_literature[system]['WFT'][-1])
            no_configuration_eads[configuration]['WFT References'].append(computational_eads_literature[system]['WFT References'][-1])
        elif system == r'CO$_2$ on MgO (001)':
            configuration = line.split(',')[4]
            if configuration not in co2_configuration_eads:
                co2_configuration_eads[configuration] = {'DFT': [], 'DFT References': [], 'WFT': [], 'WFT References': []}
            co2_configuration_eads[configuration]['WFT'].append(computational_eads_literature[system]['WFT'][-1])
            co2_configuration_eads[configuration]['WFT References'].append(computational_eads_literature[system]['WFT References'][-1])

computational_eads_literature['NO on MgO (001)'] = {method: [configuration + ': ' + ', '.join(no_configuration_eads[configuration][method]) for configuration in no_configuration_eads if len(no_configuration_eads[configuration][method]) > 0] for method in ['DFT','WFT','DFT References','WFT References']}

computational_eads_literature[r'CO$_2$ on MgO (001)'] = {method: [configuration + ': ' + ', '.join(co2_configuration_eads[configuration][method]) for configuration in co2_configuration_eads if len(co2_configuration_eads[configuration][method]) > 0] for method in ['DFT','WFT','DFT References','WFT References']}

# Convert the dictionary into a DataFrame
computational_eads_literature_df = pd.DataFrame.from_dict({system: {'DFT': ', '.join(computational_eads_literature[system]['DFT']), 'WFT': ', '.join(computational_eads_literature[system]['WFT'])} for system in computational_eads_literature}, orient='index')

with pd.option_context('display.max_colwidth', None):
    display(computational_eads_literature_df)

computational_eads_literature_df = pd.DataFrame.from_dict({system: {'DFT': ', '.join(computational_eads_literature[system]['DFT References']), 'WFT': ', '.join(computational_eads_literature[system]['WFT References'])} for system in computational_eads_literature}, orient='index')

computational_eads_literature_df.columns.name = 'System'

# Convert to latex table
convert_df_to_latex_input(
df = computational_eads_literature_df,
start_input = '\\begin{table}\n',
end_input = '\n\\end{table}',
label = "tab:comp_lit",
center = True,
caption = r"Previous density functional theory (DFT) and wave-function theory (DFT) computational $E_\textrm{ads}$ literature for the systems studied within this work.",
replace_input = {
    "+-": r"$\pm$",
    r"Δ": r"$\Delta$",
},
column_format = "lp{7cm}p{7cm}",
filename = 'Tables/SI_Table-Computational_Literature.tex')

,DFT,WFT
CH$_4$ on MgO (001),"-50, 2, 17, -300, -120, -152",-134 [MP2+ΔCC]
C$_2$H$_6$ on MgO (001),"-127, -154",-196 [MP2+ΔCC]
CO on MgO (001),"-9 to -282, -175 to -408","-207 [LNO-CCSD(T)], -199 [LNO-CCSD(T)], -230 [MP2+ΔCC], 70 [CCSD(T)], -398 [CCSD], -72 [RPA@PBE], -310 [RPA@PBE+rSE]"
N$_2$O on MgO (001),"-258, >0, -137",
C$_6$H$_6$ on MgO (001),-20,
Monomer H$_2$O on MgO (001),"-342, -500, -340, -422 to -667","-574 [ΔCC], -480 [DMC], -608 [RPA@PBE+rSE], -492 [RPA@PBE]"
NH$_3$ on MgO (001),"-668, -867",-608 [PCT]
CO$_2$ on MgO (001),"Chemisorbed: -1000, 135, -680, -640, -380, Physisorbed: -320, -90, -126, -120","Chemisorbed: 870 [MP2], Physisorbed: -340 [MP2]"
NO on MgO (001),"Hollow: -312, Bent-Mg: -520, Upright-Mg: -260, -297, Bent-O: -464, -312, Dimer: -30, -40, Bent-Bridge: -150","Bent-Mg: 29 [RPA@PBE], -360 [RPA@PBE+rSE]"
Cluster CH$_3$OH on MgO (001),"-718, -466, -640, -964, -508",-598 [MP2]


### *E*<sub>int</sub> comparison between autoSKZCAM and DFT & RPA

In [38]:
dft_xc_eint_convert = {
    '01_PBE-D30': 'PBE-D3',
    '02_PBE-MBDFI': 'PBE-MBD/FI',
    '03_rev-vdW-DF2': 'rev-vdW-DF2',
    '04_SCAN-rVV10': 'SCAN-rVV10',
    '05_R2SCAN-D4': r'r$^2$SCAN-D4',
    '06_PBE0-TSHI': 'PBE0-TS/HI',
    '07_HSE06-D4': 'HSE06-D4',
    '08_RPA': 'RPA',
    '09_RPA-rSE': 'RPA+rSE'

}

systems_studied = {
    'MgO': ['CH4','C2H6','N2O_1','N2O_2','CO','CO2_2','CO2_3','H2O','NH3'],
    'TiO2_rutile': ['CH4','CH3OH','CO2_1','CO2_2','H2O'],
    'TiO2_anatase': ['H2O','NH3']
}

dft_xc_eint_dict = {}
dft_xc_eint_error_dict = {}

for surface in ['MgO','r-TiO2','a-TiO2']:
    if surface == 'MgO':
        molecule_list = ['CH4', 'C2H6', 'CO', 'CO2 Physisorbed', 'H2O Monomer', 'N2O Parallel', 'NH3']
    elif surface == 'r-TiO2':
        molecule_list = ['CH4', 'CO2 Tilted', 'H2O', 'CH3OH']
    elif surface == 'a-TiO2':
        molecule_list = ['H2O', 'NH3']
    
    for molecule in molecule_list:
        system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule]} on {convert_to_nice_labels["surface"][surface]}'
        molecule_surface_label = f'{molecule} on {surface}'
        molecule_label = molecule.replace(' ', '_')
        dft_xc_eint_dict[system_label] = {}
        dft_xc_eint_error_dict[molecule_surface_label] = {}
        dft_xc_eint_dict[system_label]['SKZCAM'] = skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]

        for xc_func in reversed(dft_xc_eint_convert):
            if xc_func == '07_HSE06-D4':
                xc_eint_gga_normal = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_GGA_normal')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='dftd4',vasp_outcar_label='OUTCAR_GGA_normal')*Hartree*1000
                xc_eint_gga_tight = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_GGA_tight')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='dftd4',vasp_outcar_label='OUTCAR_GGA_tight')*Hartree*1000
                xc_eint_hybrid_normal = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='dftd4',vasp_outcar_label='OUTCAR')*Hartree*1000

                xc_eint = xc_eint_hybrid_normal + (xc_eint_gga_tight - xc_eint_gga_normal)
                dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]
            elif xc_func == '06_PBE0-TSHI':
                xc_eint_gga_normal = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_GGA_normal')*1000
                xc_eint_gga_tight = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_GGA_tight')*1000
                xc_eint_hybrid_normal = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR')*1000
                xc_eint = xc_eint_hybrid_normal + (xc_eint_gga_tight - xc_eint_gga_normal)
                dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]
            elif xc_func == '05_R2SCAN-D4':
                xc_eint = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='dftd4',vasp_outcar_label='OUTCAR')*Hartree*1000

                dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]
            elif xc_func == '08_RPA':
                if surface == 'MgO':
                    xc_eint = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_EXX')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp_rpa',vasp_outcar_label='OUTCAR_RPA')*1000
                    dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                    dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]
            elif xc_func == '09_RPA-rSE':
                if surface == 'MgO':
                    xc_eint = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp',vasp_outcar_label='OUTCAR_rSE')*1000 + calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp_rpa',vasp_outcar_label='OUTCAR_RPA')*1000
                    dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                    dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]
            else:
                xc_eint = calculate_eint(f'Data/Miscellaneous/DFT_Comparison/{surface}/{molecule_label}/{xc_func}', code='vasp')*1000
                dft_xc_eint_dict[system_label][dft_xc_eint_convert[xc_func]] = xc_eint
                dft_xc_eint_error_dict[molecule_surface_label][dft_xc_eint_convert[xc_func]] = xc_eint - skzcam_final_eint_dict[surface][molecule]['Final CCSD(T)'][0]

for surface in ['MgO','TiO2']:
    if surface == 'MgO':
        dft_xc_eint_error_dict['MAD MgO'] = {xc_func: np.mean(np.abs([dft_xc_eint_error_dict[system][xc_func] for system in dft_xc_eint_error_dict if 'MgO' in system])) for xc_func in dft_xc_eint_error_dict['CH4 on MgO']}
    elif surface == 'TiO2':
        dft_xc_eint_error_dict['MAD TiO2'] = {xc_func: np.mean(np.abs([dft_xc_eint_error_dict[system][xc_func] for system in dft_xc_eint_error_dict if 'TiO2' in system])) for xc_func in dft_xc_eint_error_dict['CH4 on r-TiO2']}

dft_xc_eint_error_dict['Total MAD'] = {xc_func: np.mean(np.abs([dft_xc_eint_error_dict[system][xc_func] for system in dft_xc_eint_error_dict])) for xc_func in dft_xc_eint_error_dict['CH4 on r-TiO2']}

In [39]:
# Convert the dictionary to a DataFrame
dft_xc_eint_error_dict_df = pd.DataFrame.from_dict({dft_xc: dft_xc_eint_error_dict[dft_xc] for dft_xc in dft_xc_eint_error_dict if 'MAD' not in dft_xc})
dft_xc_mad_dict_df = pd.DataFrame.from_dict({dft_xc: dft_xc_eint_error_dict[dft_xc] for dft_xc in dft_xc_eint_error_dict if 'MAD' in dft_xc})

# Replace the NaN values with 0
dft_xc_eint_error_dict_df.fillna(0, inplace=True)
dft_xc_mad_dict_df.fillna(0, inplace=True)

# Convert into a 2D array for plotting
system_eint_error_2d_array = np.zeros((dft_xc_eint_error_dict_df.values.shape[0], dft_xc_eint_error_dict_df.values.shape[1]),dtype=int)
xc_func_mad_error_2d_array = np.zeros((dft_xc_mad_dict_df.values.shape[0], dft_xc_mad_dict_df.values.shape[1]),dtype=int)

row_labels = []
column_labels = []
for row_idx in range(dft_xc_eint_error_dict_df.values.shape[0]):
    row_labels += [list(reversed(dft_xc_eint_convert.values()))[row_idx]]
    system_eint_error_2d_array[row_idx] = dft_xc_eint_error_dict_df.values[row_idx]
    xc_func_mad_error_2d_array[row_idx] = dft_xc_mad_dict_df.values[row_idx]     

for column_idx in range(dft_xc_eint_error_dict_df.values.shape[1]):
    column_name = dft_xc_eint_error_dict_df.columns[column_idx]
    if 'MgO' in column_name:
        column_labels += [re.sub(r'(\d+)', r'$_\1$', column_name.split()[0])]
    elif 'r-TiO2' in column_name:
        column_labels += ['r-' + re.sub(r'(\d+)', r'$_\1$', column_name.split()[0])]
    elif 'a-TiO2' in column_name:
        column_labels += ['a-' + re.sub(r'(\d+)', r'$_\1$', column_name.split()[0])]


# Dataframe for dft_xc_eint_error_dict
dft_xc_eint_dict_df = pd.DataFrame(dft_xc_eint_dict)
# Round to nearest integer only if it is a float
dft_xc_eint_dict_df = dft_xc_eint_dict_df.map(
    lambda x: f'{int(round(x))}' if isinstance(x, float) and not np.isnan(x) else x
)

display(dft_xc_eint_dict_df)

# Write to latex table
# Write the DataFrame to a latex input
convert_df_to_latex_input(
    dft_xc_eint_dict_df,
    start_input = r'\begin{table}',
    label = 'tab:dft_xc_compare_eint',
    caption = r'Comparison of $E_\textrm{int}$ from a set of DFT models against SKZCAM estimates.',
    end_input = r'\end{table}',
    replace_input = {
        'NaN': '-'
    },
    adjustbox = 1,
    df_latex_skip = 0,
    filename = 'Tables/SI_Table-DFT_XC_Compare_Eint.tex',
    rotate_column_header = True
)

,CH$_4$ on MgO (001),C$_2$H$_6$ on MgO (001),CO on MgO (001),Physisorbed CO$_2$ on MgO (001),Monomer H$_2$O on MgO (001),Parallel N$_2$O on MgO (001),NH$_3$ on MgO (001),CH$_4$ on TiO$_2$ rutile (110),Tilted CO$_2$ on TiO$_2$ rutile (110),H$_2$O on TiO$_2$ rutile (110),CH$_3$OH on TiO$_2$ rutile (110),H$_2$O on TiO$_2$ anatase (101),NH$_3$ on TiO$_2$ anatase (101)
SKZCAM,-122,-175,-207,-308,-703,-256,-657,-269,-493,-1310,-1634,-1208,-1377
RPA+rSE,-141,-200,-294,-328,-689,-269,-698,NaN,NaN,NaN,NaN,NaN,NaN
RPA,-96,-137,-98,-236,-614,-204,-630,NaN,NaN,NaN,NaN,NaN,NaN
HSE06-D4,-162,-233,-252,-319,-728,-245,-692,-295,-496,-1397,-1718,-1252,-1528
PBE0-TS/HI,-169,-262,-245,-289,-719,-231,-686,-333,-489,-1417,-1784,-1254,-1528
r$^2$SCAN-D4,-173,-244,-296,-380,-784,-294,-734,-306,-528,-1429,-1743,-1303,-1527
SCAN-rVV10,-182,-257,-323,-427,-823,-329,-761,-323,-573,-1492,-1821,-1363,-1576
rev-vdW-DF2,-144,-213,-272,-299,-672,-246,-665,-273,-462,-1286,-1607,-1163,-1402
PBE-MBD/FI,-121,-192,-325,-319,-689,-258,-687,-291,-472,-1289,-1608,-1183,-1414
PBE-D3,-257,-375,-320,-334,-770,-314,-764,-357,-439,-1280,-1611,-1176,-1441


In [40]:
import matplotlib.patheffects as path_effects


fig, axs = plt.subplots(1,2,figsize=(6.6,4.3),dpi=1200,tight_layout=True,width_ratios=[1,0.3],sharey=True)
plt.set_cmap('bwr')


xc_nice_list = list(reversed(dft_xc_eint_convert.values()))

xc_compare_system_nice_list =  ['CH4', r'C$_2$H$_6$', 'CO', r'CO$_2$', r'H$_2$O',r'N$_2$O', r'NH$_3$',r'r-CH$_4',r'r-CH$_3$OH',r'r-CO$_2$',r'r-H$_2$O',r'a-H$_2$O',r'a-NH$_3$'] #,r'MAD',r'\textbf{Total MAD}'] # + ['CH4']*(len(xc_compare_system_list_small))


im = axs[0].imshow(system_eint_error_2d_array, vmin=-200,vmax=200, aspect='auto')
axs[0].set_xticks(np.arange(len(column_labels)), labels=column_labels,fontsize=8)
axs[0].set_yticks(np.arange(len(row_labels)), labels=row_labels,fontsize=8)

for row_idx, xc_func in enumerate(row_labels):
    for column_idx, quantity in enumerate(column_labels):
        if 'RPA' in xc_func and column_idx > 6:
            continue
        else:
            if abs(system_eint_error_2d_array[row_idx, column_idx]) > 100:
                text = axs[0].text(column_idx, row_idx, system_eint_error_2d_array[row_idx, column_idx],
                        ha="center", va="center", color="white",fontsize=9)
            else:
                text = axs[0].text(column_idx, row_idx, system_eint_error_2d_array[row_idx, column_idx],
                        ha="center", va="center", color="k",fontsize=9)
# Assuming 'axs' and 'im' are already defined
divider = make_axes_locatable(axs[0])

# Add a temporary axes object for the colorbar, with specified inch width
cax1 = divider.append_axes("top", size="8%", pad=0.05)  # Relative size for vertical thickness
colorbar_width_inches = 2  # Set desired width of the colorbar in inches

# Create the colorbar
cbar = plt.colorbar(im, cax=cax1, orientation="horizontal")
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.set_xlabel(r'\textbf{Deviation from autoSKZCAM Interaction Energy [meV]}', fontsize=9)

# cbar.ax.set_xlabel(r'$E_\textrm{int}[\textrm{Method}]$ - $E_\textrm{int}[\textrm{SKZCAM}]$ [meV]', fontsize=8)
cbar.ax.tick_params(labelsize=8)
# Left-align each xtick label
for label in cbar.ax.get_xticklabels():
    label.set_ha('right')


# Adding vertical lines to separate columns
for i in range(len(column_labels) - 1):
    if i > 6:
        axs[0].plot([i+0.5,i+0.5],[1.5,8.5], color='white', linewidth=0.5)
    else:
        axs[0].axvline(x=i + 0.5, color='white', linewidth=0.5)

# Adding horizontal lines to separate rows
for i in range(len(row_labels) - 1):
    if i < 2:
        axs[0].plot([-0.5,6.5],[i+0.5,i+0.5], color='white', linewidth=0.5)
    else:
        axs[0].axhline(y=i + 0.5, color='white', linewidth=0.5)

axs[0].axvline(x=6.5, color='black', linewidth=0.75)
# axs.plot([14.5,14.5],[1.5,8.5], color='black', linewidth=0.75)
axs[0].plot([7.5,12.5],[1.5,1.5], color='black', linewidth=0.75)

# Hatched lines for RPA region in TiO2
axs[0].add_patch(Rectangle((6.5, -0.5), 6, 2, fill=False, hatch='////'))
plt.setp(axs[0].get_xticklabels(), rotation=90)


divider1 = make_axes_locatable(axs[1])

# Define custom colormap that transitions from invisible to black
colors = [(1, 1, 1, 0), (0, 0, 0, 0.7)]
n_bins = 100
cmap_custom = LinearSegmentedColormap.from_list('invisible_to_black', colors, N=n_bins)

# Second color bar for custom colormap at the top
cax2 = divider1.append_axes("top", size="8%", pad=0.05)
im_custom = axs[1].imshow(xc_func_mad_error_2d_array, cmap=cmap_custom, alpha=1, vmin=0, vmax=150)
cbar_custom = plt.colorbar(im_custom, cax=cax2, orientation="horizontal")
cbar_custom.ax.xaxis.set_label_position('top')
cbar_custom.ax.xaxis.set_ticks_position('top')
cbar_custom.ax.set_xlabel(r'\textbf{MAD [meV]}', fontsize=9)
cbar_custom.ax.tick_params(labelsize=8)
# Set custom ticks including 43
tick_positions = [0, 43, 100]  # Customize this list as needed
cbar_custom.set_ticks(tick_positions)
# Left-align each xtick label
for label in cbar_custom.ax.get_xticklabels():
    label.set_ha('left')




axs[1].set_xticks(np.arange(3), labels=[r'\textbf{MgO}',r'\textbf{TiO\textsubscript{2}}',r'\textbf{Total}'],fontsize=8)
axs[1].set_yticks(np.arange(len(row_labels)), labels=row_labels,fontsize=8.5)

for row_idx, xc_func in enumerate(row_labels):
    for column_idx, quantity in enumerate(['MgO','TiO2','Total']):
        if 'RPA' in xc_func and column_idx > 0:
            continue
        else:
            if xc_func_mad_error_2d_array[row_idx, column_idx] > 100:
                text = axs[1].text(column_idx, row_idx, rf'{xc_func_mad_error_2d_array[row_idx, column_idx]}',
                        ha="center", va="center", color="white",fontsize=9)
            else:
                text = axs[1].text(column_idx, row_idx, rf'{xc_func_mad_error_2d_array[row_idx, column_idx]}',
                        ha="center", va="center", color="k",fontsize=9)

# Adding vertical lines to separate columns
for i in range(2):
    axs[1].plot([i+0.5,i+0.5],[1.5,8.5], color='white', linewidth=0.5)


# Adding horizontal lines to separate rows
for i in range(len(row_labels) - 1):
    if i < 2:
        axs[1].plot([-0.5,0.5],[i+0.5,i+0.5], color='white', linewidth=0.5)
    else:
        axs[1].axhline(y=i + 0.5, color='white', linewidth=0.5)

axs[1].plot([0.5,1.5],[1.5,1.5], color='black', linewidth=0.75)
axs[1].plot([0.5,0.5],[-0.5,1.5], color='black', linewidth=0.75)

# Hatched lines for RPA region in TiO2
axs[1].add_patch(Rectangle((0.5, -0.5), 2, 2, fill=False, hatch='////'))
plt.setp(axs[1].get_xticklabels(), rotation=90)
plt.subplots_adjust(wspace=-0.2)  # Adjust horizontal space between subplots


plt.savefig('Figures/MAIN_Figure-DFT_Benchmark.png')

In [ ]:
skzcam_cost_dict = {surface: {molecule: {'Molecule': 0, 'Surface': 0, 'Molecule-Surface': 0}} for molecule, surface in [('CO','MgO'),('H2O','r-TiO2'),('H2O','a-TiO2')]}

for molecule, surface in [('CO','MgO'),('H2O','r-TiO2'),('H2O','a-TiO2')]:
    for cluster_number in ['1','2','3','4','5','6','7']:

        # Looping over possibility of small metal cation ECP core and large metal cation ECP core
        for basis_type in ['aV','awCV']:
            for structure in ['Molecule-Surface','Molecule','Surface']:
                filedir = f'Data/05b-Eint_SKZCAM/{surface}/{molecule}/Cluster_{cluster_number}'
                # First obtain the MP2 Eint values 
                outputname = f'MP2_{basis_type}'


                try:
                    for basis in ['DZ','TZ','QZ']:
                        walltime = get_orca_walltime(f'{filedir}/{structure}/{outputname}{basis}.orca.out')
                        # print(molecule,surface,cluster_number,outputname,basis,structure,walltime)
                        if surface == 'MgO':
                            skzcam_cost_dict[surface][molecule][structure] += walltime*16/3600
                        else:
                            skzcam_cost_dict[surface][molecule][structure] += walltime*4/3600

                except:
                    pass
                    


                # Then obtain the Eint values with CCSD(T)
                outputname = f'LNOCCSDT_{basis_type}'
                try:
                    for basis in ['DZ','TZ','QZ']:
                        walltime = get_mrcc_walltime(f'{filedir}/{structure}/{outputname}{basis}.mrcc.out')
                        print(molecule,surface,cluster_number,outputname,basis,structure,walltime)

                        skzcam_cost_dict[surface][molecule][structure] += walltime*18/3600
                except:
                    pass


CO MgO 1 LNOCCSDT_awCV DZ Molecule-Surface 219.0
CO MgO 1 LNOCCSDT_awCV TZ Molecule-Surface 592.0
CO MgO 1 LNOCCSDT_awCV DZ Molecule 18.0
CO MgO 1 LNOCCSDT_awCV TZ Molecule 39.0
CO MgO 1 LNOCCSDT_awCV DZ Surface 135.0
CO MgO 1 LNOCCSDT_awCV TZ Surface 391.0
CO MgO 2 LNOCCSDT_awCV DZ Molecule-Surface 4626.0
CO MgO 2 LNOCCSDT_awCV TZ Molecule-Surface 17164.0
CO MgO 2 LNOCCSDT_awCV DZ Molecule 91.0
CO MgO 2 LNOCCSDT_awCV TZ Molecule 165.0
CO MgO 2 LNOCCSDT_awCV DZ Surface 4465.0
CO MgO 2 LNOCCSDT_awCV TZ Surface 11491.0
CO MgO 3 LNOCCSDT_awCV DZ Molecule-Surface 12817.0
CO MgO 3 LNOCCSDT_awCV TZ Molecule-Surface 42113.0
CO MgO 3 LNOCCSDT_awCV DZ Molecule 134.0
CO MgO 3 LNOCCSDT_awCV TZ Molecule 396.0
CO MgO 3 LNOCCSDT_awCV DZ Surface 10977.0
CO MgO 3 LNOCCSDT_awCV TZ Surface 38028.0
H2O r-TiO2 1 LNOCCSDT_aV DZ Molecule-Surface 44890.0
H2O r-TiO2 1 LNOCCSDT_aV TZ Molecule-Surface 105741.0
H2O r-TiO2 1 LNOCCSDT_aV DZ Molecule 283.0
H2O r-TiO2 1 LNOCCSDT_aV TZ Molecule 677.0
H2O r-TiO2 1 LNO

# Insights into molecular binding

### CO<sub>2</sub> on MgO (001) - Is it physisorbed or chemisorbed?

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=450, sharey=True, constrained_layout=True) 

axs.bar(-2-0.25, autoskzcam_final_hads_dict['MgO']['CO2 Physisorbed']['Final'][0],yerr=autoskzcam_final_hads_dict['MgO']['CO2 Physisorbed']['Final'][1], width=0.5, color=color_dict['orange'], alpha=0.7)
axs.bar(-2+0.25, autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0],width=0.5,yerr=autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1], color=color_dict['green'], alpha=0.7)


for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
    axs.bar(idx_xc_func*2-0.25, dft_eads_true_dict['MgO']['CO2 Physisorbed'][xc_func] + autoskzcam_final_hads_dict['MgO']['CO2 Physisorbed']['DFT DeltaH'][0],width=0.5, color=color_dict['orange'],alpha=0.7) #label="Physisorbed" if idx_xc_func == 0 else "", alpha=0.7)
    axs.bar(idx_xc_func*2+0.25, dft_eads_true_dict['MgO']['CO2 Chemisorbed'][xc_func] + autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['DFT DeltaH'][0],width=0.5, color=color_dict['green'],alpha = 0.7) #label="Chemisorbed" if idx_xc_func == 0 else "", alpha=0.7)


# Give experimental values

physisorbed_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Physisorbed CO2'), 'Hads'].values[0])
physisorbed_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Physisorbed CO2'), 'Error'].values[0])
chemisorbed_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Chemisorbed CO2'), 'Hads'].values[0])
chemisorbed_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Chemisorbed CO2'), 'Error'].values[0])

axs.fill_between([-3,11], physisorbed_hads - physisorbed_error,  physisorbed_hads + physisorbed_error, color=color_dict['red'],alpha=0.1, edgecolor='none')
axs.plot([-3, 11], [physisorbed_hads,physisorbed_hads], '--',color=color_dict['red'],linewidth=0.7, alpha=0.5,label=r"Experiment (Meixner \textit{et al.})")

axs.fill_between([-3,11], chemisorbed_hads - chemisorbed_error, chemisorbed_hads + chemisorbed_error, color=color_dict['red'],alpha=0.2, edgecolor='none')
axs.plot([-3,11], [chemisorbed_hads,chemisorbed_hads], '--',color=color_dict['red'],linewidth=0.7, label=r"Experiment (Chakradhar and Burghaus)")


axs.set_xticks(np.arange(-2,11,2))
axs.set_xticklabels([r'\textbf{autoSKZCAM}'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['MgO']],rotation=90,ha='center')


# handles, labels = axs.get_legend_handles_labels()
# order = [2,3,1,0]  # Specify the new order
# axs.legend([handles[i] for i in order], [labels[i] for i in order],frameon=False, ncol=2,fontsize=7,loc='lower right')
axs.legend(frameon=False, fontsize=7.5,loc='lower right')
axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')
axs.xaxis.tick_top()

axs.set_xlim([-3,11])
axs.set_ylim([-1000,0])

plt.savefig('Figures/SI_Figure-Hads_CO2_Configurations.png')
plt.savefig('Figures/EXTENDED_Figure-Hads_CO2_Configurations.png')

In [ ]:
# Reanalyse Meixner et al. data with more appropriate temperature
print(f"autoSKZCAM Chemisorbed CO2                                              : {autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0]:.0f} +/- {autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1]:.0f}")
print(f"Meixner TPD experiment assuming (un-calibrated) T=120K                  : {physisorbed_hads:.0f} +/- {physisorbed_error:.0f}")
print(f"Meixner TPD experiment assuming T=~200K from Chakradhar and Burghaus   : {physisorbed_hads*200/120:.0f} +/- {physisorbed_error*200/120:.0f}")

autoSKZCAM Chemisorbed CO2                                              : -721 +/- 95
Meixner TPD experiment assuming (un-calibrated) T=120K                  : -426 +/- 48
Meixner TPD experiment assuming T=~200K from Chakradhar and Burghaus   : -710 +/- 80


### H<sub>2</sub>O and CH<sub>3</sub>OH on MgO (001) - How do these molecules bind on the MgO surface?

In [ ]:
# Analyse the revPBE-D4 CH3OH monomer, dimer, trimer and tetramer and H2O tetramer
ch3oh_cluster_systems = ['Monomer_Molecular','Monomer_Dissociated','Dimer_Molecular','Dimer_Dissociated','Trimer_Molecular','Trimer_Dissociated','Tetramer_Molecular','Tetramer_Dissociated']
h2o_cluster_systems = ['Tetramer_Molecular','Tetramer_Dissociated']

dft_ch3oh_cluster_hads = {system: {'Eads': 0 ,'DeltaH': 0, 'Hads':0} for system in ch3oh_cluster_systems}
dft_h2o_cluster_hads = {system: {'Eads': 0 ,'DeltaH': 0, 'Hads':0} for system in h2o_cluster_systems}

for system in ch3oh_cluster_systems:
    num_monomers = 4 if 'Tetramer' in system else 3 if 'Trimer' in system else 2 if 'Dimer' in system else 1
    dft_ch3oh_cluster_hads[system]['Eads'] = calculate_eint(f'Data/Miscellaneous/Dissociation_Tests/CH3OH', code='vasp', structure_labels=[system, "Surface", "Molecule"], num_monomers=num_monomers)*1000
    dft_ch3oh_cluster_hads[system]['Hads'] = dft_ch3oh_cluster_hads[system]['Eads'] + autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][0]

for system in h2o_cluster_systems:
    num_monomers = 4 if 'Tetramer' in system else 3 if 'Trimer' in system else 2 if 'Dimer' in system else 1
    dft_h2o_cluster_hads[system]['Eads'] = calculate_eint(f'Data/Miscellaneous/Dissociation_Tests/H2O', code='vasp', structure_labels=[system, "Surface", "Molecule"], num_monomers=num_monomers)*1000
    dft_h2o_cluster_hads[system]['Hads'] = dft_h2o_cluster_hads[system]['Eads'] + autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['DFT DeltaH'][0]    

#### Comparison of DFT performance and autoSKZCAM for monomers and clusters against experimental Hads

In [ ]:
# # Combine CH3OH and H2O clusters

# fig, axs = plt.subplots(1,2,figsize=(6.69,4.5),dpi=1200, sharey=True, constrained_layout=True, gridspec_kw={'width_ratios': [3,2.1]})

# x_list0 = np.arange(1,4)*2

# for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
#     xc_func_hads_parallel = dft_eads_true_dict['MgO']['CH3OH Parallel'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['DFT DeltaH'][0]
#     xc_func_hads_parallel_error = autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['DFT DeltaH'][1]
#     xc_func_hads_tilted = dft_eads_true_dict['MgO']['CH3OH Tilted'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['DFT DeltaH'][0]
#     xc_func_hads_tilted_error = autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['DFT DeltaH'][1]
#     xc_func_hads_tetramer = dft_eads_true_dict['MgO']['CH3OH Tetramer'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][0]
#     xc_func_hads_tetramer_error = autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][1]
#     axs[0].bar(x_list0 + 0.2*idx_xc_func, [xc_func_hads_parallel, xc_func_hads_tilted,xc_func_hads_tetramer], yerr= [xc_func_hads_parallel_error, xc_func_hads_tilted_error,xc_func_hads_tetramer_error], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2)
    

# axs[0].bar(x_list0 - 0.2, [autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][0]], yerr=[autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][1]],label='autoSKZCAM',width=0.2, color=color_dict['blue'], edgecolor='black',alpha=0.7)

# ch3oh_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Hads'].values[0])
# ch3oh_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Error'].values[0])

# axs[0].plot([0,9], [ch3oh_expt_hads, ch3oh_expt_hads], color=color_dict['red'], linestyle='--', label=r'Expt. $H_\textrm{ads}$')
# axs[0].fill_between([0,9], ch3oh_expt_hads - ch3oh_expt_error, ch3oh_expt_hads + ch3oh_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none')

# axs[0].set_xticks(x_list0 + 0.4)
# axs[0].set_xticklabels(['Parallel Monomer','Tilted Monomer','Tetramer'],rotation=90,ha='center')
# axs[0].set_ylabel(r'$H_\textrm{ads}$ [meV]')
# axs[0].xaxis.tick_top()
# axs[0].set_ylim(-1000,-400)
# axs[0].set_xlim([1.5,7.3])

# x_list1 = np.arange(1,3)*2

# for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
#     xc_func_hads_monomer = dft_eads_true_dict['MgO']['H2O Monomer'][xc_func] + autoskzcam_final_hads_dict['MgO']['H2O Monomer']['DFT DeltaH'][0]
#     xc_func_hads_tetramer = dft_eads_true_dict['MgO']['H2O Tetramer'][xc_func] + autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['DFT DeltaH'][0]
#     # axs[1].bar(x_list1 + 0.2*idx_xc_func, [xc_func_hads_monomer, xc_func_hads_tetramer], yerr=[autoskzcam_final_hads_dict['MgO']['H2O Monomer']['DFT DeltaH'][1],autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['DFT DeltaH'][1]], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2)
#     axs[1].bar(x_list1 + 0.2*idx_xc_func, [xc_func_hads_monomer, xc_func_hads_tetramer], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2)
# #     if xc_func == '02_revPBE-D4':
# #         print(xc_func_hads_monomer, xc_func_hads_tetramer)

# # print(autoskzcam_final_hads_dict['MgO']['H2O Monomer']['Final'][0], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][0])

# axs[1].bar(x_list1 - 0.2, [autoskzcam_final_hads_dict['MgO']['H2O Monomer']['Final'][0], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][0]], yerr=[autoskzcam_final_hads_dict['MgO']['H2O Monomer']['Final'][1], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][1]], label='autoSKZCAM',width=0.2, color='gray', edgecolor='black',alpha=0.7)

# h2o_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster H2O'), 'Hads'].values[0])
# h2o_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster H2O'), 'Error'].values[0])


# axs[1].plot([0,9], [h2o_expt_hads, h2o_expt_hads], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
# axs[1].fill_between([0,9], h2o_expt_hads - h2o_expt_error, h2o_expt_hads + h2o_expt_error, color=color_dict['black'],alpha=0.1, edgecolor='none')

# axs[1].set_xticks(x_list1 + 0.4)

# axs[1].set_xticklabels(['Monomer','Tetramer'],rotation=90,ha='center')
# # Set xlabels to the top
# axs[1].xaxis.tick_top()
# axs[1].legend(frameon=False,ncol=2,loc='lower left',fontsize=8)
# axs[1].set_ylim(-1100,-400)
# axs[1].set_xlim([1.5, 5.3])

# plt.savefig('Figures/SI_Figure-Hads_CH3OH_H2O_DFT_Benchmark.png')

In [ ]:
# CH3OH clusters calculations
fig, axs = plt.subplots(figsize=(6.69,4.5),dpi=1200, sharey=True, constrained_layout=True)

x_list0 = np.arange(1,5)*2


dft_color_list = ['#9b2226','#bb3e03','#ee9b00','#e9d8a6','#0a9396','#588157']# ['olive','magenta','lime','cyan','purple','teal']
delta_dissoc = dft_ch3oh_cluster_hads['Tetramer_Dissociated']['Hads'] - dft_ch3oh_cluster_hads['Tetramer_Molecular']['Hads']

for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
    xc_func_hads_parallel = dft_eads_true_dict['MgO']['CH3OH Parallel'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['DFT DeltaH'][0]
    xc_func_hads_parallel_error = autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['DFT DeltaH'][1]
    xc_func_hads_tilted = dft_eads_true_dict['MgO']['CH3OH Tilted'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['DFT DeltaH'][0]
    xc_func_hads_tilted_error = autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['DFT DeltaH'][1]
    xc_func_hads_tetramer = dft_eads_true_dict['MgO']['CH3OH Tetramer'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][0]
    xc_func_hads_tetramer_error = autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][1]
    xc_func_hads_tetramer_dissoc = dft_eads_true_dict['MgO']['CH3OH Tetramer'][xc_func] + autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][0] + delta_dissoc
    xc_func_hads_tetramer_dissoc_error = autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['DFT DeltaH'][1]    
    axs.bar(x_list0 + 0.2*idx_xc_func, [xc_func_hads_parallel, xc_func_hads_tilted,xc_func_hads_tetramer,xc_func_hads_tetramer_dissoc], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2,color=dft_color_list[idx_xc_func])
    

axs.bar(x_list0 - 0.2, [autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][0] + delta_dissoc], yerr=[autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][1],autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][1]],label='autoSKZCAM',width=0.2, color=color_dict['blue'], edgecolor='black')

ch3oh_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Hads'].values[0])
ch3oh_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Error'].values[0])

axs.plot([0,11], [ch3oh_expt_hads, ch3oh_expt_hads], color=color_dict['red'], linestyle='--', label=r'Expt. $H_\textrm{ads}$',zorder=2)
axs.fill_between([0,11], ch3oh_expt_hads - ch3oh_expt_error, ch3oh_expt_hads + ch3oh_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none',zorder=1)

axs.set_xticks(x_list0 + 0.4)
axs.set_xticklabels(['Parallel Monomer','Tilted Monomer','Tetramer', 'Dissociated Tetramer'],rotation=90,ha='center')
axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')
axs.xaxis.tick_top()
axs.set_ylim(-1100,-300)
axs.set_xlim([1.5,9.3])

# axs.legend(ncol=3,loc='lower left',fontsize=8,frameon=False)
axs.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=8, frameon=False)


plt.savefig('Figures/EXTENDED_Figure-Hads_CH3OH_DFT_Benchmark.png')

#### Comparison between monomers, clusters and dissociated clusters for CH3OH

In [ ]:
fig, axs = plt.subplots(figsize=(6.69,4.5),dpi=1200, sharey=True, constrained_layout=True)

x_list = np.array([1,2-0.37,3-0.19,4-0.185,5-0.23,6+0.03,7+0.03,8])

revpbed4_parallel_hads = dft_eads_true_dict['MgO']['CH3OH Parallel']['02_revPBE-D4'] + dft_ezpv_etherm_dict['MgO']['CH3OH Parallel']['02_revPBE-D4']['DeltaH']
revpbed4_tilted_hads = dft_eads_true_dict['MgO']['CH3OH Tilted']['02_revPBE-D4'] + dft_ezpv_etherm_dict['MgO']['CH3OH Tilted']['02_revPBE-D4']['DeltaH']

axs.bar(x_list, [revpbed4_parallel_hads, revpbed4_tilted_hads] + [dft_ch3oh_cluster_hads[system]['Hads'] for system in ['Dimer_Molecular','Trimer_Molecular','Tetramer_Molecular','Dimer_Dissociated','Trimer_Dissociated','Tetramer_Dissociated']], color=color_dict['grey'], width=0.3, alpha=0.9,label='revPBE-D4',facecolor=dft_color_list[1],edgecolor='black')
axs.errorbar(x_list[[0,1,4]], [autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][0], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][0]], yerr =  [autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][1], autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][1]], color=color_dict['blue'], alpha=0.7,label='autoSKZCAM', ls='none', marker='o', capsize=2)

ch3oh_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Hads'].values[0])
ch3oh_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster CH3OH'), 'Error'].values[0])

axs.plot([0,9], [ch3oh_expt_hads,ch3oh_expt_hads], color=color_dict['red'], linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([0,9], ch3oh_expt_hads - ch3oh_expt_error, ch3oh_expt_hads + ch3oh_expt_error, color=color_dict['red'],alpha=0.3, edgecolor='none')

axs.set_xlim([0.5,8.5])
axs.set_ylim(-1100,-400)
# Set xticklabels to the top
axs.xaxis.tick_top()
axs.set_xticks(x_list)
axs.set_xticklabels(['Parallel','Tilted','Dimer','Trimer','Tetramer','Dissociated Dimer','Dissociated Trimer','Dissociated Tetramer'],rotation=90)
axs.legend(ncol=3,fontsize=8, loc='lower right',frameon=False)

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')
plt.savefig('Figures/SI_Figure-Hads_CH3OH_Configurations.png')

#### Comparison between clusters and dissociated clusters for H2O

In [ ]:
# H2O clusters calculations
fig, axs = plt.subplots(figsize=(5.35,3.45),dpi=1200, sharey=True, constrained_layout=True)

x_list1 = np.arange(1,4)*2

dft_color_list = ['#9b2226','#bb3e03','#ee9b00','#e9d8a6','#0a9396','#588157']# ['olive','magenta','lime','cyan','purple','teal']
delta_dissoc = dft_h2o_cluster_hads['Tetramer_Dissociated']['Hads'] - dft_h2o_cluster_hads['Tetramer_Molecular']['Hads']

for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
    xc_func_hads_monomer = dft_eads_true_dict['MgO']['H2O Monomer'][xc_func] + autoskzcam_final_hads_dict['MgO']['H2O Monomer']['DFT DeltaH'][0]
    xc_func_hads_tetramer = dft_eads_true_dict['MgO']['H2O Tetramer'][xc_func] + autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['DFT DeltaH'][0]
    xc_func_hads_tetramer_dissoc = dft_eads_true_dict['MgO']['H2O Tetramer'][xc_func] + delta_dissoc + autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['DFT DeltaH'][0]

    axs.bar(x_list1 + 0.2*idx_xc_func, [xc_func_hads_monomer, xc_func_hads_tetramer,xc_func_hads_tetramer_dissoc], label=convert_to_nice_labels['xc_functionals'][xc_func],width=0.2,color=dft_color_list[idx_xc_func])

    

axs.bar(x_list1 - 0.2, [autoskzcam_final_hads_dict['MgO']['H2O Monomer']['Final'][0], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][0], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][0] + delta_dissoc], yerr=[autoskzcam_final_hads_dict['MgO']['H2O Monomer']['Final'][1], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][1], autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][1]], label='autoSKZCAM',width=0.2, color=color_dict['blue'], edgecolor='black')

h2o_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster H2O'), 'Hads'].values[0])
h2o_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster H2O'), 'Error'].values[0])


axs.plot([0,9], [h2o_expt_hads, h2o_expt_hads], color=color_dict['red'], linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([0,9], h2o_expt_hads - h2o_expt_error, h2o_expt_hads + h2o_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none')

axs.set_xticks(x_list1 + 0.4)

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')

axs.set_xticklabels(['Monomer','Tetramer','Dissociated Tetramer'],rotation=90,ha='center')
# Set xlabels to the top
axs.xaxis.tick_top()
axs.legend(frameon=False,ncol=2,loc='lower left',fontsize=8)
axs.set_ylim(-900,-400)
axs.set_xlim([1.5, 7.3])

# axs.legend(ncol=3,loc='lower left',fontsize=8,frameon=False)
axs.legend(ncol=1, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=8, frameon=False)


plt.savefig('Figures/EXTENDED_Figure-Hads_H2O_DFT_Benchmark.png')

In [ ]:
ch3oh_cluster_hads_dict = {'Parallel Monomer (autoSKZCAM)': int(round(autoskzcam_final_hads_dict['MgO']['CH3OH Parallel']['Final'][0])), 'Tilted Monomer (autoSKZCAM)': int(round(autoskzcam_final_hads_dict['MgO']['CH3OH Tilted']['Final'][0])), 'Molecular Tetramer (autoSKZCAM)': int(round(autoskzcam_final_hads_dict['MgO']['CH3OH Tetramer']['Final'][0]))}

ch3oh_cluster_hads_dict.update({'Parallel Monomer (revPBE-D4)': int(round(revpbed4_parallel_hads)), 'Tilted Monomer (revPBE-D4)': int(round(revpbed4_tilted_hads))})
ch3oh_cluster_hads_dict.update({f"{system.split('_')[1]} {system.split('_')[0]} (revPBE-D4)": int(round(dft_ch3oh_cluster_hads[system]['Hads'])) for system in ['Dimer_Molecular','Trimer_Molecular','Tetramer_Molecular','Dimer_Dissociated','Trimer_Dissociated','Tetramer_Dissociated']})
ch3oh_cluster_hads_dict.update({'Experiment': f'{int(round(ch3oh_expt_hads))} ± {int(round(ch3oh_expt_error))}'})

ch3oh_cluster_hads_df = pd.DataFrame.from_dict(ch3oh_cluster_hads_dict, orient='index', columns=['Hads [meV]'])
ch3oh_cluster_hads_df

,Hads [meV]
Parallel Monomer (autoSKZCAM),-462
Tilted Monomer (autoSKZCAM),-625
Molecular Tetramer (autoSKZCAM),-741
Parallel Monomer (revPBE-D4),-456
Tilted Monomer (revPBE-D4),-608
Molecular Dimer (revPBE-D4),-670
Molecular Trimer (revPBE-D4),-675
Molecular Tetramer (revPBE-D4),-795
Dissociated Dimer (revPBE-D4),-672
Dissociated Trimer (revPBE-D4),-718


In [ ]:
# Experimental H2O estimate

h2o_cluster_hads_dict = {'Molecular Tetramer (autoSKZCAM)': f"{autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][0]:.0f} ± {autoskzcam_final_hads_dict['MgO']['H2O Tetramer']['Final'][1]:.0f}", 'Molecular Tetramer (revPBE-D4)': f"{dft_h2o_cluster_hads['Tetramer_Molecular']['Hads']:.0f}", 'Dissociated Tetramer (revPBE-D4)': f"{dft_h2o_cluster_hads['Tetramer_Dissociated']['Hads']:.0f}", 'Experiment': f"{h2o_expt_hads:.0f} ± {h2o_expt_error:.0f}"}

h2o_cluster_hads_df = pd.DataFrame.from_dict(h2o_cluster_hads_dict, orient='index', columns=['Hads [meV]'])
display(h2o_cluster_hads_df)

,Hads [meV]
Molecular Tetramer (autoSKZCAM),-632 ± 20
Molecular Tetramer (revPBE-D4),-553
Dissociated Tetramer (revPBE-D4),-634
Experiment,-704 ± 75


### NO on MgO (001) - What is its adsorption configuration?

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3),dpi=1200, sharey=True, constrained_layout=True)

no_hads_xc_compare_dict = {struct: {xc_func: 0 for xc_func in ['autoSKZCAM'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['MgO']]} for struct in ['Dimer','Bent-Mg','Vertical-Mg','Bent-Bridge','Bent-O','Vertical-Hollow']}

no_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster NO'), 'Hads'].values[0])
no_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster NO'), 'Error'].values[0])

ticklabels = []

axs.fill_between([-1,13], no_expt_hads -no_expt_error, no_expt_hads + no_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none', label='Experiment')
axs.plot([-2, 14], [no_expt_hads,no_expt_hads], '--',color=color_dict['red'],linewidth=0.7, alpha = 0.5)

xc_func_marker_dict = {
    '01_PBE-D2-Ne' : 'o',
    '02_revPBE-D4' : '^',
    '03_vdW-DF' : 's',
    '04_rev-vdW-DF2' : 'o',
    '05_PBE0-D4' : 'P',
    '06_B3LYP-D2-Ne' : 's'
}


for i, struct in enumerate(['Dimer','Bent-Mg','Vertical-Mg','Bent-Bridge','Bent-O','Vertical-Hollow']):
    dft_no_hads_list = []
    for xc_func in crystal_xc_func_ensemble['MgO']:
        no_hads_xc_compare_dict[struct][convert_to_nice_labels['xc_functionals'][xc_func]] = f"{int(round(dft_eads_true_dict['MgO'][f'NO {struct}'][xc_func] + autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['DFT DeltaH'][0]))} ± {int(round(autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['DFT DeltaH'][1]))}"
        dft_no_hads_list += [dft_eads_true_dict['MgO'][f'NO {struct}'][xc_func] + autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['DFT DeltaH'][0] ] #- no_expt_hads]

    autoskzcam_no_hads = autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['Final'][0] #-  no_expt_hads
    autoskzcam_no_hads_err = autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['Final'][1]
    no_hads_xc_compare_dict[struct]['autoSKZCAM'] = f"{int(round(autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['Final'][0]))} ± {int(round(autoskzcam_no_hads_err))}"

    if struct == 'Dimer':
        axs.bar(i*2 + 0.25, np.ptp(dft_no_hads_list),bottom = np.min(dft_no_hads_list),width=0.5, color=color_dict['black'],alpha=0.2)
        axs.scatter([i*2 + 0.25]*len(dft_no_hads_list), dft_no_hads_list, marker='.', color=color_dict['black'],s=4, alpha=0.5)
        axs.scatter(i*2 + 0.25, dft_no_hads_list[5], marker='o', color=color_dict['black'],s=6, facecolor='none', label='B3LYP-D2-Ne')
        axs.scatter(i*2 + 0.25, dft_no_hads_list[3], marker='s', color=color_dict['black'],s=6, facecolor='none', label='rev-vdW-DF2')
        axs.plot([i*2-0.75,i*2-0.25], [autoskzcam_no_hads]*2, '-',color=color_dict['blue'],linewidth=1)
        axs.fill_between([i*2-0.75,i*2-0.25], autoskzcam_no_hads - autoskzcam_no_hads_err, autoskzcam_no_hads + autoskzcam_no_hads_err, color=color_dict['blue'],alpha=0.1, edgecolor='none',label='autoSKZCAM')
    else:
        axs.bar(i*2 + 0.25, np.ptp(dft_no_hads_list),bottom = np.min(dft_no_hads_list),width=0.5, color=color_dict['black'],alpha=0.2)
        axs.scatter([i*2 + 0.25]*len(dft_no_hads_list), dft_no_hads_list, marker='.', color=color_dict['black'],s=4, alpha=0.5)
        axs.scatter(i*2 + 0.25, dft_no_hads_list[5], marker='o', color=color_dict['black'],s=6, facecolor='none')
        axs.scatter(i*2 + 0.25, dft_no_hads_list[3], marker='s', color=color_dict['black'],s=6, facecolor='none')
        axs.plot([i*2-0.75,i*2-0.25], [autoskzcam_no_hads]*2, '-',color=color_dict['blue'],linewidth=1)
        axs.fill_between([i*2-0.75,i*2-0.25], autoskzcam_no_hads - autoskzcam_no_hads_err, autoskzcam_no_hads + autoskzcam_no_hads_err, color=color_dict['blue'],alpha=0.1, edgecolor='none')

    ticklabels += [struct]



axs.set_xticks(np.arange(0,11,2))
axs.set_xticklabels(ticklabels,rotation=90,ha='center')

handles, labels = plt.gca().get_legend_handles_labels()
custom_order = ['Experiment', 'autoSKZCAM', 'DFT range', 'B3LYP-D2-Ne', 'rev-vdW-DF2']
sorted_handles_labels = [(h, l) for l, h in sorted(zip(labels, handles), key=lambda pair: custom_order.index(pair[0]))]
handles, labels = zip(*sorted_handles_labels)

axs.legend(handles,labels,frameon=False, fontsize=7.5, ncols=2 , loc = 'lower right')
axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')

axs.set_xlim([-1,11])
axs.set_ylim([-800,200])
axs.set_yticks(np.arange(-800, 201, 50), minor=True)

plt.savefig('Figures/MAIN_Figure-Hads_NO_Configurations.png')

In [ ]:
no_hads_xc_compare_df = pd.DataFrame(no_hads_xc_compare_dict).T
display(no_hads_xc_compare_df)

# Write the DataFrame to a latex input
convert_df_to_latex_input(
    no_hads_xc_compare_df,
    start_input = r'\begin{table}',
    label = 'tab:no_configurations_dft_hads',
    caption = r'Comparison between autoSKZCAM and 6 DFT models in their predicted $H_\textrm{ads}$ for the different configurations of NO on MgO(001). The experimental $H_\textrm{ads}$ is $-235 {\pm} 29\,$meV.',
    end_input = r'\end{table}',
    filename = 'Tables/SI_Table-NO_Configurations_DFT_Hads.tex',
    rotate_column_header=True
)

,autoSKZCAM,PBE-D2[Ne],revPBE-D4,vdW-DF,rev-vdW-DF2,PBE0-D4,B3LYP-D2[Ne]
Dimer,-232 ± 30,-505 ± 5,-468 ± 5,-432 ± 5,-608 ± 5,-173 ± 5,-97 ± 5
Bent-Mg,-93 ± 13,-201 ± 3,-177 ± 3,-241 ± 3,-253 ± 3,-170 ± 3,-117 ± 3
Vertical-Mg,-25 ± 21,-159 ± 3,-130 ± 3,-189 ± 3,-205 ± 3,-121 ± 3,-73 ± 3
Bent-Bridge,24 ± 23,-291 ± 5,-284 ± 5,-289 ± 5,-348 ± 5,-182 ± 5,-126 ± 5
Bent-O,49 ± 22,-197 ± 3,-196 ± 3,-225 ± 3,-247 ± 3,-124 ± 3,-82 ± 3
Vertical-Hollow,90 ± 46,-178 ± 6,-148 ± 6,-181 ± 6,-220 ± 6,-95 ± 6,-63 ± 6


In [ ]:
fig, axs = plt.subplots(figsize=(6.695,3.3),dpi=1200, sharey=True, constrained_layout=True)

no_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster NO'), 'Hads'].values[0])
no_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Cluster NO'), 'Error'].values[0])

ticklabels = []

xc_func_color_list = ['#9b2226','#bb3e03','#ee9b00','#e9d8a6','#0a9396','#588157']

for i, struct in enumerate(['Dimer','Bent-Mg','Vertical-Mg','Bent-Bridge','Bent-O','Vertical-Hollow']):
    dft_no_hads_list = []
    dft_no_hads_error_list = []
    for xc_func_idx, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
        dft_hads = dft_eads_true_dict['MgO'][f'NO {struct}'][xc_func] + autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['DFT DeltaH'][0]
        dft_hads_error = autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['DFT DeltaH'][1]
        if i == 0:
            axs.bar(i*4 + xc_func_idx*0.5, dft_hads, width=0.5, color=xc_func_color_list[xc_func_idx], alpha=0.7, label=convert_to_nice_labels['xc_functionals'][xc_func])
        else:
            axs.bar(i*4 + xc_func_idx*0.5, dft_hads, width=0.5, color=xc_func_color_list[xc_func_idx], alpha=0.7)

    autoskzcam_no_hads = autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['Final'][0]
    autoskzcam_no_hads_err = autoskzcam_final_hads_dict['MgO'][f'NO {struct}']['Final'][1]
    if i == 0:
        axs.bar(i*4 -0.5, autoskzcam_no_hads, yerr=autoskzcam_no_hads_err, width=0.5, color=color_dict['blue'], edgecolor='black', label='autoSKZCAM')
    else:
        axs.bar(i*4 -0.5, autoskzcam_no_hads, yerr=autoskzcam_no_hads_err, width=0.5, color=color_dict['blue'], edgecolor='black')
        
    ticklabels += [struct]

axs.fill_between([-2,30], no_expt_hads-no_expt_error, no_expt_hads + no_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none', label='Experiment')
axs.plot([-2, 30], [no_expt_hads,no_expt_hads], '--',color=color_dict['red'],linewidth=0.7)
axs.plot([-2, 30], [0,0], '-',color=color_dict['black'],linewidth=1)

xtick_positions = [i * 4 + 1 for i in range(6)]
axs.set_xticks(xtick_positions)
axs.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
axs.spines['top'].set_position('zero')
axs.spines['bottom'].set_visible(False)

axs.set_xticklabels(ticklabels, ha='center')
for label in axs.get_xticklabels():
    label.set_y(label.get_position()[1] + 135)


axs.legend(frameon=False, fontsize=8, loc = 'lower right',ncol=2)
axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')

axs.set_xlim([-1,23])
axs.set_ylim([-600,200])

plt.savefig('Figures/SI_Figure-Hads_NO_Configurations.png')

### N<sub>2</sub>O on MgO (001) - Is it tilted or parallel on the surface?

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=600, sharey=True, constrained_layout=True)

autoskzcam_n2o_parallel_hads = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][0]
autoskzcam_n2o_parallel_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][1]
dft_xc_n2o_parallel_hads = [dft_eads_true_dict['MgO']['N2O Parallel'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_parallel_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]
autoskzcam_n2o_tilted_hads = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][0]
autoskzcam_n2o_tilted_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][1]
dft_xc_n2o_tilted_hads = [dft_eads_true_dict['MgO']['N2O Tilted'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_tilted_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]

n2o_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Hads'].values[0])
n2o_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Error'].values[0])


axs.bar(-2-0.25, autoskzcam_n2o_parallel_hads, yerr= autoskzcam_n2o_parallel_hads_error, width=0.5, color=color_dict['orange'],alpha=0.7)
axs.bar(-2+0.25, autoskzcam_n2o_tilted_hads, yerr= autoskzcam_n2o_tilted_hads_error,width=0.5, color=color_dict['green'],alpha=0.7)

for idx_xc_func, xc_func in enumerate(crystal_xc_func_ensemble['MgO']):
    axs.bar(idx_xc_func*2-0.25, dft_xc_n2o_parallel_hads[idx_xc_func], width=0.5, color=color_dict['orange'],alpha=0.7)
    axs.bar(idx_xc_func*2+0.25, dft_xc_n2o_tilted_hads[idx_xc_func],width=0.5, color=color_dict['green'],alpha=0.7)

axs.fill_between([-3,11], n2o_expt_hads - n2o_expt_error, n2o_expt_hads + n2o_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none')
axs.plot([-3,11], [n2o_expt_hads,n2o_expt_hads], '--',color=color_dict['red'],linewidth=0.7, label=r"Experiment")

axs.set_xticks(np.arange(-2,11,2))
axs.set_xticklabels([r'\textbf{autoSKZCAM}'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['MgO']],rotation=90,ha='center')

axs.legend(frameon=False, fontsize=7.5, ncol=3,loc = 'lower right')
axs.xaxis.tick_top()

axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')

axs.set_xlim([-3,11])
axs.set_ylim([-350,0])

plt.savefig('Figures/SI_Figure-Hads_N2O_Configurations.png')


### CO<sub>2</sub> on TiO<sub>2</sub> rutile (110) - Is it tilted or parallel on the surface?

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=1200, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})

autoskzcam_co2_parallel_hads = autoskzcam_final_hads_dict['r-TiO2']['CO2 Parallel']['Final'][0]
autoskzcam_co2_parallel_hads_error = autoskzcam_final_hads_dict['r-TiO2']['CO2 Parallel']['Final'][1]
dft_xc_co2_parallel_hads = [dft_eads_true_dict['r-TiO2']['CO2 Parallel'][xc_func] + autoskzcam_final_hads_dict['r-TiO2']['CO2 Parallel']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['r-TiO2']]
dft_xc_co2_parallel_hads_error = [autoskzcam_final_hads_dict['r-TiO2']['CO2 Parallel']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['r-TiO2']]
autoskzcam_co2_tilted_hads = autoskzcam_final_hads_dict['r-TiO2']['CO2 Tilted']['Final'][0]
autoskzcam_co2_tilted_hads_error = autoskzcam_final_hads_dict['r-TiO2']['CO2 Tilted']['Final'][1]
dft_xc_co2_tilted_hads = [dft_eads_true_dict['r-TiO2']['CO2 Tilted'][xc_func] + autoskzcam_final_hads_dict['r-TiO2']['CO2 Tilted']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['r-TiO2']]
dft_xc_co2_tilted_hads_error = [autoskzcam_final_hads_dict['r-TiO2']['CO2 Tilted']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['r-TiO2']]

rutile_co2_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'r-TiO2') & (expt_hads_df['Molecule'] == 'CO2'), 'Hads'].values[0])
rutile_co2_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'r-TiO2') & (expt_hads_df['Molecule'] == 'CO2'), 'Error'].values[0])

axs.bar(np.array(list(range(7)))*2-0.25-2, [autoskzcam_co2_parallel_hads] + dft_xc_co2_parallel_hads, yerr = [autoskzcam_co2_parallel_hads_error] + [0]*6, color=color_dict['orange'], alpha=0.7,width=0.5)
axs.bar(np.array(list(range(7)))*2+0.25-2, [autoskzcam_co2_tilted_hads] + dft_xc_co2_tilted_hads, yerr = [autoskzcam_co2_tilted_hads_error] + [0]*6, color=color_dict['green'], alpha=0.7,width=0.5)

axs.plot([-3,11], [rutile_co2_expt_hads, rutile_co2_expt_hads], color=color_dict['red'], linestyle='--', label=r'Experiment')
axs.fill_between([-3,11], rutile_co2_expt_hads - rutile_co2_expt_error, rutile_co2_expt_hads + rutile_co2_expt_error, color=color_dict['red'],alpha=0.1, edgecolor='none')


axs.set_xticks(np.arange(-2,11,2))
axs.set_xticklabels([r'\textbf{autoSKZCAM}'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['r-TiO2']],rotation=90,ha='center')
axs.legend(frameon=False, ncol=3, fontsize=7.5, loc='lower right')
axs.set_ylabel(r'Adsorption enthalpy $H_\textrm{ads}$ [meV]')
axs.set_ylim([-600,-300])
axs.set_xlim([-3,11])
axs.xaxis.tick_top()
plt.savefig('Figures/SI_Figure-Hads_rutile_CO2_Configurations.png')


co2_literature_citation_key = {'Sorescu et al.': r'~\cite{sorescuCoadsorptionPropertiesCO22012}', 'Kubas et al.': r'~\cite{kubasSurfaceAdsorptionEnergetics2016c}', 'Dohnalek et al.': r'~\cite{linStructureDynamicsCO22012}', 'This work': ''}

prev_co2_rutile_dft_literature = {
    'Sorescu et al.': {'Parallel': -371, 'Tilted': -389, 'Delta': -18},
    'Kubas et al.': {'Parallel': -446, 'Tilted': -397, 'Delta': 49},
    'Dohnalek et al.': {'Parallel': -420, 'Tilted': -450, 'Delta': -30},
    'This work': {'Parallel': -345, 'Tilted': -382, 'Delta': -37}
}

prev_co2_rutile_dft_literature_df = pd.DataFrame(prev_co2_rutile_dft_literature).T

display(prev_co2_rutile_dft_literature_df)

prev_co2_rutile_dft_literature_df = pd.DataFrame({authors + co2_literature_citation_key[authors]: prev_co2_rutile_dft_literature[authors] for authors in prev_co2_rutile_dft_literature}).T

convert_df_to_latex_input(
    prev_co2_rutile_dft_literature_df,
    start_input = r'\begin{table}',
    label = 'tab:co2_rutile_dft_literature',
    caption = r'Comparison of the $E_\textrm{ads}$ of the CO$_2$ on TiO$_2$ rutile (110) adsorption configurations from previous DFT studies for the PBE-D3 (with zero-damping).',
    end_input = r'\end{table}',
    replace_input = {
        'Delta': r'$\Delta$'
    },
    filename = 'Tables/SI_Table-CO2_Rutile_DFT_Literature.tex',
    rotate_column_header=True
)


,Parallel,Tilted,Delta
Sorescu et al.,-371,-389,-18
Kubas et al.,-446,-397,49
Dohnalek et al.,-420,-450,-30
This work,-345,-382,-37


# Comparison of accuracy and cost of DFT against autoSKZCAM

In [43]:
computational_cost_dict = { # in CPUh
    'CO2': {
        'GGA': 0,
        'hybrid': 0,
        'autoSKZCAM': 0
    },
    'H2O': {
        'GGA': 0,
        'hybrid': 0,
        'autoSKZCAM': 0
    }
}

for mol_surface in ['CO2','H2O']:
    for structure in ['Molecule-Surface', 'Surface','Molecule']:
        # Get the Periodic DFT computational cost
        computational_cost_dict[mol_surface]['GGA'] += get_vasp_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_GGA')*288/3600
        computational_cost_dict[mol_surface]['hybrid'] += get_vasp_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_GGA')*288/3600
        if mol_surface == 'CO2':
            computational_cost_dict[mol_surface]['hybrid'] += get_vasp_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_hybrid')*288/3600
        elif mol_surface == 'H2O' and structure == 'Molecule':
            computational_cost_dict[mol_surface]['hybrid'] += get_vasp_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_hybrid')*144/3600
        elif mol_surface == 'H2O' and structure == 'Molecule-Surface':
            computational_cost_dict[mol_surface]['hybrid'] += get_vasp_looptime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_hybrid')*24*144/3600
        elif mol_surface == 'H2O' and structure == 'Surface':
            computational_cost_dict[mol_surface]['hybrid'] += get_vasp_looptime(f'Data/Miscellaneous/autoSKZCAM_Cost/Periodic_DFT/{mol_surface}/{structure}/OUTCAR_hybrid')*25*144/3600
        # Get the cluster computational cost
        for cluster_number in range(1,8):
            for basis_set in ['aVDZ','aVTZ','aVQZ','awCVDZ','awCVTZ','awCVQZ']:
                # Check if the file exists
                if os.path.exists(f'Data/Miscellaneous/autoSKZCAM_Cost/autoSKZCAM/{mol_surface}/{cluster_number}/{structure}/orca_{basis_set}.out'):
                    try:
                        computational_cost_dict[mol_surface]['autoSKZCAM'] += get_orca_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/autoSKZCAM/{mol_surface}/{cluster_number}/{structure}/orca_{basis_set}.out')*8/3600                       
                    except:
                        print('ORCA',mol_surface, cluster_number, structure, basis_set)
                        pass
                if os.path.exists(f'Data/Miscellaneous/autoSKZCAM_Cost/autoSKZCAM/{mol_surface}/{cluster_number}/{structure}/mrcc_{basis_set}.out'):
                    try:
                        computational_cost_dict[mol_surface]['autoSKZCAM'] += get_mrcc_walltime(f'Data/Miscellaneous/autoSKZCAM_Cost/autoSKZCAM/{mol_surface}/{cluster_number}/{structure}/mrcc_{basis_set}.out')*8/3600 
                    except:
                        print('MRCC',mol_surface, cluster_number, structure, basis_set)
                        pass





In [58]:
co2_mgo_dft_literature_hads = [-1000,135,-680,-640,-380,-320,-90,-126,-120]
co2_mgo_cwft_literature_hads = [-492,870]
h2o_rutile_dft_literature_hads = [-442,-1638]
h2o_rutile_cwft_literature_hads = [-1492,-984,-964,-1390]

fig, axs = plt.subplots(2,2,figsize=(3.365,4),dpi=600, sharey='row', sharex = True, constrained_layout=True)

# Get experimental values for CO2 on MgO
chemisorbed_co2_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Chemisorbed CO2'), 'Hads'].values[0])
chemisorbed_co2_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'Chemisorbed CO2'), 'Error'].values[0])

# Plot dotted line for the deviation from the experimental value
axs[0,0].plot([0, 6], [0,0], color=color_dict['red'], linestyle='--',alpha=0.5,zorder=1, label='Expt. error')
axs[0,0].fill_between([0,6], -chemisorbed_co2_error, chemisorbed_co2_error, color=color_dict['red'], alpha=0.1,edgecolor='none',zorder=2)

# Plot the autoSKZCAM value for CO2 on MgO
# axs[0,0].plot([1.1,1.9], [autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads,autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads], color=color_dict['blue'], linestyle='-', label='autoSKZCAM')
# axs[0,0].bar([1.5], height = 2* autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1], bottom = autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads - autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1], color=color_dict['blue'], alpha=0.3,width=0.8)
axs[0,0].plot([0.6,1.4], [autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads,autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads], color=color_dict['blue'], linestyle='-', label='autoSKZCAM')
axs[0,0].bar([1], height = 2* autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1], bottom = autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][0] - chemisorbed_co2_hads - autoskzcam_final_hads_dict['MgO']['CO2 Chemisorbed']['Final'][1], color=color_dict['blue'], alpha=0.3,width=0.8)

# Plot range of DFT values for CO2 on MgO
# axs[0,0].bar(3.7, height = np.max(co2_mgo_dft_literature_hads) - np.min(co2_mgo_dft_literature_hads) , color=color_dict['black'], bottom = np.min(co2_mgo_dft_literature_hads) - chemisorbed_co2_hads, width=0.8,alpha=0.4, label = 'DFT literature')
axs[0,0].bar(2.5, height = np.max(co2_mgo_dft_literature_hads) - np.min(co2_mgo_dft_literature_hads) , color=color_dict['black'], bottom = np.min(co2_mgo_dft_literature_hads) - chemisorbed_co2_hads, width=0.8,alpha=0.4, label = 'DFT literature')

# Plot range of cWFT values for CO2 on MgO
axs[0,0].bar(4, height = np.max(co2_mgo_cwft_literature_hads) - np.min(co2_mgo_cwft_literature_hads) , color=color_dict['yellow'], bottom = np.min(co2_mgo_cwft_literature_hads) - chemisorbed_co2_hads, width=0.8,alpha=0.6, label = 'cWFT literature')
# An arrow pointing at top of the bar indicating maximum error
axs[0, 0].annotate(
    f'{int(np.max(co2_mgo_cwft_literature_hads) - chemisorbed_co2_hads)}',
    xy=(4, 1000),
    xytext=(4, 600),
    arrowprops=dict(
        arrowstyle='-|>',  # Makes the arrowhead more distinct
        lw=1.5,           # Increases the line width for better visibility
        color='black',     # Sets the arrow color for better contrast
    ),
    fontsize=8,          # Increases font size for better readability
    color='black',        # Ensures the text color stands out
    ha='center',          # Aligns the text horizontally
    va='center'           # Aligns the text vertically
)

# np.max(co2_mgo_cwft_literature_hads) - chemisorbed_co2_hads),

# Get experimental values for H2O on rutile
rutile_h2o_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'r-TiO2') & (expt_hads_df['Molecule'] == 'H2O'), 'Hads'].values[0])
rutile_h2o_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'r-TiO2') & (expt_hads_df['Molecule'] == 'H2O'), 'Error'].values[0])

# # Plot range of cWFT values for H2O on rutile
# axs[0,0].bar(6, height = np.max(h2o_rutile_cwft_literature_hads) - np.min(h2o_rutile_cwft_literature_hads) , color=color_dict['yellow'], bottom = np.min(h2o_rutile_cwft_literature_hads) - rutile_h2o_hads, width=0.8,alpha=0.6, label = 'cWFT literature')


# Plot dotted line for the deviation from the experimental value
axs[0,1].plot([-1,9], [0,0], color=color_dict['red'],alpha=0.5, linestyle='--',zorder=1)
axs[0,1].fill_between([-1,9], -rutile_h2o_error, rutile_h2o_error, color=color_dict['red'], alpha=0.1,edgecolor='none',zorder=2)

# Plot the autoSKZCAM value for H2O on rutile
axs[0,1].plot([0.6,1.4], [autoskzcam_final_hads_dict['r-TiO2']['H2O']['Final'][0] - rutile_h2o_hads,autoskzcam_final_hads_dict['r-TiO2']['H2O']['Final'][0] - rutile_h2o_hads], color=color_dict['blue'], linestyle='-')
axs[0,1].bar([1], height = 2* autoskzcam_final_hads_dict['r-TiO2']['H2O']['Final'][1], bottom = autoskzcam_final_hads_dict['r-TiO2']['H2O']['Final'][0] - rutile_h2o_hads - autoskzcam_final_hads_dict['r-TiO2']['H2O']['Final'][1], color=color_dict['blue'], alpha=0.3, label='autoSKZCAM',width=0.8)

# Plot range of DFT values for H2O on rutile
axs[0,1].bar(2.5, height = np.max(h2o_rutile_dft_literature_hads) - np.min(h2o_rutile_dft_literature_hads) , color=color_dict['black'], bottom = np.min(h2o_rutile_dft_literature_hads) - rutile_h2o_hads, width=0.8,alpha=0.4, label = 'DFT literature')

# Plot range of cWFT values for H2O on rutile
axs[0,1].bar(4, height = np.max(h2o_rutile_cwft_literature_hads) - np.min(h2o_rutile_cwft_literature_hads) , color=color_dict['yellow'], bottom = np.min(h2o_rutile_cwft_literature_hads) - rutile_h2o_hads, width=0.8,alpha=0.6, label = 'cWFT literature')



# Plot the costs for CO2 on MgO as a bar graph
axs[1,0].bar(4, computational_cost_dict['CO2']['GGA'], color=color_dict['black'], label='GGA-DFT', width=0.8, alpha=0.4)
axs[1,0].bar(2.5, computational_cost_dict['CO2']['hybrid'], color=color_dict['black'], label='hybrid-DFT', width=0.8, alpha=0.4)
axs[1,0].bar(1, computational_cost_dict['CO2']['autoSKZCAM'], color=color_dict['blue'], width=0.8, alpha=0.6,edgecolor='none')
axs[1,0].bar(1, computational_cost_dict['CO2']['autoSKZCAM'], color=color_dict['black'], label='autoSKZCAM',edgecolor='k',facecolor='none')


# Plot the costs for H2O on rutile as a bar graph
axs[1,1].bar(4, computational_cost_dict['H2O']['GGA'], color=color_dict['black'], label='DFT-GGA', width=0.8, alpha=0.4)
bars1 = axs[1,1].bar(2.5, computational_cost_dict['H2O']['hybrid'], color=color_dict['black'], label='hybrid', width=0.8, alpha=0.4,edgecolor='k')
axs[1,1].bar(1, computational_cost_dict['CO2']['autoSKZCAM'], color=color_dict['blue'], width=0.8, alpha=0.6,edgecolor='none')
axs[1,1].bar(1, computational_cost_dict['CO2']['autoSKZCAM'], color=color_dict['blue'], label='autoSKZCAM', width=0.8, facecolor='none',edgecolor='k')



hatches = ['/'] #, '\\', 'x']  # Example hatch patterns


# Add figure labels to the plot
# axs[0,0].text(-0.2, 1.1, '(a)', transform=axs[0,0].transAxes, fontsize=9, fontweight='bold', va='top')
# axs[1,0].text(-0.2, 1.0, '(b)', transform=axs[1,0].transAxes, fontsize=9, fontweight='bold', va='top')


axs[0,0].set_title(r'CO$_2$ on MgO(001)',fontsize=9)
axs[0,1].set_title(r'H$_2$O on TiO$_2$ rutile(110)',fontsize=9)


axs[0,0].set_xticks([])
axs[1,0].set_xticks([])

axs[1,0].set_yscale('log')

axs[1,0].legend(fontsize=7,frameon=False,loc='upper right')
axs[0,0].legend(fontsize=7,frameon=False,loc='lower left')


axs[0,0].set_ylabel(r'Deviation from expt. $H_\textrm{ads}$ [meV]')
axs[1,0].set_ylabel(r'Computational cost [CPUh]')

axs[0,0].set_xlim([0,5])
axs[0,0].set_ylim([-1000,1000])


plt.savefig('Figures/MAIN_Figure-CO2_H2O_Accuracy_Cost.png')

# Miscellaneous

### Plot schematic TPD curve

In [586]:
# Plot the TPD curves

# Pandas DataFrame containing the TPD data
tpd_schematic_df = pd.read_csv("Data/Miscellaneous/TPD_Schematic_Data.csv")

# Make the header a row then set new header  'Temperature (K)' followed by 0.1 to 1.0 in 0.1 increments
tpd_schematic_df.columns = ['Temperature (K)'] + [f'{i:.1f}' for i in np.arange(0.1, 1.1, 0.1)]

# Plot each of the columns with the first column as x and the rest as y
fig, axs = plt.subplots(figsize=(3.365, 2.5),dpi=450,constrained_layout=True)
for column in tpd_schematic_df.columns[1:]:
    axs.plot(tpd_schematic_df['Temperature (K)'], tpd_schematic_df[column], label=r'$\theta{=}$' + column)

axs.legend(loc='upper left',fontsize=8)
axs.set_xlabel('Temperature (K)')
axs.set_ylabel(r'Desorption rate ${-}d\theta/dt$ (ML/s)')

axs.set_xlim([55,76])
axs.set_yticks([])
axs.set_xticks([70])
axs.set_xticklabels([r'T$_{p}$'])

plt.savefig('Figures/SI_Figure-TPD_Schematic.png')
